In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import timedelta
import os
import statistics
import math
from scipy.special import logsumexp
from ipynb.fs.full.Func_Reformating_Roku_Data import Rd_Cl

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
# ******************************
# Comments

# - Obtjective for this code is to 
#         a) Merge the Master (The Schedule for each channel with content) with hourly performance Amagi Data,
#         since downloading Amagi data with content has been deemed unreliable
#         b) Use statistice and Data Analysis to find actionble KPI's
#         c) Build out excel report
        
# - First started writing this code over a year ago so some of these steps i might have done differently if i would have done it today

# *******************************

In [4]:
pd.set_option('display.max_rows',500)
pd.set_option('display.max_columns', None)

In [5]:
# All Channels   (CONtv, ComedyDynamics, Docurama, Dove, ElRey, BloodyDisgusting, SoReal,Towie, LoneStar, FilmDetective, RetroCrush)

In [6]:
Channel= 'ComedyDynamics'
EOM= '2023-04-30'

# ElREy and Towie '_ET'
if (Channel == 'CONtv' or  Channel == 'ComedyDynamics' or Channel == 'Docurama' or Channel == 'Dove' or Channel == 'BloodyDisgusting' or Channel == 'SoReal'):
    ET=''
    Timezone= 'West'
else:
    ET='_ET'
    Timezone= 'East'
    
filepathM='C:/Users/sortiz/Box/Digital Networks/REPORTING + ANALYTICS/Linear/Channel Masters glossary/Masters Clean/' +Channel+' Masters/'
filepathA='C:/Users/sortiz/Box/Digital Networks/REPORTING + ANALYTICS/Linear/Amagi DB/Amagi_hourly_noContent'+ET+'/'
    
glossary_channel =pd.read_excel('C:/Users/sortiz/Box/Digital Networks/REPORTING + ANALYTICS/Linear/Amagi DB/Mapping Amagi_Channel_Platform.xlsx')

In [7]:
# **********************************
# ISSUES 1

# -Different Month lists need to be made because i have different amount of Masters for each Month
# -Some Channels have Masters made in Eastern some in Pacific Time

# **********************************

In [8]:
if (Channel == 'CONtv' or  Channel == 'ComedyDynamics' or Channel == 'Docurama' or Channel == 'Dove'):
    #Dates
    Date_span= 'Jul21_Mar23'
    M_list = {'07_2021':'2021','08_2021':'2021','09_2021':'2021','10_2021':'2021','11_2021':'2021','12_2021':'2021',
              '01_2022':'2022','02_2022':'2022','03_2022':'2022','04_2022':'2022','05_2022':'2022','06_2022':'2022','07_2022':'2022','08_2022':'2022','09_2022':'2022','10_2022':'2022','11_2022':'2022','12_2022':'2022',
              '01_2023':'2023','02_2023':'2023','03_2023':'2023','04_2023':'2023'}
    
elif (Channel == 'ElRey' or Channel == 'SoReal'):
    #Dates
    Date_span= 'Oct21_Mar23'    
    M_list = {'10_2021':'2021','11_2021':'2021','12_2021':'2021',
              '01_2022':'2022','02_2022':'2022','03_2022':'2022','04_2022':'2022','05_2022':'2022','06_2022':'2022','07_2022':'2022','08_2022':'2022','09_2022':'2022','10_2022':'2022','11_2022':'2022','12_2022':'2022',
              '01_2023':'2023','02_2023':'2023','03_2023':'2023','04_2023':'2023'}
    
elif (Channel == 'BloodyDisgusting' or Channel == 'Towie'):
    #Dates
    Date_span= 'Jan22_Mar23'
    M_list = {'01_2022':'2022','02_2022':'2022','03_2022':'2022','04_2022':'2022','05_2022':'2022','06_2022':'2022','07_2022':'2022','08_2022':'2022','09_2022':'2022','10_2022':'2022','11_2022':'2022','12_2022':'2022',
              '01_2023':'2023','02_2023':'2023','03_2023':'2023','04_2023':'2023'}
    
elif (Channel == 'RetroCrush'):
    #Dates
    Date_span= 'Jan23_Mar23'
    M_list = {'01_2023':'2023','02_2023':'2023','03_2023':'2023','04_2023':'2023'}
    
elif (Channel == 'AsianCrush'):
    #Dates
    Date_span= 'Jan23_Mar23'
    M_list = {'03_2023':'2023','04_2023':'2023'}
    
else:
    #LoneStar and FilmDetective
    #Dates
    Date_span= 'Aug22_Mar23'
    M_list = {'08_2022':'2022','09_2022':'2022','10_2022':'2022','11_2022':'2022','12_2022':'2022',
              '01_2023':'2023','02_2023':'2023','03_2023':'2023','04_2023':'2023'}

In [9]:
M_L=[]
# for i in M_list:
#     M_L = M_L + [filepathM + 'Master '+ Channel + ' - ' + i + '.xlsx']
for month,year in M_list.items():
    M_L = M_L + [filepathM + year + '/' +'Master '+ Channel + ' - ' + month + '.xlsx']    
    
Amg_L=[]
# for i in M_list:
#     Amg_L = Amg_L + [filepathA + i + '_hourly_noContent' + ET + '.csv']
for month,year in M_list.items():
    Amg_L = Amg_L + [filepathA + year + '/' + 'Amagi_hourly_noContent_' + month + ET + '.csv']

In [10]:
Master = pd.concat([pd.read_excel(file) for file in M_L])
glossary_content = pd.concat([pd.read_excel(file, sheet_name= 'Glossary') for file in M_L])
Amagi = pd.concat([pd.read_csv(file) for file in Amg_L])

In [11]:
Amagi['Date']= pd.to_datetime(Amagi['Date'])

In [12]:
Amagi

,Date,Channel Name,Session Count,Total Viewership Minutes,Unique Viewers,Avg Session Count,Avg Session Duration Per Session,Avg Session Duration Per Viewer
0,2021-07-01 00:00:00,bloodydisgusting_littlstar,1,60.00,1,NaN,NaN,NaN
1,2021-07-01 00:00:00,BloodyDisgusting_OandO,1,60.00,1,NaN,NaN,NaN
2,2021-07-01 00:00:00,Bloody Disgusting Roku US,1728,22318.00,1312,NaN,NaN,NaN
3,2021-07-01 00:00:00,bobross_plex,39,1337.00,30,NaN,NaN,NaN
4,2021-07-01 00:00:00,Bob Ross Roku US,1742,63913.00,1541,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
39797,2023-04-30 23:00:00,RetroCrush LG US,688,2914.45,562,1.22,4.24,5.19
39798,2023-04-30 23:00:00,RetroCrush Plex,23,571.20,22,1.05,24.83,25.96
39799,2023-04-30 23:00:00,RetroCrush Vizio US,103,3685.82,99,1.04,35.78,37.23
39800,2023-04-30 23:00:00,soreal LG CA,63,260.70,44,1.43,4.14,5.93


In [13]:
Master

,Unnamed: 0,HOUR,SEGMENT ID,ASSET ID,SEASON ID,SEASON TITLE,ASSET ID.1,TYPE,TITLE,YEAR,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Seg,ASSET
0,1,2021-07-01 00:00:00:00,NaN,CD10010210,NaN,NaN,TomGreenLiveClean_Film,feature_film,Tom Green: Live! - Clean,2012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,2021-07-01 01:05:41:08,NaN,CD10010171,NaN,NaN,DaveFoleyRelativelyWellClean_Film,feature_film,Dave Foley: Relatively Well - Clean,2013,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,2021-07-01 02:27:05:07,NaN,CEG10027749,NaN,NaN,HarlandWilliamsWhatATreatClean_Film,feature_film,Harland Williams: What A Treat - Clean,2005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,2021-07-01 03:45:00:13,NaN,CEG10032516,NaN,NaN,NeverBeDoneTheRichardGlenLett_Film,feature_film,Never Be Done: The Richard Glen Lett Story - C...,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,2021-07-01 05:22:18:20,NaN,CEG10030236,NaN,NaN,HarlandWilliamsAForceofNature_Film,feature_film,Harland Williams: A Force of Nature - Clean,2005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
598,599,2023-04-30 19:08:29:29,NaN,1000000003242,NaN,NaN,1000000003242,feature_film,Phil Kopczinski: Full Grown Man,2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
599,600,2023-04-30 19:36:14:04,NaN,CD10010200,NaN,NaN,RobSchneiderSoySauceandtheHol_Film,feature_film,Rob Schneider: Soy Sauce and the Holocaust - C...,2013,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
600,601,2023-04-30 20:48:32:06,NaN,CD10010178,NaN,NaN,IlizaShlesingerWarPaintClean_Film,feature_film,Iliza Shlesinger: War Paint - Clean,2013,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
601,602,2023-04-30 22:15:57:04,NaN,CEG10027755,NaN,NaN,DougBensonDougDynastyClean_Film,feature_film,Doug Benson: Doug Dynasty - Clean,2014,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
# ******************************************

# ISSUES 2

# - Masters need to be Cleaned
#     a) some of the titles in the masters change from master to master
#     b) some of the masters only have episode names and not Series names
#     c) all master titles are tied to an ID
#     d) all masters are cleaned seperately in excel and then merged with existing Master
    
# ******************************************

In [15]:
Master= Master.loc[:,('HOUR','ASSET ID')]
Master.rename(columns={'ASSET ID':'Content ID'},inplace=True)

glossary_content = glossary_content.loc[:,('ID', 'TITLE', 'Series/Film')]
glossary_content.rename(columns= {'ID': 'Content ID'}, inplace=True)

In [16]:
glossary_content['TITLE'] = glossary_content.groupby(['Content ID'])['TITLE'].transform('last')

In [17]:
glossary_content = glossary_content.drop_duplicates()

In [18]:
Master = pd.merge(Master, glossary_content, on = 'Content ID', how='left')
Master.rename(columns= {'TITLE':'Title'}, inplace=True)

In [19]:
Master['TS']=pd.to_datetime(Master['HOUR'],format = '%Y-%m-%d %H:%M:%S:%f')

In [20]:
Master= Master.loc[:,('TS', 'Content ID', 'Title', 'Series/Film')]
Master['Hour_Start']= Master['TS'].dt.hour
Master['Minutes_Start']=Master['TS'].dt.minute
Master['Length']= Master['TS'].shift(-1) - Master['TS']
Master['TS End']= Master['TS'].shift(-1)
Master['Hour_End']= Master['TS End'].dt.hour
Master['Minutes_End']= Master['TS End'].dt.minute


In [21]:
# Have to transform TITLE after they have been merged

Master['TITLE'] = Master.groupby(['Content ID'])['Title'].transform('last')

In [22]:
Master

,TS,Content ID,Title,Series/Film,Hour_Start,Minutes_Start,Length,TS End,Hour_End,Minutes_End,TITLE
0,2021-07-01 00:00:00.000,CD10010210,Tom Green: Live! - Clean,Film,0,0,0 days 01:05:41.080000,2021-07-01 01:05:41.080,1.0,5.0,Tom Green: Live! - Clean
1,2021-07-01 01:05:41.080,CD10010171,Dave Foley: Relatively Well - Clean,Film,1,5,0 days 01:21:23.990000,2021-07-01 02:27:05.070,2.0,27.0,Dave Foley: Relatively Well - Clean
2,2021-07-01 02:27:05.070,CEG10027749,Harland Williams: What A Treat - Clean,Film,2,27,0 days 01:17:55.060000,2021-07-01 03:45:00.130,3.0,45.0,Harland Williams: What A Treat - Clean
3,2021-07-01 03:45:00.130,CEG10032516,Never Be Done: The Richard Glen Lett Story,Film,3,45,0 days 01:37:18.070000,2021-07-01 05:22:18.200,5.0,22.0,Never Be Done: The Richard Glen Lett Story
4,2021-07-01 05:22:18.200,CEG10030236,Harland Williams: A Force of Nature,Film,5,22,0 days 01:05:07.930000,2021-07-01 06:27:26.130,6.0,27.0,Harland Williams: A Force of Nature
...,...,...,...,...,...,...,...,...,...,...,...
15381,2023-04-30 19:08:29.290,1000000003242,Phil Kopczinski: Full Grown Man,Film,19,8,0 days 00:27:44.750000,2023-04-30 19:36:14.040,19.0,36.0,Phil Kopczinski: Full Grown Man
15382,2023-04-30 19:36:14.040,CD10010200,Rob Schneider: Soy Sauce and the Holocaust - C...,Film,19,36,0 days 01:12:18.020000,2023-04-30 20:48:32.060,20.0,48.0,Rob Schneider: Soy Sauce and the Holocaust - C...
15383,2023-04-30 20:48:32.060,CD10010178,Iliza Shlesinger: War Paint - Clean,Film,20,48,0 days 01:27:24.980000,2023-04-30 22:15:57.040,22.0,15.0,Iliza Shlesinger: War Paint - Clean
15384,2023-04-30 22:15:57.040,CEG10027755,Doug Benson: Doug Dynasty - Clean,Film,22,15,0 days 01:12:41.230000,2023-04-30 23:28:38.270,23.0,28.0,Doug Benson: Doug Dynasty - Clean


In [23]:
# ******************************************

# ISSUES 3

# - Merging of Masters and Amagi data is difficult because 
#     a) Amagi data is broken down by HOUR and Masters are broken down by a timestamp that gets specific to the Minute

    
# ******************************************


# Fixing last row

In [24]:
Master.iloc[-1, 7]= EOM + ' 23:59:59.27'
Master['TS End']= pd.to_datetime(Master['TS End'], format = '%Y-%m-%d %H:%M:%S')
Master.iloc[-1,6]= Master.iloc[-1,7]- Master.iloc[-1,0]
Master.iloc[-1,8]= 23
Master.iloc[-1,9]= 59
Master['Hour_End']= Master['Hour_End'].astype(int)

In [25]:
Master

,TS,Content ID,Title,Series/Film,Hour_Start,Minutes_Start,Length,TS End,Hour_End,Minutes_End,TITLE
0,2021-07-01 00:00:00.000,CD10010210,Tom Green: Live! - Clean,Film,0,0,0 days 01:05:41.080000,2021-07-01 01:05:41.080,1,5.0,Tom Green: Live! - Clean
1,2021-07-01 01:05:41.080,CD10010171,Dave Foley: Relatively Well - Clean,Film,1,5,0 days 01:21:23.990000,2021-07-01 02:27:05.070,2,27.0,Dave Foley: Relatively Well - Clean
2,2021-07-01 02:27:05.070,CEG10027749,Harland Williams: What A Treat - Clean,Film,2,27,0 days 01:17:55.060000,2021-07-01 03:45:00.130,3,45.0,Harland Williams: What A Treat - Clean
3,2021-07-01 03:45:00.130,CEG10032516,Never Be Done: The Richard Glen Lett Story,Film,3,45,0 days 01:37:18.070000,2021-07-01 05:22:18.200,5,22.0,Never Be Done: The Richard Glen Lett Story
4,2021-07-01 05:22:18.200,CEG10030236,Harland Williams: A Force of Nature,Film,5,22,0 days 01:05:07.930000,2021-07-01 06:27:26.130,6,27.0,Harland Williams: A Force of Nature
...,...,...,...,...,...,...,...,...,...,...,...
15381,2023-04-30 19:08:29.290,1000000003242,Phil Kopczinski: Full Grown Man,Film,19,8,0 days 00:27:44.750000,2023-04-30 19:36:14.040,19,36.0,Phil Kopczinski: Full Grown Man
15382,2023-04-30 19:36:14.040,CD10010200,Rob Schneider: Soy Sauce and the Holocaust - C...,Film,19,36,0 days 01:12:18.020000,2023-04-30 20:48:32.060,20,48.0,Rob Schneider: Soy Sauce and the Holocaust - C...
15383,2023-04-30 20:48:32.060,CD10010178,Iliza Shlesinger: War Paint - Clean,Film,20,48,0 days 01:27:24.980000,2023-04-30 22:15:57.040,22,15.0,Iliza Shlesinger: War Paint - Clean
15384,2023-04-30 22:15:57.040,CEG10027755,Doug Benson: Doug Dynasty - Clean,Film,22,15,0 days 01:12:41.230000,2023-04-30 23:28:38.270,23,28.0,Doug Benson: Doug Dynasty - Clean


# Adding Middle Hours

In [26]:
#When Lenght crosses 3 hours
Master.loc[Master['Hour_End']== Master['Hour_Start']+2 , 'Middle_1_Hour'] = Master['Hour_Start']+1
#exeptions
Master.loc[(Master['Hour_Start']== 22) & (Master['Hour_End']==0), 'Middle_1_Hour'] = 23

Master.loc[(Master['Hour_Start']== 23) & (Master['Hour_End']==1), 'Middle_1_Hour'] = 0


#When Legth crosses 4 hours
Master.loc[Master['Hour_End']== Master['Hour_Start']+3, 'Middle_2_Hour'] = Master['Hour_Start']+2
Master.loc[Master['Hour_End']== Master['Hour_Start']+3, 'Middle_1_Hour'] = Master['Hour_Start']+1
#exeptions
Master.loc[(Master['Hour_Start']== 21) & (Master['Hour_End']==0), 'Middle_2_Hour']=23
Master.loc[(Master['Hour_Start']== 21) & (Master['Hour_End']==0), 'Middle_1_Hour']=22

Master.loc[(Master['Hour_Start']== 22) & (Master['Hour_End']==1), 'Middle_2_Hour']=0
Master.loc[(Master['Hour_Start']== 22) & (Master['Hour_End']==1), 'Middle_1_Hour']=23

Master.loc[(Master['Hour_Start']== 23) & (Master['Hour_End']==2), 'Middle_2_Hour']=1
Master.loc[(Master['Hour_Start']== 23) & (Master['Hour_End']==2), 'Middle_1_Hour']=0


#when Length crosses 5 hours (ex. 3hr15min film starts at 6:50 ends at 10:05, thats 6,7,8,9,10)
Master.loc[Master['Hour_End']== Master['Hour_Start']+4, 'Middle_3_Hour'] = Master['Hour_Start']+3
Master.loc[Master['Hour_End']== Master['Hour_Start']+4, 'Middle_2_Hour'] = Master['Hour_Start']+2
Master.loc[Master['Hour_End']== Master['Hour_Start']+4, 'Middle_1_Hour'] = Master['Hour_Start']+1
#exeptions
Master.loc[(Master['Hour_Start']== 20) & (Master['Hour_End']==0), 'Middle_3_Hour']=23
Master.loc[(Master['Hour_Start']== 20) & (Master['Hour_End']==0), 'Middle_2_Hour']=22
Master.loc[(Master['Hour_Start']== 20) & (Master['Hour_End']==0), 'Middle_1_Hour']=21

Master.loc[(Master['Hour_Start']== 21) & (Master['Hour_End']==1), 'Middle_3_Hour']=0
Master.loc[(Master['Hour_Start']== 21) & (Master['Hour_End']==1), 'Middle_2_Hour']=23
Master.loc[(Master['Hour_Start']== 21) & (Master['Hour_End']==1), 'Middle_1_Hour']=22

Master.loc[(Master['Hour_Start']== 22) & (Master['Hour_End']==2), 'Middle_3_Hour']=1
Master.loc[(Master['Hour_Start']== 22) & (Master['Hour_End']==2), 'Middle_2_Hour']=0
Master.loc[(Master['Hour_Start']== 22) & (Master['Hour_End']==2), 'Middle_1_Hour']=23

Master.loc[(Master['Hour_Start']== 23) & (Master['Hour_End']==3), 'Middle_3_Hour']=2
Master.loc[(Master['Hour_Start']== 23) & (Master['Hour_End']==3), 'Middle_2_Hour']=1
Master.loc[(Master['Hour_Start']== 23) & (Master['Hour_End']==3), 'Middle_1_Hour']=0

In [27]:
Master

,TS,Content ID,Title,Series/Film,Hour_Start,Minutes_Start,Length,TS End,Hour_End,Minutes_End,TITLE,Middle_1_Hour,Middle_2_Hour,Middle_3_Hour
0,2021-07-01 00:00:00.000,CD10010210,Tom Green: Live! - Clean,Film,0,0,0 days 01:05:41.080000,2021-07-01 01:05:41.080,1,5.0,Tom Green: Live! - Clean,NaN,NaN,NaN
1,2021-07-01 01:05:41.080,CD10010171,Dave Foley: Relatively Well - Clean,Film,1,5,0 days 01:21:23.990000,2021-07-01 02:27:05.070,2,27.0,Dave Foley: Relatively Well - Clean,NaN,NaN,NaN
2,2021-07-01 02:27:05.070,CEG10027749,Harland Williams: What A Treat - Clean,Film,2,27,0 days 01:17:55.060000,2021-07-01 03:45:00.130,3,45.0,Harland Williams: What A Treat - Clean,NaN,NaN,NaN
3,2021-07-01 03:45:00.130,CEG10032516,Never Be Done: The Richard Glen Lett Story,Film,3,45,0 days 01:37:18.070000,2021-07-01 05:22:18.200,5,22.0,Never Be Done: The Richard Glen Lett Story,4.0,NaN,NaN
4,2021-07-01 05:22:18.200,CEG10030236,Harland Williams: A Force of Nature,Film,5,22,0 days 01:05:07.930000,2021-07-01 06:27:26.130,6,27.0,Harland Williams: A Force of Nature,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15381,2023-04-30 19:08:29.290,1000000003242,Phil Kopczinski: Full Grown Man,Film,19,8,0 days 00:27:44.750000,2023-04-30 19:36:14.040,19,36.0,Phil Kopczinski: Full Grown Man,NaN,NaN,NaN
15382,2023-04-30 19:36:14.040,CD10010200,Rob Schneider: Soy Sauce and the Holocaust - C...,Film,19,36,0 days 01:12:18.020000,2023-04-30 20:48:32.060,20,48.0,Rob Schneider: Soy Sauce and the Holocaust - C...,NaN,NaN,NaN
15383,2023-04-30 20:48:32.060,CD10010178,Iliza Shlesinger: War Paint - Clean,Film,20,48,0 days 01:27:24.980000,2023-04-30 22:15:57.040,22,15.0,Iliza Shlesinger: War Paint - Clean,21.0,NaN,NaN
15384,2023-04-30 22:15:57.040,CEG10027755,Doug Benson: Doug Dynasty - Clean,Film,22,15,0 days 01:12:41.230000,2023-04-30 23:28:38.270,23,28.0,Doug Benson: Doug Dynasty - Clean,NaN,NaN,NaN


In [28]:
#percentages
#First Hour
Master.loc[Master['Hour_Start']== (Master['Hour_End']),   'First'] = Master['Length']/timedelta(minutes=60)
Master.loc[Master['Hour_Start']+1 == Master['Hour_End'] , 'First'] = (60 -Master['TS'].dt.minute)/60
Master.loc[Master['Hour_Start']+2 == (Master['Hour_End']),'First'] = (60 -Master['TS'].dt.minute)/60
Master.loc[Master['Hour_Start']+3 == (Master['Hour_End']),'First'] = (60 -Master['TS'].dt.minute)/60
Master.loc[Master['Hour_Start']+4 == (Master['Hour_End']),'First'] = (60 -Master['TS'].dt.minute)/60


Master.loc[(Master['Hour_Start']== Master['Hour_End']+23),'First'] = (60 -Master['TS'].dt.minute)/60
Master.loc[(Master['Hour_Start']== Master['Hour_End']+22),'First'] = (60 -Master['TS'].dt.minute)/60
Master.loc[(Master['Hour_Start']== Master['Hour_End']+21),'First'] = (60 -Master['TS'].dt.minute)/60
Master.loc[(Master['Hour_Start']== Master['Hour_End']+20),'First'] = (60 -Master['TS'].dt.minute)/60


#Last Hour
Master.loc[Master['Hour_Start'] != Master['Hour_End'] ,   'Last'] = Master['Minutes_End']/60

#Mid Hours if necessarry
Master.loc[pd.notna(Master['Middle_1_Hour']),'Mid_1']=1
Master.loc[pd.notna(Master['Middle_2_Hour']),'Mid_2']=1
Master.loc[pd.notna(Master['Middle_3_Hour']),'Mid_3']=1


In [29]:
Master

,TS,Content ID,Title,Series/Film,Hour_Start,Minutes_Start,Length,TS End,Hour_End,Minutes_End,TITLE,Middle_1_Hour,Middle_2_Hour,Middle_3_Hour,First,Last,Mid_1,Mid_2,Mid_3
0,2021-07-01 00:00:00.000,CD10010210,Tom Green: Live! - Clean,Film,0,0,0 days 01:05:41.080000,2021-07-01 01:05:41.080,1,5.0,Tom Green: Live! - Clean,NaN,NaN,NaN,1.000000,0.083333,NaN,NaN,NaN
1,2021-07-01 01:05:41.080,CD10010171,Dave Foley: Relatively Well - Clean,Film,1,5,0 days 01:21:23.990000,2021-07-01 02:27:05.070,2,27.0,Dave Foley: Relatively Well - Clean,NaN,NaN,NaN,0.916667,0.450000,NaN,NaN,NaN
2,2021-07-01 02:27:05.070,CEG10027749,Harland Williams: What A Treat - Clean,Film,2,27,0 days 01:17:55.060000,2021-07-01 03:45:00.130,3,45.0,Harland Williams: What A Treat - Clean,NaN,NaN,NaN,0.550000,0.750000,NaN,NaN,NaN
3,2021-07-01 03:45:00.130,CEG10032516,Never Be Done: The Richard Glen Lett Story,Film,3,45,0 days 01:37:18.070000,2021-07-01 05:22:18.200,5,22.0,Never Be Done: The Richard Glen Lett Story,4.0,NaN,NaN,0.250000,0.366667,1.0,NaN,NaN
4,2021-07-01 05:22:18.200,CEG10030236,Harland Williams: A Force of Nature,Film,5,22,0 days 01:05:07.930000,2021-07-01 06:27:26.130,6,27.0,Harland Williams: A Force of Nature,NaN,NaN,NaN,0.633333,0.450000,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15381,2023-04-30 19:08:29.290,1000000003242,Phil Kopczinski: Full Grown Man,Film,19,8,0 days 00:27:44.750000,2023-04-30 19:36:14.040,19,36.0,Phil Kopczinski: Full Grown Man,NaN,NaN,NaN,0.462431,NaN,NaN,NaN,NaN
15382,2023-04-30 19:36:14.040,CD10010200,Rob Schneider: Soy Sauce and the Holocaust - C...,Film,19,36,0 days 01:12:18.020000,2023-04-30 20:48:32.060,20,48.0,Rob Schneider: Soy Sauce and the Holocaust - C...,NaN,NaN,NaN,0.400000,0.800000,NaN,NaN,NaN
15383,2023-04-30 20:48:32.060,CD10010178,Iliza Shlesinger: War Paint - Clean,Film,20,48,0 days 01:27:24.980000,2023-04-30 22:15:57.040,22,15.0,Iliza Shlesinger: War Paint - Clean,21.0,NaN,NaN,0.200000,0.250000,1.0,NaN,NaN
15384,2023-04-30 22:15:57.040,CEG10027755,Doug Benson: Doug Dynasty - Clean,Film,22,15,0 days 01:12:41.230000,2023-04-30 23:28:38.270,23,28.0,Doug Benson: Doug Dynasty - Clean,NaN,NaN,NaN,0.750000,0.466667,NaN,NaN,NaN


# Amagi Master merge Set Up

In [30]:
Amagi.rename(columns={'Channel Name':'Platform/Channel'}, inplace= True)
Amagi= pd.merge(Amagi, glossary_channel, on= 'Platform/Channel', how='left')

In [31]:
#Change Channel Name **************************************************************************************
Amagi=Amagi.loc[Amagi['Channel']== Channel].copy()

In [32]:
Amagi

,Date,Platform/Channel,Session Count,Total Viewership Minutes,Unique Viewers,Avg Session Count,Avg Session Duration Per Session,Avg Session Duration Per Viewer,Platform,Channel,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
9,2021-07-01 00:00:00,Cinedigm_ComedyDynamics_TCL,4,69.00,4,NaN,NaN,NaN,TCL,ComedyDynamics,NaN,NaN,NaN,NaN
20,2021-07-01 00:00:00,comedydynamics,198,2244.00,186,NaN,NaN,NaN,Xumo(?),ComedyDynamics,NaN,NaN,NaN,NaN
21,2021-07-01 00:00:00,ComedyDynamics Clean Samsung US,2387,31445.00,1611,NaN,NaN,NaN,Samsung,ComedyDynamics,NaN,NaN,NaN,NaN
22,2021-07-01 00:00:00,comedydynamics_junction,1,0.00,1,NaN,NaN,NaN,Junction,ComedyDynamics,NaN,NaN,NaN,NaN
23,2021-07-01 00:00:00,comedydynamics_littlstar,2,60.00,2,NaN,NaN,NaN,Littlstar,ComedyDynamics,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
967743,2023-04-30 23:00:00,Comedy Dynamics Samsung UK,399,845.82,268,1.49,2.12,3.16,Samsung,ComedyDynamics,NaN,NaN,NaN,NaN
967744,2023-04-30 23:00:00,Comedydynamics TCL,5,62.13,5,1.00,12.43,12.43,TCL,ComedyDynamics,NaN,NaN,NaN,NaN
967745,2023-04-30 23:00:00,Comedy Dynamics Vizio US,99,1957.13,84,1.18,19.77,23.30,Vizio,ComedyDynamics,NaN,NaN,NaN,NaN
967746,2023-04-30 23:00:00,ComedyDynamics Xiaomi,147,49.08,123,1.20,0.33,0.40,Xiaomi,ComedyDynamics,NaN,NaN,NaN,NaN


# Uploading and Merging Roku Linear Viewership Data

In [33]:
Months_Year = {'2023-03_1':'2023','2023-03_2':'2023','2023-04_1':'2023','2023-04_2':'2023'}
filepathRL = 'C:/Users/sortiz/Box/Digital Networks/REPORTING + ANALYTICS/Linear/Roku/Roku Viewership/Hourly Roku Data/'

In [34]:
Month_list_Roku = []

for i in Months_Year:
    Month_list_Roku = Month_list_Roku + [filepathRL + Channel + '/' + Channel + ' Hourly noContent ' + i + '.csv'] 

In [35]:
def Rd_Cl(data):
    
    C = pd.read_csv(data)
    C.columns = C.iloc[0,:]
    C = C.iloc[1:,1:]
    C.rename(columns={'Streaming Hours Sum':'Total Viewership Hours','Stream Start Event Count':'Session Count'},inplace=True)
    
    Cond = [
    C['Start Hour']=='12am EST', C['Start Hour']=='1am EST', C['Start Hour']=='2am EST', C['Start Hour']=='3am EST',
    C['Start Hour']=='4am EST',  C['Start Hour']=='5am EST', C['Start Hour']=='6am EST', C['Start Hour']=='7am EST',
    C['Start Hour']=='8am EST',  C['Start Hour']=='9am EST', C['Start Hour']=='10am EST',C['Start Hour']=='11am EST',
    C['Start Hour']=='12pm EST', C['Start Hour']=='1pm EST', C['Start Hour']=='2pm EST', C['Start Hour']=='3pm EST',
    C['Start Hour']=='4pm EST',  C['Start Hour']=='5pm EST', C['Start Hour']=='6pm EST', C['Start Hour']=='7pm EST',
    C['Start Hour']=='8pm EST',  C['Start Hour']=='9pm EST', C['Start Hour']=='10pm EST',C['Start Hour']=='11pm EST'
    ]
    
    Values = ['00:00:00','01:00:00','02:00:00','03:00:00','04:00:00','05:00:00','06:00:00','07:00:00','08:00:00','09:00:00','10:00:00','11:00:00',
              '12:00:00','13:00:00','14:00:00','15:00:00','16:00:00','17:00:00','18:00:00','19:00:00','20:00:00','21:00:00','22:00:00','23:00:00']
    
    C['MT'] = np.select(Cond,Values)
    C['Session Count'] = C['Session Count'].str.replace(',','',regex=False)
    C['Total Viewership Hours'] = C['Total Viewership Hours'].str.replace(',','',regex=False)
    
    C['Date'] = C['Date Dim'] + ' ' + C['MT']
    C['Total Viewership Minutes'] = C['Total Viewership Hours'].astype(float)*60
    C['Date'] = pd.to_datetime(C['Date'])
    C['Session Count'] = C['Session Count'].astype(int)
    C['Platform']='Roku'
    C['Channel']= Channel
    
    C = C.loc[:,('Date','Channel','Platform','Total Viewership Minutes','Session Count')].drop_duplicates()
    C = C.sort_values('Date')

    return C.copy()

In [36]:
Roku_V = pd.DataFrame()

for data in Month_list_Roku:
    Roku_V = Roku_V.append(Rd_Cl(data))

In [37]:
Roku_V

,Date,Channel,Platform,Total Viewership Minutes,Session Count
6,2023-03-01 00:00:00,ComedyDynamics,Roku,8058.6,1381
7,2023-03-01 01:00:00,ComedyDynamics,Roku,38698.2,947
8,2023-03-01 02:00:00,ComedyDynamics,Roku,3460.2,828
9,2023-03-01 03:00:00,ComedyDynamics,Roku,30009.0,604
10,2023-03-01 04:00:00,ComedyDynamics,Roku,29016.6,454
...,...,...,...,...,...
337,2023-04-30 19:00:00,ComedyDynamics,Roku,36723.0,3165
338,2023-04-30 20:00:00,ComedyDynamics,Roku,43404.0,3449
339,2023-04-30 21:00:00,ComedyDynamics,Roku,47687.4,3180
340,2023-04-30 22:00:00,ComedyDynamics,Roku,10322.4,2341


# Merge Roku with Amagi Data

In [38]:
Amagi = pd.concat([Amagi,Roku_V])

In [39]:
#REMOVING ROKU**************

#Amagi= Amagi.loc[Amagi['Platform'] != 'Roku',:].copy()

In [40]:
Amagi

,Date,Platform/Channel,Session Count,Total Viewership Minutes,Unique Viewers,Avg Session Count,Avg Session Duration Per Session,Avg Session Duration Per Viewer,Platform,Channel,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
9,2021-07-01 00:00:00,Cinedigm_ComedyDynamics_TCL,4,69.0,4.0,NaN,NaN,NaN,TCL,ComedyDynamics,NaN,NaN,NaN,NaN
20,2021-07-01 00:00:00,comedydynamics,198,2244.0,186.0,NaN,NaN,NaN,Xumo(?),ComedyDynamics,NaN,NaN,NaN,NaN
21,2021-07-01 00:00:00,ComedyDynamics Clean Samsung US,2387,31445.0,1611.0,NaN,NaN,NaN,Samsung,ComedyDynamics,NaN,NaN,NaN,NaN
22,2021-07-01 00:00:00,comedydynamics_junction,1,0.0,1.0,NaN,NaN,NaN,Junction,ComedyDynamics,NaN,NaN,NaN,NaN
23,2021-07-01 00:00:00,comedydynamics_littlstar,2,60.0,2.0,NaN,NaN,NaN,Littlstar,ComedyDynamics,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
337,2023-04-30 19:00:00,NaN,3165,36723.0,NaN,NaN,NaN,NaN,Roku,ComedyDynamics,NaN,NaN,NaN,NaN
338,2023-04-30 20:00:00,NaN,3449,43404.0,NaN,NaN,NaN,NaN,Roku,ComedyDynamics,NaN,NaN,NaN,NaN
339,2023-04-30 21:00:00,NaN,3180,47687.4,NaN,NaN,NaN,NaN,Roku,ComedyDynamics,NaN,NaN,NaN,NaN
340,2023-04-30 22:00:00,NaN,2341,10322.4,NaN,NaN,NaN,NaN,Roku,ComedyDynamics,NaN,NaN,NaN,NaN


In [41]:
Amagi['Hour']= Amagi['Date'].dt.hour
Amagi['Day']= Amagi['Date'].dt.day
Amagi['Year']= Amagi['Date'].dt.year
Amagi['Month']= Amagi['Date'].dt.month

In [42]:
#Amagi.loc[Amagi['Day']==1, ('Platform/Channel','Month','Day')].drop_duplicates()

In [43]:
Platform=Amagi

In [44]:
Amagi_Min=Amagi.groupby(['Year','Month','Day','Hour'])['Total Viewership Minutes'].agg(sum).reset_index()
Amagi_Sess=Amagi.groupby(['Year','Month','Day','Hour'])['Session Count'].agg(sum).reset_index()
Amagi_Min.rename(columns={'Total Viewership Minutes':'Min'}, inplace=True)
Amagi_Sess.rename(columns={'Session Count':'Sess'}, inplace=True)


In [45]:
Master['Day']= Master['TS'].dt.day
Master['Month']= Master['TS'].dt.month
Master['Year']= Master['TS'].dt.year

# Merging Master with Amagi Data

In [46]:
H=('Hour_Start','Middle_1_Hour','Middle_2_Hour','Middle_3_Hour','Hour_End')
M=('start','Mid1','Mid2','Mid3','End')


for i,j in zip(H,M):
    Master= pd.merge(Master, Amagi_Min ,left_on=['Year','Month','Day',i], right_on=['Year','Month','Day', 'Hour'], how='left' )
    Master= pd.merge(Master, Amagi_Sess ,left_on=['Year','Month','Day',i], right_on=['Year','Month','Day', 'Hour'], how='left' )
    Master.drop(columns=['Hour_x','Hour_y'], inplace= True)
    Master.rename(columns={'Min':'Min_Hour_'+j}, inplace=True)
    Master.rename(columns={'Sess':'Sess_Hour_'+j}, inplace= True)

In [47]:
Master.loc[(Master['Hour_Start']== Master['Hour_End']), 'Min_Hour_End']= 0
Master.loc[(Master['Hour_Start']== Master['Hour_End']), 'Sess_Hour_End']= 0

In [48]:
Master.loc[Master['Month']==2]

,TS,Content ID,Title,Series/Film,Hour_Start,Minutes_Start,Length,TS End,Hour_End,Minutes_End,TITLE,Middle_1_Hour,Middle_2_Hour,Middle_3_Hour,First,Last,Mid_1,Mid_2,Mid_3,Day,Month,Year,Min_Hour_start,Sess_Hour_start,Min_Hour_Mid1,Sess_Hour_Mid1,Min_Hour_Mid2,Sess_Hour_Mid2,Min_Hour_Mid3,Sess_Hour_Mid3,Min_Hour_End,Sess_Hour_End
5049,2022-02-01 00:00:15.140,CEG10027757,"David Cross: Oh, Come On - Clean",Film,0,0,0 days 01:25:11.980000,2022-02-01 01:25:27.120,1,25.0,"David Cross: Oh, Come On - Clean",NaN,NaN,NaN,1.000000,0.416667,NaN,NaN,NaN,1,2,2022,46627.55,5499.0,NaN,NaN,NaN,NaN,NaN,NaN,44913.44,3796.0
5050,2022-02-01 01:25:27.120,CD10010174,Eddie Griffin: You Can Tell 'Em I Said It - Clean,Film,1,25,-1 days +22:34:32.880000,2022-02-01 00:00:00.000,0,0.0,Eddie Griffin: You Can Tell 'Em I Said It - Clean,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,1,2,2022,44913.44,3796.0,NaN,NaN,NaN,NaN,NaN,NaN,46627.55,5499.0
5051,2022-02-01 00:00:00.000,1000000003674,Costaki Economoupolos: Now That I'm a Grown Up,Film,0,0,0 days 00:48:54.100000,2022-02-01 00:48:54.100,0,48.0,Costaki Economoupolos: Now That I'm a Grown Up,NaN,NaN,NaN,0.815028,NaN,NaN,NaN,NaN,1,2,2022,46627.55,5499.0,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0.0
5052,2022-02-01 00:48:54.100,1000000003701,Scotty K: Not Country Music,Film,0,48,0 days 00:27:57.130000,2022-02-01 01:16:51.230,1,16.0,Scotty K: Not Country Music,NaN,NaN,NaN,0.200000,0.266667,NaN,NaN,NaN,1,2,2022,46627.55,5499.0,NaN,NaN,NaN,NaN,NaN,NaN,44913.44,3796.0
5053,2022-02-01 01:16:51.230,1000000003693,Michael Palascak: 1984,Film,1,16,0 days 00:37:46,2022-02-01 01:54:37.230,1,54.0,Michael Palascak: 1984,NaN,NaN,NaN,0.629444,NaN,NaN,NaN,NaN,1,2,2022,44913.44,3796.0,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14118,2023-02-28 17:44:10.280,1000000002304,Kortney Shane Williams: Unsubstantiated,Film,17,44,0 days 00:49:00.770000,2023-02-28 18:33:11.050,18,33.0,Kortney Shane Williams: Unsubstantiated,NaN,NaN,NaN,0.266667,0.550000,NaN,NaN,NaN,28,2,2023,48033.11,9029.0,NaN,NaN,NaN,NaN,NaN,NaN,46135.13,8974.0
14119,2023-02-28 18:33:11.050,CD10010210,Tom Green: Live! - Clean,Film,18,33,0 days 01:06:42.960000,2023-02-28 19:39:54.010,19,39.0,Tom Green: Live! - Clean,NaN,NaN,NaN,0.450000,0.650000,NaN,NaN,NaN,28,2,2023,46135.13,8974.0,NaN,NaN,NaN,NaN,NaN,NaN,44487.82,8283.0
14120,2023-02-28 19:39:54.010,1000000008121,The Comedy Show Show,Series,19,39,0 days 01:15:06.180000,2023-02-28 20:55:00.190,20,55.0,The Comedy Show Show,NaN,NaN,NaN,0.350000,0.916667,NaN,NaN,NaN,28,2,2023,44487.82,8283.0,NaN,NaN,NaN,NaN,NaN,NaN,37564.98,7061.0
14121,2023-02-28 20:55:00.190,1000000006839,SXSW Comedy with W. Kamau Bell Part 2,Film,20,55,0 days 01:34:31.810000,2023-02-28 22:29:32.000,22,29.0,SXSW Comedy with W. Kamau Bell Part 2,21.0,NaN,NaN,0.083333,0.483333,1.0,NaN,NaN,28,2,2023,37564.98,7061.0,35636.44,5518.0,NaN,NaN,NaN,NaN,35421.42,4468.0


# Fill nan with 0

In [49]:
L=['Last','Mid_1','Mid_2','Mid_3','Min_Hour_Mid1','Min_Hour_Mid2',
 'Min_Hour_Mid3','Min_Hour_End','Sess_Hour_Mid1','Sess_Hour_Mid2',
 'Sess_Hour_Mid3','Sess_Hour_End']
for i in L:
    Master[i]=Master[i].fillna(0)

# Multiply percent of hour with data for whole hour

In [50]:
# *******************************

# ISSUE 4

# - Because amagi data is broken down by hour then some titles will share an hour of data
#     -depending on how much time a title will have in an hour will determine what percentage of the kpi will be allocated

# ******************************

In [51]:
Master['Total Minutes']= ((Master['First']*Master['Min_Hour_start']) + 
(Master['Mid_1']*Master['Min_Hour_Mid1']) + 
(Master['Mid_2'] * Master['Min_Hour_Mid2']) + 
(Master['Mid_3'] * Master['Min_Hour_Mid3']) +                           
(Master['Last'] * Master['Min_Hour_End']))

Master['Total Session']= ((Master['First'] * Master['Sess_Hour_start']) + 
(Master['Mid_1'] * Master['Sess_Hour_Mid1']) + 
(Master['Mid_2'] * Master['Sess_Hour_Mid2']) + 
(Master['Mid_3'] * Master['Sess_Hour_Mid3']) + 
(Master['Last'] * Master['Sess_Hour_End']))

Master.drop(Master[Master['TS']==Master['TS End']].index, inplace=True)
Master.reset_index()

,index,TS,Content ID,Title,Series/Film,Hour_Start,Minutes_Start,Length,TS End,Hour_End,Minutes_End,TITLE,Middle_1_Hour,Middle_2_Hour,Middle_3_Hour,First,Last,Mid_1,Mid_2,Mid_3,Day,Month,Year,Min_Hour_start,Sess_Hour_start,Min_Hour_Mid1,Sess_Hour_Mid1,Min_Hour_Mid2,Sess_Hour_Mid2,Min_Hour_Mid3,Sess_Hour_Mid3,Min_Hour_End,Sess_Hour_End,Total Minutes,Total Session
0,0,2021-07-01 00:00:00.000,CD10010210,Tom Green: Live! - Clean,Film,0,0,0 days 01:05:41.080000,2021-07-01 01:05:41.080,1,5.0,Tom Green: Live! - Clean,NaN,NaN,NaN,1.000000,0.083333,0.0,0.0,0.0,1,7,2021,54322.00,3958.0,0.00,0.0,0.0,0.0,0.0,0.0,53040.00,3439.0,58742.000000,4244.583333
1,1,2021-07-01 01:05:41.080,CD10010171,Dave Foley: Relatively Well - Clean,Film,1,5,0 days 01:21:23.990000,2021-07-01 02:27:05.070,2,27.0,Dave Foley: Relatively Well - Clean,NaN,NaN,NaN,0.916667,0.450000,0.0,0.0,0.0,1,7,2021,53040.00,3439.0,0.00,0.0,0.0,0.0,0.0,0.0,51499.00,3265.0,71794.550000,4621.666667
2,2,2021-07-01 02:27:05.070,CEG10027749,Harland Williams: What A Treat - Clean,Film,2,27,0 days 01:17:55.060000,2021-07-01 03:45:00.130,3,45.0,Harland Williams: What A Treat - Clean,NaN,NaN,NaN,0.550000,0.750000,0.0,0.0,0.0,1,7,2021,51499.00,3265.0,0.00,0.0,0.0,0.0,0.0,0.0,48104.00,3581.0,64402.450000,4481.500000
3,3,2021-07-01 03:45:00.130,CEG10032516,Never Be Done: The Richard Glen Lett Story,Film,3,45,0 days 01:37:18.070000,2021-07-01 05:22:18.200,5,22.0,Never Be Done: The Richard Glen Lett Story,4.0,NaN,NaN,0.250000,0.366667,1.0,0.0,0.0,1,7,2021,48104.00,3581.0,42377.00,4100.0,0.0,0.0,0.0,0.0,41059.00,4601.0,69457.966667,6682.283333
4,4,2021-07-01 05:22:18.200,CEG10030236,Harland Williams: A Force of Nature,Film,5,22,0 days 01:05:07.930000,2021-07-01 06:27:26.130,6,27.0,Harland Williams: A Force of Nature,NaN,NaN,NaN,0.633333,0.450000,0.0,0.0,0.0,1,7,2021,41059.00,4601.0,0.00,0.0,0.0,0.0,0.0,0.0,38671.00,5164.0,43405.983333,5237.766667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15381,15381,2023-04-30 19:08:29.290,1000000003242,Phil Kopczinski: Full Grown Man,Film,19,8,0 days 00:27:44.750000,2023-04-30 19:36:14.040,19,36.0,Phil Kopczinski: Full Grown Man,NaN,NaN,NaN,0.462431,0.000000,0.0,0.0,0.0,30,4,2023,48008.86,5401.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.0,22200.763801,2497.587431
15382,15382,2023-04-30 19:36:14.040,CD10010200,Rob Schneider: Soy Sauce and the Holocaust - C...,Film,19,36,0 days 01:12:18.020000,2023-04-30 20:48:32.060,20,48.0,Rob Schneider: Soy Sauce and the Holocaust - C...,NaN,NaN,NaN,0.400000,0.800000,0.0,0.0,0.0,30,4,2023,48008.86,5401.0,0.00,0.0,0.0,0.0,0.0,0.0,53788.94,5195.0,62234.696000,6316.400000
15383,15383,2023-04-30 20:48:32.060,CD10010178,Iliza Shlesinger: War Paint - Clean,Film,20,48,0 days 01:27:24.980000,2023-04-30 22:15:57.040,22,15.0,Iliza Shlesinger: War Paint - Clean,21.0,NaN,NaN,0.200000,0.250000,1.0,0.0,0.0,30,4,2023,53788.94,5195.0,57582.93,4604.0,0.0,0.0,0.0,0.0,18880.25,3523.0,73060.780500,6523.750000
15384,15384,2023-04-30 22:15:57.040,CEG10027755,Doug Benson: Doug Dynasty - Clean,Film,22,15,0 days 01:12:41.230000,2023-04-30 23:28:38.270,23,28.0,Doug Benson: Doug Dynasty - Clean,NaN,NaN,NaN,0.750000,0.466667,0.0,0.0,0.0,30,4,2023,18880.25,3523.0,0.00,0.0,0.0,0.0,0.0,0.0,30632.21,2973.0,28455.218833,4029.650000


# Master Breakdown

In [52]:
Master['First_Hour_Min']=Master['First']*Master['Min_Hour_start']
Master['Second_Hour_Min']=Master['Mid_1']*Master['Min_Hour_Mid1']
Master['Third_Hour_Min']=Master['Mid_2']*Master['Min_Hour_Mid2']
Master['Fourth_Hour_Min']=Master['Mid_3']*Master['Min_Hour_Mid3']
Master['Last_Hour_Min']=Master['Last']*Master['Min_Hour_End']

Master['First_Hour_Sess']=Master['First']*Master['Sess_Hour_start']
Master['Second_Hour_Sess']=Master['Mid_1']*Master['Sess_Hour_Mid1']
Master['Third_Hour_Sess']=Master['Mid_2']*Master['Sess_Hour_Mid2']
Master['Fourth_Hour_Sess']=Master['Mid_3']*Master['Sess_Hour_Mid3']
Master['Last_Hour_Sess']=Master['Last']*Master['Sess_Hour_End']


In [53]:
Master

,TS,Content ID,Title,Series/Film,Hour_Start,Minutes_Start,Length,TS End,Hour_End,Minutes_End,TITLE,Middle_1_Hour,Middle_2_Hour,Middle_3_Hour,First,Last,Mid_1,Mid_2,Mid_3,Day,Month,Year,Min_Hour_start,Sess_Hour_start,Min_Hour_Mid1,Sess_Hour_Mid1,Min_Hour_Mid2,Sess_Hour_Mid2,Min_Hour_Mid3,Sess_Hour_Mid3,Min_Hour_End,Sess_Hour_End,Total Minutes,Total Session,First_Hour_Min,Second_Hour_Min,Third_Hour_Min,Fourth_Hour_Min,Last_Hour_Min,First_Hour_Sess,Second_Hour_Sess,Third_Hour_Sess,Fourth_Hour_Sess,Last_Hour_Sess
0,2021-07-01 00:00:00.000,CD10010210,Tom Green: Live! - Clean,Film,0,0,0 days 01:05:41.080000,2021-07-01 01:05:41.080,1,5.0,Tom Green: Live! - Clean,NaN,NaN,NaN,1.000000,0.083333,0.0,0.0,0.0,1,7,2021,54322.00,3958.0,0.00,0.0,0.0,0.0,0.0,0.0,53040.00,3439.0,58742.000000,4244.583333,54322.000000,0.00,0.0,0.0,4420.000000,3958.000000,0.0,0.0,0.0,286.583333
1,2021-07-01 01:05:41.080,CD10010171,Dave Foley: Relatively Well - Clean,Film,1,5,0 days 01:21:23.990000,2021-07-01 02:27:05.070,2,27.0,Dave Foley: Relatively Well - Clean,NaN,NaN,NaN,0.916667,0.450000,0.0,0.0,0.0,1,7,2021,53040.00,3439.0,0.00,0.0,0.0,0.0,0.0,0.0,51499.00,3265.0,71794.550000,4621.666667,48620.000000,0.00,0.0,0.0,23174.550000,3152.416667,0.0,0.0,0.0,1469.250000
2,2021-07-01 02:27:05.070,CEG10027749,Harland Williams: What A Treat - Clean,Film,2,27,0 days 01:17:55.060000,2021-07-01 03:45:00.130,3,45.0,Harland Williams: What A Treat - Clean,NaN,NaN,NaN,0.550000,0.750000,0.0,0.0,0.0,1,7,2021,51499.00,3265.0,0.00,0.0,0.0,0.0,0.0,0.0,48104.00,3581.0,64402.450000,4481.500000,28324.450000,0.00,0.0,0.0,36078.000000,1795.750000,0.0,0.0,0.0,2685.750000
3,2021-07-01 03:45:00.130,CEG10032516,Never Be Done: The Richard Glen Lett Story,Film,3,45,0 days 01:37:18.070000,2021-07-01 05:22:18.200,5,22.0,Never Be Done: The Richard Glen Lett Story,4.0,NaN,NaN,0.250000,0.366667,1.0,0.0,0.0,1,7,2021,48104.00,3581.0,42377.00,4100.0,0.0,0.0,0.0,0.0,41059.00,4601.0,69457.966667,6682.283333,12026.000000,42377.00,0.0,0.0,15054.966667,895.250000,4100.0,0.0,0.0,1687.033333
4,2021-07-01 05:22:18.200,CEG10030236,Harland Williams: A Force of Nature,Film,5,22,0 days 01:05:07.930000,2021-07-01 06:27:26.130,6,27.0,Harland Williams: A Force of Nature,NaN,NaN,NaN,0.633333,0.450000,0.0,0.0,0.0,1,7,2021,41059.00,4601.0,0.00,0.0,0.0,0.0,0.0,0.0,38671.00,5164.0,43405.983333,5237.766667,26004.033333,0.00,0.0,0.0,17401.950000,2913.966667,0.0,0.0,0.0,2323.800000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15381,2023-04-30 19:08:29.290,1000000003242,Phil Kopczinski: Full Grown Man,Film,19,8,0 days 00:27:44.750000,2023-04-30 19:36:14.040,19,36.0,Phil Kopczinski: Full Grown Man,NaN,NaN,NaN,0.462431,0.000000,0.0,0.0,0.0,30,4,2023,48008.86,5401.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.0,22200.763801,2497.587431,22200.763801,0.00,0.0,0.0,0.000000,2497.587431,0.0,0.0,0.0,0.000000
15382,2023-04-30 19:36:14.040,CD10010200,Rob Schneider: Soy Sauce and the Holocaust - C...,Film,19,36,0 days 01:12:18.020000,2023-04-30 20:48:32.060,20,48.0,Rob Schneider: Soy Sauce and the Holocaust - C...,NaN,NaN,NaN,0.400000,0.800000,0.0,0.0,0.0,30,4,2023,48008.86,5401.0,0.00,0.0,0.0,0.0,0.0,0.0,53788.94,5195.0,62234.696000,6316.400000,19203.544000,0.00,0.0,0.0,43031.152000,2160.400000,0.0,0.0,0.0,4156.000000
15383,2023-04-30 20:48:32.060,CD10010178,Iliza Shlesinger: War Paint - Clean,Film,20,48,0 days 01:27:24.980000,2023-04-30 22:15:57.040,22,15.0,Iliza Shlesinger: War Paint - Clean,21.0,NaN,NaN,0.200000,0.250000,1.0,0.0,0.0,30,4,2023,53788.94,5195.0,57582.93,4604.0,0.0,0.0,0.0,0.0,18880.25,3523.0,73060.780500,6523.750000,10757.788000,57582.93,0.0,0.0,4720.062500,1039.000000,4604.0,0.0,0.0,880.750000
15384,2023-04-30 22:15:57.040,CEG10027755,Doug Benson: Doug Dynasty - Clean,Film,22,15,0 days 01:12:41.230000,2023-04-30 23:28:38.270,23,28.0,Doug Benson: Doug Dynasty - Clean,NaN,NaN,NaN,0.750000,0.466667,

In [54]:
Master['Length_1']=Master['First']*60
Master['Length_2']=Master['Mid_1']*60
Master['Length_3']=Master['Mid_2']*60
Master['Length_4']=Master['Mid_3']*60
Master['Length_5']=Master['Last']*60

In [55]:
Master

,TS,Content ID,Title,Series/Film,Hour_Start,Minutes_Start,Length,TS End,Hour_End,Minutes_End,TITLE,Middle_1_Hour,Middle_2_Hour,Middle_3_Hour,First,Last,Mid_1,Mid_2,Mid_3,Day,Month,Year,Min_Hour_start,Sess_Hour_start,Min_Hour_Mid1,Sess_Hour_Mid1,Min_Hour_Mid2,Sess_Hour_Mid2,Min_Hour_Mid3,Sess_Hour_Mid3,Min_Hour_End,Sess_Hour_End,Total Minutes,Total Session,First_Hour_Min,Second_Hour_Min,Third_Hour_Min,Fourth_Hour_Min,Last_Hour_Min,First_Hour_Sess,Second_Hour_Sess,Third_Hour_Sess,Fourth_Hour_Sess,Last_Hour_Sess,Length_1,Length_2,Length_3,Length_4,Length_5
0,2021-07-01 00:00:00.000,CD10010210,Tom Green: Live! - Clean,Film,0,0,0 days 01:05:41.080000,2021-07-01 01:05:41.080,1,5.0,Tom Green: Live! - Clean,NaN,NaN,NaN,1.000000,0.083333,0.0,0.0,0.0,1,7,2021,54322.00,3958.0,0.00,0.0,0.0,0.0,0.0,0.0,53040.00,3439.0,58742.000000,4244.583333,54322.000000,0.00,0.0,0.0,4420.000000,3958.000000,0.0,0.0,0.0,286.583333,60.000000,0.0,0.0,0.0,5.0
1,2021-07-01 01:05:41.080,CD10010171,Dave Foley: Relatively Well - Clean,Film,1,5,0 days 01:21:23.990000,2021-07-01 02:27:05.070,2,27.0,Dave Foley: Relatively Well - Clean,NaN,NaN,NaN,0.916667,0.450000,0.0,0.0,0.0,1,7,2021,53040.00,3439.0,0.00,0.0,0.0,0.0,0.0,0.0,51499.00,3265.0,71794.550000,4621.666667,48620.000000,0.00,0.0,0.0,23174.550000,3152.416667,0.0,0.0,0.0,1469.250000,55.000000,0.0,0.0,0.0,27.0
2,2021-07-01 02:27:05.070,CEG10027749,Harland Williams: What A Treat - Clean,Film,2,27,0 days 01:17:55.060000,2021-07-01 03:45:00.130,3,45.0,Harland Williams: What A Treat - Clean,NaN,NaN,NaN,0.550000,0.750000,0.0,0.0,0.0,1,7,2021,51499.00,3265.0,0.00,0.0,0.0,0.0,0.0,0.0,48104.00,3581.0,64402.450000,4481.500000,28324.450000,0.00,0.0,0.0,36078.000000,1795.750000,0.0,0.0,0.0,2685.750000,33.000000,0.0,0.0,0.0,45.0
3,2021-07-01 03:45:00.130,CEG10032516,Never Be Done: The Richard Glen Lett Story,Film,3,45,0 days 01:37:18.070000,2021-07-01 05:22:18.200,5,22.0,Never Be Done: The Richard Glen Lett Story,4.0,NaN,NaN,0.250000,0.366667,1.0,0.0,0.0,1,7,2021,48104.00,3581.0,42377.00,4100.0,0.0,0.0,0.0,0.0,41059.00,4601.0,69457.966667,6682.283333,12026.000000,42377.00,0.0,0.0,15054.966667,895.250000,4100.0,0.0,0.0,1687.033333,15.000000,60.0,0.0,0.0,22.0
4,2021-07-01 05:22:18.200,CEG10030236,Harland Williams: A Force of Nature,Film,5,22,0 days 01:05:07.930000,2021-07-01 06:27:26.130,6,27.0,Harland Williams: A Force of Nature,NaN,NaN,NaN,0.633333,0.450000,0.0,0.0,0.0,1,7,2021,41059.00,4601.0,0.00,0.0,0.0,0.0,0.0,0.0,38671.00,5164.0,43405.983333,5237.766667,26004.033333,0.00,0.0,0.0,17401.950000,2913.966667,0.0,0.0,0.0,2323.800000,38.000000,0.0,0.0,0.0,27.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15381,2023-04-30 19:08:29.290,1000000003242,Phil Kopczinski: Full Grown Man,Film,19,8,0 days 00:27:44.750000,2023-04-30 19:36:14.040,19,36.0,Phil Kopczinski: Full Grown Man,NaN,NaN,NaN,0.462431,0.000000,0.0,0.0,0.0,30,4,2023,48008.86,5401.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.0,22200.763801,2497.587431,22200.763801,0.00,0.0,0.0,0.000000,2497.587431,0.0,0.0,0.0,0.000000,27.745833,0.0,0.0,0.0,0.0
15382,2023-04-30 19:36:14.040,CD10010200,Rob Schneider: Soy Sauce and the Holocaust - C...,Film,19,36,0 days 01:12:18.020000,2023-04-30 20:48:32.060,20,48.0,Rob Schneider: Soy Sauce and the Holocaust - C...,NaN,NaN,NaN,0.400000,0.800000,0.0,0.0,0.0,30,4,2023,48008.86,5401.0,0.00,0.0,0.0,0.0,0.0,0.0,53788.94,5195.0,62234.696000,6316.400000,19203.544000,0.00,0.0,0.0,43031.152000,2160.400000,0.0,0.0,0.0,4156.000000,24.000000,0.0,0.0,0.0,48.0
15383,2023-04-30 20:48:32.060,CD10010178,Iliza Shlesinger: War Paint - Clean,Film,20,48,0 days 01:27:24.980000,2023-04-30 22:15:57.040,22,15.0,Iliza Shlesinger: War Paint - Clean,21.0,NaN,NaN,0.200000,0.250000,1.0,0.0,0.0,30,4,2023,53788.94,5195.0,57582.93,4604.0,0.0,0.0,0.0,0.0,18880.25,3523.0,73060.780500,6523.750000,10757.788000,57582.93,0.0,0.0,472

In [56]:
Master1= Master.loc[:,('Content ID','TS','TS End','Title','Series/Film','Hour_Start','First_Hour_Min','First_Hour_Sess',
                       'Length_1','Total Minutes','Total Session','Year','Month')]
Master2= Master.loc[:,('Content ID','TS','TS End','Title','Series/Film','Middle_1_Hour','Second_Hour_Min','Second_Hour_Sess',
                       'Length_2','Total Minutes','Total Session','Year','Month')]
Master3= Master.loc[:,('Content ID','TS','TS End','Title','Series/Film','Middle_2_Hour','Third_Hour_Min','Third_Hour_Sess',
                       'Length_3','Total Minutes','Total Session','Year','Month')]
Master4= Master.loc[:,('Content ID','TS','TS End','Title','Series/Film','Middle_3_Hour','Fourth_Hour_Min','Fourth_Hour_Sess',
                       'Length_4','Total Minutes','Total Session','Year','Month')]
Master5= Master.loc[:,('Content ID','TS','TS End','Title','Series/Film','Hour_End','Last_Hour_Min','Last_Hour_Sess',
                       'Length_5','Total Minutes','Total Session','Year','Month')]

In [57]:
Master1

,Content ID,TS,TS End,Title,Series/Film,Hour_Start,First_Hour_Min,First_Hour_Sess,Length_1,Total Minutes,Total Session,Year,Month
0,CD10010210,2021-07-01 00:00:00.000,2021-07-01 01:05:41.080,Tom Green: Live! - Clean,Film,0,54322.000000,3958.000000,60.000000,58742.000000,4244.583333,2021,7
1,CD10010171,2021-07-01 01:05:41.080,2021-07-01 02:27:05.070,Dave Foley: Relatively Well - Clean,Film,1,48620.000000,3152.416667,55.000000,71794.550000,4621.666667,2021,7
2,CEG10027749,2021-07-01 02:27:05.070,2021-07-01 03:45:00.130,Harland Williams: What A Treat - Clean,Film,2,28324.450000,1795.750000,33.000000,64402.450000,4481.500000,2021,7
3,CEG10032516,2021-07-01 03:45:00.130,2021-07-01 05:22:18.200,Never Be Done: The Richard Glen Lett Story,Film,3,12026.000000,895.250000,15.000000,69457.966667,6682.283333,2021,7
4,CEG10030236,2021-07-01 05:22:18.200,2021-07-01 06:27:26.130,Harland Williams: A Force of Nature,Film,5,26004.033333,2913.966667,38.000000,43405.983333,5237.766667,2021,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15381,1000000003242,2023-04-30 19:08:29.290,2023-04-30 19:36:14.040,Phil Kopczinski: Full Grown Man,Film,19,22200.763801,2497.587431,27.745833,22200.763801,2497.587431,2023,4
15382,CD10010200,2023-04-30 19:36:14.040,2023-04-30 20:48:32.060,Rob Schneider: Soy Sauce and the Holocaust - C...,Film,19,19203.544000,2160.400000,24.000000,62234.696000,6316.400000,2023,4
15383,CD10010178,2023-04-30 20:48:32.060,2023-04-30 22:15:57.040,Iliza Shlesinger: War Paint - Clean,Film,20,10757.788000,1039.000000,12.000000,73060.780500,6523.750000,2023,4
15384,CEG10027755,2023-04-30 22:15:57.040,2023-04-30 23:28:38.270,Doug Benson: Doug Dynasty - Clean,Film,22,14160.187500,2642.250000,45.000000,28455.218833,4029.650000,2023,4


In [58]:
Master1.rename(columns={'Hour_Start':'Hour','First_Hour_Min':'Minutes Viewed in Hour','Length_1':'Length(min) in the Hour',
                        'First_Hour_Sess':'Sessions in Hour'},inplace=True)
Master2.rename(columns={'Middle_1_Hour':'Hour','Second_Hour_Min':'Minutes Viewed in Hour','Length_2':'Length(min) in the Hour',
                        'Second_Hour_Sess':'Sessions in Hour'},inplace=True)
Master3.rename(columns={'Middle_2_Hour':'Hour','Third_Hour_Min':'Minutes Viewed in Hour','Length_3':'Length(min) in the Hour',
                        'Third_Hour_Sess':'Sessions in Hour'},inplace=True)
Master4.rename(columns={'Middle_3_Hour':'Hour','Fourth_Hour_Min':'Minutes Viewed in Hour','Length_4':'Length(min) in the Hour',
                        'Fourth_Hour_Sess':'Sessions in Hour'},inplace=True)
Master5.rename(columns={'Hour_End':'Hour','Last_Hour_Min':'Minutes Viewed in Hour','Length_5':'Length(min) in the Hour',
                        'Last_Hour_Sess':'Sessions in Hour'},inplace=True)



In [59]:
DF= Master1.append([Master2,Master3,Master4,Master5])
DF.dropna(inplace=True)
DF['Weekday']= DF['TS'].dt.weekday

In [60]:
DF.head(20)

,Content ID,TS,TS End,Title,Series/Film,Hour,Minutes Viewed in Hour,Sessions in Hour,Length(min) in the Hour,Total Minutes,Total Session,Year,Month,Weekday
0,CD10010210,2021-07-01 00:00:00.000,2021-07-01 01:05:41.080,Tom Green: Live! - Clean,Film,0.0,54322.000000,3958.000000,60.0000,58742.000000,4244.583333,2021,7,3
1,CD10010171,2021-07-01 01:05:41.080,2021-07-01 02:27:05.070,Dave Foley: Relatively Well - Clean,Film,1.0,48620.000000,3152.416667,55.0000,71794.550000,4621.666667,2021,7,3
2,CEG10027749,2021-07-01 02:27:05.070,2021-07-01 03:45:00.130,Harland Williams: What A Treat - Clean,Film,2.0,28324.450000,1795.750000,33.0000,64402.450000,4481.500000,2021,7,3
3,CEG10032516,2021-07-01 03:45:00.130,2021-07-01 05:22:18.200,Never Be Done: The Richard Glen Lett Story,Film,3.0,12026.000000,895.250000,15.0000,69457.966667,6682.283333,2021,7,3
4,CEG10030236,2021-07-01 05:22:18.200,2021-07-01 06:27:26.130,Harland Williams: A Force of Nature,Film,5.0,26004.033333,2913.966667,38.0000,43405.983333,5237.766667,2021,7,3
5,CEG10033188,2021-07-01 06:27:26.130,2021-07-01 08:15:03.170,37-Teen,Film,6.0,21269.050000,2840.200000,33.0000,71231.050000,9823.200000,2021,7,3
6,CEG10032522,2021-07-01 08:15:03.170,2021-07-01 09:18:04.060,"Debra DiGiovanni: Single, Awkward, Female - Clean",Film,8.0,31722.000000,4440.000000,45.0000,45407.400000,6351.300000,2021,7,3
7,CEG10027749,2021-07-01 09:18:04.060,2021-07-01 10:35:59.140,Harland Williams: What A Treat - Clean,Film,9.0,31932.600000,4459.700000,42.0000,58436.350000,8422.283333,2021,7,3
8,CD10010210,2021-07-01 10:35:59.140,2021-07-01 11:39:39.230,Tom Green: Live! - Clean,Film,10.0,18931.250000,2830.416667,25.0000,51849.200000,7526.016667,2021,7,3
9,CD10010171,2021-07-01 11:39:39.230,2021-07-01 13:01:03.210,Dave Foley: Relatively Well - Clean,Film,11.0,17725.050000,2528.400000,21.0000,67617.750000,10031.316667,2021,7,3


In [61]:
# ********************

# Adding Dayparts for the first time here

# ********************

In [62]:
if Timezone == 'West':
    Conditions1 =[
        (DF['Weekday'] < 5) & (DF['Hour'] < 6) & (DF['Hour'] >= 3),
        (DF['Weekday'] < 5) & (DF['Hour'] < 13) & (DF['Hour'] >= 6),
        (DF['Weekday'] < 5) & (DF['Hour'] < 17) & (DF['Hour'] >= 13),
        (DF['Weekday'] < 5) & (DF['Hour'] < 20) & (DF['Hour'] >= 17),
        (DF['Weekday'] < 5) & (DF['Hour'] < 23) & (DF['Hour'] >= 20),
        (DF['Weekday'] < 5) & ((DF['Hour'] < 3) | (DF['Hour'] >= 23)),
        (DF['Weekday'] >= 5) & (DF['Hour'] < 7) & (DF['Hour'] >= 3),
        (DF['Weekday'] >= 5) & (DF['Hour'] < 17) & (DF['Hour'] >= 7),
        (DF['Weekday'] >= 5) & (DF['Hour'] < 20) & (DF['Hour'] >= 17),
        (DF['Weekday'] >= 5) & (DF['Hour'] < 23) & (DF['Hour'] >= 20),
        (DF['Weekday'] >= 5) & ((DF['Hour'] < 3) | (DF['Hour'] >= 23)),

    ]
else:
    Conditions1 =[
        (DF['Weekday'] < 5) & (DF['Hour'] < 9) & (DF['Hour'] >= 6),
        (DF['Weekday'] < 5) & (DF['Hour'] < 16) & (DF['Hour'] >= 9),
        (DF['Weekday'] < 5) & (DF['Hour'] < 20) & (DF['Hour'] >= 16),
        (DF['Weekday'] < 5) & (DF['Hour'] < 23) & (DF['Hour'] >= 20),
        (DF['Weekday'] < 5) & ((DF['Hour'] < 2) | (DF['Hour'] >= 23)),
        (DF['Weekday'] < 5) & (DF['Hour'] < 6) & (DF['Hour'] >= 2),
        (DF['Weekday'] >= 5) & (DF['Hour'] < 10) & (DF['Hour'] >= 6),
        (DF['Weekday'] >= 5) & (DF['Hour'] < 20) & (DF['Hour'] >= 10),
        (DF['Weekday'] >= 5) & (DF['Hour'] < 23) & (DF['Hour'] >= 20),
        (DF['Weekday'] >= 5) & ((DF['Hour'] < 2) | (DF['Hour'] >= 23)),
        (DF['Weekday'] >= 5) & (DF['Hour'] < 6) & (DF['Hour'] >= 2),

    ]
    

Daypart1 = ['Weekday Morning','Weekday Day', 'Weekday Fringe','Prime', 'Late Night', 'Overnight',
           'Weekend Morning', 'Weekend Day', 'Prime (wknd)', 'Late Night (wknd)', 'Overnight (wknd)'
         ]

DF['Daypart']= np.select(Conditions1, Daypart1, default= np.nan)

In [63]:
DF

,Content ID,TS,TS End,Title,Series/Film,Hour,Minutes Viewed in Hour,Sessions in Hour,Length(min) in the Hour,Total Minutes,Total Session,Year,Month,Weekday,Daypart
0,CD10010210,2021-07-01 00:00:00.000,2021-07-01 01:05:41.080,Tom Green: Live! - Clean,Film,0.0,54322.000000,3958.000000,60.0,58742.000000,4244.583333,2021,7,3,Overnight
1,CD10010171,2021-07-01 01:05:41.080,2021-07-01 02:27:05.070,Dave Foley: Relatively Well - Clean,Film,1.0,48620.000000,3152.416667,55.0,71794.550000,4621.666667,2021,7,3,Overnight
2,CEG10027749,2021-07-01 02:27:05.070,2021-07-01 03:45:00.130,Harland Williams: What A Treat - Clean,Film,2.0,28324.450000,1795.750000,33.0,64402.450000,4481.500000,2021,7,3,Overnight
3,CEG10032516,2021-07-01 03:45:00.130,2021-07-01 05:22:18.200,Never Be Done: The Richard Glen Lett Story,Film,3.0,12026.000000,895.250000,15.0,69457.966667,6682.283333,2021,7,3,Weekday Morning
4,CEG10030236,2021-07-01 05:22:18.200,2021-07-01 06:27:26.130,Harland Williams: A Force of Nature,Film,5.0,26004.033333,2913.966667,38.0,43405.983333,5237.766667,2021,7,3,Weekday Morning
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15381,1000000003242,2023-04-30 19:08:29.290,2023-04-30 19:36:14.040,Phil Kopczinski: Full Grown Man,Film,19.0,0.000000,0.000000,0.0,22200.763801,2497.587431,2023,4,6,Prime (wknd)
15382,CD10010200,2023-04-30 19:36:14.040,2023-04-30 20:48:32.060,Rob Schneider: Soy Sauce and the Holocaust - C...,Film,20.0,43031.152000,4156.000000,48.0,62234.696000,6316.400000,2023,4,6,Late Night (wknd)
15383,CD10010178,2023-04-30 20:48:32.060,2023-04-30 22:15:57.040,Iliza Shlesinger: War Paint - Clean,Film,22.0,4720.062500,880.750000,15.0,73060.780500,6523.750000,2023,4,6,Late Night (wknd)
15384,CEG10027755,2023-04-30 22:15:57.040,2023-04-30 23:28:38.270,Doug Benson: Doug Dynasty - Clean,Film,23.0,14295.031333,1387.400000,28.0,28455.218833,4029.650000,2023,4,6,Overnight (wknd)


In [64]:
DF['Minutes per Minute']= DF['Minutes Viewed in Hour']/DF['Length(min) in the Hour']
DF['Sessions per Minute']= DF['Sessions in Hour']/DF['Length(min) in the Hour']
DF['Session Duration']= DF['Minutes Viewed in Hour']/DF['Sessions in Hour']
DF['Day']= DF['TS'].dt.day

In [65]:
DF

,Content ID,TS,TS End,Title,Series/Film,Hour,Minutes Viewed in Hour,Sessions in Hour,Length(min) in the Hour,Total Minutes,Total Session,Year,Month,Weekday,Daypart,Minutes per Minute,Sessions per Minute,Session Duration,Day
0,CD10010210,2021-07-01 00:00:00.000,2021-07-01 01:05:41.080,Tom Green: Live! - Clean,Film,0.0,54322.000000,3958.000000,60.0,58742.000000,4244.583333,2021,7,3,Overnight,905.366667,65.966667,13.724608,1
1,CD10010171,2021-07-01 01:05:41.080,2021-07-01 02:27:05.070,Dave Foley: Relatively Well - Clean,Film,1.0,48620.000000,3152.416667,55.0,71794.550000,4621.666667,2021,7,3,Overnight,884.000000,57.316667,15.423088,1
2,CEG10027749,2021-07-01 02:27:05.070,2021-07-01 03:45:00.130,Harland Williams: What A Treat - Clean,Film,2.0,28324.450000,1795.750000,33.0,64402.450000,4481.500000,2021,7,3,Overnight,858.316667,54.416667,15.773047,1
3,CEG10032516,2021-07-01 03:45:00.130,2021-07-01 05:22:18.200,Never Be Done: The Richard Glen Lett Story,Film,3.0,12026.000000,895.250000,15.0,69457.966667,6682.283333,2021,7,3,Weekday Morning,801.733333,59.683333,13.433119,1
4,CEG10030236,2021-07-01 05:22:18.200,2021-07-01 06:27:26.130,Harland Williams: A Force of Nature,Film,5.0,26004.033333,2913.966667,38.0,43405.983333,5237.766667,2021,7,3,Weekday Morning,684.316667,76.683333,8.923930,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15381,1000000003242,2023-04-30 19:08:29.290,2023-04-30 19:36:14.040,Phil Kopczinski: Full Grown Man,Film,19.0,0.000000,0.000000,0.0,22200.763801,2497.587431,2023,4,6,Prime (wknd),NaN,NaN,NaN,30
15382,CD10010200,2023-04-30 19:36:14.040,2023-04-30 20:48:32.060,Rob Schneider: Soy Sauce and the Holocaust - C...,Film,20.0,43031.152000,4156.000000,48.0,62234.696000,6316.400000,2023,4,6,Late Night (wknd),896.482333,86.583333,10.353983,30
15383,CD10010178,2023-04-30 20:48:32.060,2023-04-30 22:15:57.040,Iliza Shlesinger: War Paint - Clean,Film,22.0,4720.062500,880.750000,15.0,73060.780500,6523.750000,2023,4,6,Late Night (wknd),314.670833,58.716667,5.359140,30
15384,CEG10027755,2023-04-30 22:15:57.040,2023-04-30 23:28:38.270,Doug Benson: Doug Dynasty - Clean,Film,23.0,14295.031333,1387.400000,28.0,28455.218833,4029.650000,2023,4,6,Overnight (wknd),510.536833,49.550000,10.303468,30


# Month Breakdown

In [66]:
Condwk=[
    DF['Weekday']== 0,DF['Weekday']== 1,DF['Weekday']== 2,DF['Weekday']== 3,
    DF['Weekday']== 4,DF['Weekday']== 5,DF['Weekday']== 6   
]

wkdays= ['Mon','Tue','Wed','Thu','Fri','Sat','Sun']

DF['Weekday(name)']= np.select(Condwk,wkdays, default=np.nan)

In [67]:
DF= DF.loc[:,('Content ID','TS','TS End','Length(min) in the Hour','Day','Weekday','Weekday(name)','Hour','Series/Film',
             'Title','Daypart','Minutes per Minute','Sessions per Minute','Session Duration','Year','Month')]

In [68]:
DF

,Content ID,TS,TS End,Length(min) in the Hour,Day,Weekday,Weekday(name),Hour,Series/Film,Title,Daypart,Minutes per Minute,Sessions per Minute,Session Duration,Year,Month
0,CD10010210,2021-07-01 00:00:00.000,2021-07-01 01:05:41.080,60.0,1,3,Thu,0.0,Film,Tom Green: Live! - Clean,Overnight,905.366667,65.966667,13.724608,2021,7
1,CD10010171,2021-07-01 01:05:41.080,2021-07-01 02:27:05.070,55.0,1,3,Thu,1.0,Film,Dave Foley: Relatively Well - Clean,Overnight,884.000000,57.316667,15.423088,2021,7
2,CEG10027749,2021-07-01 02:27:05.070,2021-07-01 03:45:00.130,33.0,1,3,Thu,2.0,Film,Harland Williams: What A Treat - Clean,Overnight,858.316667,54.416667,15.773047,2021,7
3,CEG10032516,2021-07-01 03:45:00.130,2021-07-01 05:22:18.200,15.0,1,3,Thu,3.0,Film,Never Be Done: The Richard Glen Lett Story,Weekday Morning,801.733333,59.683333,13.433119,2021,7
4,CEG10030236,2021-07-01 05:22:18.200,2021-07-01 06:27:26.130,38.0,1,3,Thu,5.0,Film,Harland Williams: A Force of Nature,Weekday Morning,684.316667,76.683333,8.923930,2021,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15381,1000000003242,2023-04-30 19:08:29.290,2023-04-30 19:36:14.040,0.0,30,6,Sun,19.0,Film,Phil Kopczinski: Full Grown Man,Prime (wknd),NaN,NaN,NaN,2023,4
15382,CD10010200,2023-04-30 19:36:14.040,2023-04-30 20:48:32.060,48.0,30,6,Sun,20.0,Film,Rob Schneider: Soy Sauce and the Holocaust - C...,Late Night (wknd),896.482333,86.583333,10.353983,2023,4
15383,CD10010178,2023-04-30 20:48:32.060,2023-04-30 22:15:57.040,15.0,30,6,Sun,22.0,Film,Iliza Shlesinger: War Paint - Clean,Late Night (wknd),314.670833,58.716667,5.359140,2023,4
15384,CEG10027755,2023-04-30 22:15:57.040,2023-04-30 23:28:38.270,28.0,30,6,Sun,23.0,Film,Doug Benson: Doug Dynasty - Clean,Overnight (wknd),510.536833,49.550000,10.303468,2023,4


In [69]:
DF.dropna(inplace=True)
DF.sort_values(by=['TS','TS End'])
DF['Hour']=DF['Hour'].astype(int)
DF.rename(columns={'Hour':'Hour(24-Pac)','Weekday':'Weekday(num)','Weekday(name)':'Weekday'},inplace=True)

In [70]:
Con4=[
    (DF['Hour(24-Pac)']== 0),(DF['Hour(24-Pac)']== 1),(DF['Hour(24-Pac)']== 2),(DF['Hour(24-Pac)']== 3),
    (DF['Hour(24-Pac)']== 4),(DF['Hour(24-Pac)']== 5),(DF['Hour(24-Pac)']== 6),(DF['Hour(24-Pac)']== 7),
    (DF['Hour(24-Pac)']== 8),(DF['Hour(24-Pac)']== 9),(DF['Hour(24-Pac)']== 10),(DF['Hour(24-Pac)']== 11),
    (DF['Hour(24-Pac)']== 12),(DF['Hour(24-Pac)']== 13),(DF['Hour(24-Pac)']== 14),(DF['Hour(24-Pac)']== 15),
    (DF['Hour(24-Pac)']== 16),(DF['Hour(24-Pac)']== 17),(DF['Hour(24-Pac)']== 18),(DF['Hour(24-Pac)']== 19),
    (DF['Hour(24-Pac)']== 20),(DF['Hour(24-Pac)']== 21),(DF['Hour(24-Pac)']== 22),(DF['Hour(24-Pac)']== 23),
    
]

if Timezone == 'West':

    pac=['0am-pt','1am-pt','2am-pt','3am-pt','4am-pt','5am-pt',
        '6am-pt','7am-pt','8am-pt','9am-pt','10am-pt','11am-pt',
        '12pm-pt','1pm-pt','2pm-pt','3pm-pt','4pm-pt','5pm-pt',
        '6pm-pt','7pm-pt','8pm-pt','9pm-pt','10pm-pt','11pm-pt',]

    east=['3am-et','4am-et','5am-et','6am-et','7am-et','8am-et',
          '9am-et','10am-et','11am-et','12pm-et','1pm-et','2pm-et',
          '3pm-et','4pm-et','5pm-et','6pm-et','7pm-et','8pm-et',
          '9pm-et','10pm-et','11pm-et','0am-et','1am-et','2am-et']
else:
    pac=['9pm-pt','10pm-pt','11pm-pt','0am-pt','1am-pt','2am-pt',
         '3am-pt','4am-pt','5am-pt','6am-pt','7am-pt','8am-pt',
         '9am-pt','10am-pt','11am-pt','12pm-pt','1pm-pt','2pm-pt',
         '3pm-pt','4pm-pt','5pm-pt','6pm-pt','7pm-pt','8pm-pt']

    east=['0am-et','1am-et','2am-et','3am-et','4am-et','5am-et',
          '6am-et','7am-et','8am-et','9am-et','10am-et','11am-et',
          '12pm-et','1pm-et','2pm-et','3pm-et','4pm-et','5pm-et',
          '6pm-et','7pm-et','8pm-et','9pm-et','10pm-et','11pm-et']


DF['Pacific_Time']=np.select(Con4,pac,default=np.nan)
DF['East_Time']=np.select(Con4,east,default=np.nan)

In [71]:
DF= DF.loc[:,('Content ID','TS','TS End','Length(min) in the Hour','Day','Weekday(num)','Weekday','Hour(24-Pac)','Pacific_Time',
              'East_Time','Series/Film','Title','Daypart','Minutes per Minute','Sessions per Minute','Session Duration','Year','Month')]

In [72]:
DF

,Content ID,TS,TS End,Length(min) in the Hour,Day,Weekday(num),Weekday,Hour(24-Pac),Pacific_Time,East_Time,Series/Film,Title,Daypart,Minutes per Minute,Sessions per Minute,Session Duration,Year,Month
0,CD10010210,2021-07-01 00:00:00.000,2021-07-01 01:05:41.080,60.0,1,3,Thu,0,0am-pt,3am-et,Film,Tom Green: Live! - Clean,Overnight,905.366667,65.966667,13.724608,2021,7
1,CD10010171,2021-07-01 01:05:41.080,2021-07-01 02:27:05.070,55.0,1,3,Thu,1,1am-pt,4am-et,Film,Dave Foley: Relatively Well - Clean,Overnight,884.000000,57.316667,15.423088,2021,7
2,CEG10027749,2021-07-01 02:27:05.070,2021-07-01 03:45:00.130,33.0,1,3,Thu,2,2am-pt,5am-et,Film,Harland Williams: What A Treat - Clean,Overnight,858.316667,54.416667,15.773047,2021,7
3,CEG10032516,2021-07-01 03:45:00.130,2021-07-01 05:22:18.200,15.0,1,3,Thu,3,3am-pt,6am-et,Film,Never Be Done: The Richard Glen Lett Story,Weekday Morning,801.733333,59.683333,13.433119,2021,7
4,CEG10030236,2021-07-01 05:22:18.200,2021-07-01 06:27:26.130,38.0,1,3,Thu,5,5am-pt,8am-et,Film,Harland Williams: A Force of Nature,Weekday Morning,684.316667,76.683333,8.923930,2021,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15379,CD10010242,2023-04-30 16:57:06.010,2023-04-30 18:07:59.230,7.0,30,6,Sun,18,6pm-pt,9pm-et,Film,Kevin Hart Presents: Lil Rel Howery - RELevent...,Prime (wknd),611.633167,84.816667,7.211238,2023,4
15380,CD10010199,2023-04-30 18:07:59.230,2023-04-30 19:08:29.290,8.0,30,6,Sun,19,7pm-pt,10pm-et,Film,Rita Rudner: A Tale of Two Dresses - Clean,Prime (wknd),800.147667,90.016667,8.888884,2023,4
15382,CD10010200,2023-04-30 19:36:14.040,2023-04-30 20:48:32.060,48.0,30,6,Sun,20,8pm-pt,11pm-et,Film,Rob Schneider: Soy Sauce and the Holocaust - C...,Late Night (wknd),896.482333,86.583333,10.353983,2023,4
15383,CD10010178,2023-04-30 20:48:32.060,2023-04-30 22:15:57.040,15.0,30,6,Sun,22,10pm-pt,1am-et,Film,Iliza Shlesinger: War Paint - Clean,Late Night (wknd),314.670833,58.716667,5.359140,2023,4


# Building Confidence Intervals

In [73]:
DF.sort_values(by=['TS','Hour(24-Pac)'], inplace=True)
DF=DF.reset_index()

In [74]:
# **************************

# Issue

# -fixing error when title changes day, month, or year
# -only changes the last hour of a title

# **************************

In [75]:
DF.loc[(DF['TS'].dt.hour != DF['TS End'].dt.hour) & (DF['Hour(24-Pac)']==0),'Day'] = DF['TS End'].dt.day
DF.loc[(DF['TS'].dt.hour != DF['TS End'].dt.hour) & (DF['Hour(24-Pac)']==0),'Month'] = DF['TS End'].dt.month
DF.loc[(DF['TS'].dt.hour != DF['TS End'].dt.hour) & (DF['Hour(24-Pac)']==0),'Year'] = DF['TS End'].dt.year

In [76]:
DF.loc[(DF['TS'].dt.hour != DF['TS End'].dt.hour) & (DF['Hour(24-Pac)']==1),'Day'] = DF['TS End'].dt.day
DF.loc[(DF['TS'].dt.hour != DF['TS End'].dt.hour) & (DF['Hour(24-Pac)']==1),'Month'] = DF['TS End'].dt.month
DF.loc[(DF['TS'].dt.hour != DF['TS End'].dt.hour) & (DF['Hour(24-Pac)']==1),'Year'] = DF['TS End'].dt.year

In [77]:
DF.loc[(DF['TS'].dt.hour != DF['TS End'].dt.hour) & (DF['Hour(24-Pac)']==2),'Day'] = DF['TS End'].dt.day
DF.loc[(DF['TS'].dt.hour != DF['TS End'].dt.hour) & (DF['Hour(24-Pac)']==2),'Month'] = DF['TS End'].dt.month
DF.loc[(DF['TS'].dt.hour != DF['TS End'].dt.hour) & (DF['Hour(24-Pac)']==2),'Year'] = DF['TS End'].dt.year

In [78]:
DF.loc[(DF['TS'].dt.hour != DF['TS End'].dt.hour) & (DF['Hour(24-Pac)']==0),'Weekday(num)'] = DF['TS End'].dt.weekday

In [79]:
Condwk=[
    DF['Weekday(num)']== 0,DF['Weekday(num)']== 1,DF['Weekday(num)']== 2,DF['Weekday(num)']== 3,
    DF['Weekday(num)']== 4,DF['Weekday(num)']== 5,DF['Weekday(num)']== 6   
]

wkdays= ['Mon','Tue','Wed','Thu','Fri','Sat','Sun']

DF['Weekday(name)']= np.select(Condwk,wkdays, default=np.nan)

In [80]:
if Timezone == 'West':
    Conditions1 =[
        (DF['Weekday(num)'] < 5) & (DF['Hour(24-Pac)'] < 6) & (DF['Hour(24-Pac)'] >= 3),
        (DF['Weekday(num)'] < 5) & (DF['Hour(24-Pac)'] < 13) & (DF['Hour(24-Pac)'] >= 6),
        (DF['Weekday(num)'] < 5) & (DF['Hour(24-Pac)'] < 17) & (DF['Hour(24-Pac)'] >= 13),
        (DF['Weekday(num)'] < 5) & (DF['Hour(24-Pac)'] < 20) & (DF['Hour(24-Pac)'] >= 17),
        (DF['Weekday(num)'] < 5) & (DF['Hour(24-Pac)'] < 23) & (DF['Hour(24-Pac)'] >= 20),
        (DF['Weekday(num)'] < 5) & ((DF['Hour(24-Pac)'] < 3) | (DF['Hour(24-Pac)'] >= 23)),
        (DF['Weekday(num)'] >= 5) & (DF['Hour(24-Pac)'] < 7) & (DF['Hour(24-Pac)'] >= 3),
        (DF['Weekday(num)'] >= 5) & (DF['Hour(24-Pac)'] < 17) & (DF['Hour(24-Pac)'] >= 7),
        (DF['Weekday(num)'] >= 5) & (DF['Hour(24-Pac)'] < 20) & (DF['Hour(24-Pac)'] >= 17),
        (DF['Weekday(num)'] >= 5) & (DF['Hour(24-Pac)'] < 23) & (DF['Hour(24-Pac)'] >= 20),
        (DF['Weekday(num)'] >= 5) & ((DF['Hour(24-Pac)'] < 3) | (DF['Hour(24-Pac)'] >= 23)),

    ]
else:
    Conditions1 =[
        (DF['Weekday(num)'] < 5) & (DF['Hour(24-Pac)'] < 9) & (DF['Hour(24-Pac)'] >= 6),
        (DF['Weekday(num)'] < 5) & (DF['Hour(24-Pac)'] < 16) & (DF['Hour(24-Pac)'] >= 9),
        (DF['Weekday(num)'] < 5) & (DF['Hour(24-Pac)'] < 20) & (DF['Hour(24-Pac)'] >= 16),
        (DF['Weekday(num)'] < 5) & (DF['Hour(24-Pac)'] < 23) & (DF['Hour(24-Pac)'] >= 20),
        (DF['Weekday(num)'] < 5) & ((DF['Hour(24-Pac)'] < 2) | (DF['Hour(24-Pac)'] >= 23)),
        (DF['Weekday(num)'] < 5) & (DF['Hour(24-Pac)'] < 6) & (DF['Hour(24-Pac)'] >= 2),
        (DF['Weekday(num)'] >= 5) & (DF['Hour(24-Pac)'] < 10) & (DF['Hour(24-Pac)'] >= 6),
        (DF['Weekday(num)'] >= 5) & (DF['Hour(24-Pac)'] < 20) & (DF['Hour(24-Pac)'] >= 10),
        (DF['Weekday(num)'] >= 5) & (DF['Hour(24-Pac)'] < 23) & (DF['Hour(24-Pac)'] >= 20),
        (DF['Weekday(num)'] >= 5) & ((DF['Hour(24-Pac)'] < 2) | (DF['Hour(24-Pac)'] >= 23)),
        (DF['Weekday(num)'] >= 5) & (DF['Hour(24-Pac)'] < 6) & (DF['Hour(24-Pac)'] >= 2),

    ]
    

Daypart1 = ['Weekday Morning','Weekday Day', 'Weekday Fringe','Prime', 'Late Night', 'Overnight',
           'Weekend Morning', 'Weekend Day', 'Prime (wknd)', 'Late Night (wknd)', 'Overnight (wknd)'
         ]

DF['Daypart1']= np.select(Conditions1, Daypart1, default= np.nan)

In [81]:
DF.rename(columns={'Daypart':'no','Daypart1':'Daypart'}, inplace=True)

In [82]:
DF=DF.loc[:,('Content ID','TS','TS End','Length(min) in the Hour','Day','Weekday(num)','Weekday(name)','Hour(24-Pac)',
            'Pacific_Time','East_Time','Series/Film','Title','Daypart','Minutes per Minute',
             'Sessions per Minute','Session Duration','Year','Month')]

In [83]:
Cond=[
    (DF['Weekday(num)']== 5) | (DF['Weekday(num)']==6),
    (DF['Weekday(num)']== 0)|(DF['Weekday(num)']==1)|(DF['Weekday(num)']== 2)|(DF['Weekday(num)']==3)|
    (DF['Weekday(num)']== 4)
]

Values=['Weekend','Weekday']

DF['Weekdays/Weekends']= np.select(Cond,Values)

# What is the Average

In [84]:
# ********************************

# ISSUE

# -defining what an Average is

# ***********************

In [85]:
HR=DF.groupby(['Year','Month','Weekdays/Weekends','Day','Hour(24-Pac)']).mean().reset_index()

In [86]:
HR.head(10)

,Year,Month,Weekdays/Weekends,Day,Hour(24-Pac),Length(min) in the Hour,Weekday(num),Minutes per Minute,Sessions per Minute,Session Duration
0,2021,7,Weekday,1,0,60.0,3,905.366667,65.966667,13.724608
1,2021,7,Weekday,1,1,30.0,3,884.000000,57.316667,15.423088
2,2021,7,Weekday,1,2,30.0,3,858.316667,54.416667,15.773047
3,2021,7,Weekday,1,3,30.0,3,801.733333,59.683333,13.433119
4,2021,7,Weekday,1,4,60.0,3,706.283333,68.333333,10.335854
5,2021,7,Weekday,1,5,30.0,3,684.316667,76.683333,8.923930
6,2021,7,Weekday,1,6,30.0,3,644.516667,86.066667,7.488575
7,2021,7,Weekday,1,7,60.0,3,656.466667,91.716667,7.157550
8,2021,7,Weekday,1,8,30.0,3,704.933333,98.666667,7.144595
9,2021,7,Weekday,1,9,30.0,3,760.300000,106.183333,7.160257


In [87]:
Hourly_AVG= HR.loc[:,('Minutes per Minute','Weekdays/Weekends','Hour(24-Pac)','Year','Month')].groupby(['Year','Month','Weekdays/Weekends',
                                        'Hour(24-Pac)']).mean().reset_index()
Hourly_STD= HR.loc[:,('Minutes per Minute','Weekdays/Weekends','Hour(24-Pac)','Year','Month')].groupby(['Year','Month','Weekdays/Weekends',
                                        'Hour(24-Pac)']).std().reset_index()
Hourly_Count= HR.loc[:,('Minutes per Minute','Weekdays/Weekends','Hour(24-Pac)','Year','Month')].groupby(['Year','Month','Weekdays/Weekends',
                                        'Hour(24-Pac)']).count().reset_index()

In [88]:
Hourly_AVG.rename(columns={'Minutes per Minute':'Hourly AVG'},inplace=True)
Hourly_STD.rename(columns={'Minutes per Minute':'Hourly STD'},inplace=True)
Hourly_Count.rename(columns={'Minutes per Minute':'Hourly Count'},inplace=True)

In [89]:
DF= pd.merge(DF,Hourly_AVG, on=['Year','Month','Weekdays/Weekends','Hour(24-Pac)'], how='left')
DF= pd.merge(DF,Hourly_STD, on=['Year','Month','Weekdays/Weekends','Hour(24-Pac)'], how='left')
DF= pd.merge(DF,Hourly_Count, on=['Year','Month','Weekdays/Weekends','Hour(24-Pac)'], how='left')

In [90]:
DF.head(10)

,Content ID,TS,TS End,Length(min) in the Hour,Day,Weekday(num),Weekday(name),Hour(24-Pac),Pacific_Time,East_Time,Series/Film,Title,Daypart,Minutes per Minute,Sessions per Minute,Session Duration,Year,Month,Weekdays/Weekends,Hourly AVG,Hourly STD,Hourly Count
0,CD10010210,2021-07-01 00:00:00.000,2021-07-01 01:05:41.080,60.0,1,3,Thu,0,0am-pt,3am-et,Film,Tom Green: Live! - Clean,Overnight,905.366667,65.966667,13.724608,2021,7,Weekday,994.709848,235.542747,22
1,CD10010210,2021-07-01 00:00:00.000,2021-07-01 01:05:41.080,5.0,1,3,Thu,1,1am-pt,4am-et,Film,Tom Green: Live! - Clean,Overnight,884.000000,57.316667,15.423088,2021,7,Weekday,849.512121,190.435589,22
2,CD10010171,2021-07-01 01:05:41.080,2021-07-01 02:27:05.070,55.0,1,3,Thu,1,1am-pt,4am-et,Film,Dave Foley: Relatively Well - Clean,Overnight,884.000000,57.316667,15.423088,2021,7,Weekday,849.512121,190.435589,22
3,CD10010171,2021-07-01 01:05:41.080,2021-07-01 02:27:05.070,27.0,1,3,Thu,2,2am-pt,5am-et,Film,Dave Foley: Relatively Well - Clean,Overnight,858.316667,54.416667,15.773047,2021,7,Weekday,773.711364,156.460209,22
4,CEG10027749,2021-07-01 02:27:05.070,2021-07-01 03:45:00.130,33.0,1,3,Thu,2,2am-pt,5am-et,Film,Harland Williams: What A Treat - Clean,Overnight,858.316667,54.416667,15.773047,2021,7,Weekday,773.711364,156.460209,22
5,CEG10027749,2021-07-01 02:27:05.070,2021-07-01 03:45:00.130,45.0,1,3,Thu,3,3am-pt,6am-et,Film,Harland Williams: What A Treat - Clean,Weekday Morning,801.733333,59.683333,13.433119,2021,7,Weekday,723.616667,122.115810,22
6,CEG10032516,2021-07-01 03:45:00.130,2021-07-01 05:22:18.200,15.0,1,3,Thu,3,3am-pt,6am-et,Film,Never Be Done: The Richard Glen Lett Story,Weekday Morning,801.733333,59.683333,13.433119,2021,7,Weekday,723.616667,122.115810,22
7,CEG10032516,2021-07-01 03:45:00.130,2021-07-01 05:22:18.200,60.0,1,3,Thu,4,4am-pt,7am-et,Film,Never Be Done: The Richard Glen Lett Story,Weekday Morning,706.283333,68.333333,10.335854,2021,7,Weekday,692.358333,110.220167,22
8,CEG10032516,2021-07-01 03:45:00.130,2021-07-01 05:22:18.200,22.0,1,3,Thu,5,5am-pt,8am-et,Film,Never Be Done: The Richard Glen Lett Story,Weekday Morning,684.316667,76.683333,8.923930,2021,7,Weekday,692.197727,120.233459,22
9,CEG10030236,2021-07-01 05:22:18.200,2021-07-01 06:27:26.130,38.0,1,3,Thu,5,5am-pt,8am-et,Film,Harland Williams: A Force of Nature,Weekday Morning,684.316667,76.683333,8.923930,2021,7,Weekday,692.197727,120.233459,22


In [91]:
# *****************************

# -Here i am creating upper and lower bounds using confidence intervals around the average

# ******************************

# Adjust T-Value

In [92]:
data={'Count':[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30],
      'Tvalue':[12.706,4.303,3.182,2.776,2.571,2.447,2.365,2.306,2.262,2.228,2.201,2.179,2.16,2.145,2.131,2.12,2.11,2.101,2.093,2.086,2.08,2.074,2.069,2.064,2.06,2.056,2.052,2.048,2.045,2.042]   
}
T_values=pd.DataFrame(data)

In [93]:
DF=pd.merge(DF,T_values, left_on='Hourly Count',right_on='Count', how='left')

In [94]:
# formula for upper and lower bounds are here

DF['Upper CI(95%)']= DF['Hourly AVG']+(DF['Tvalue']* (DF['Hourly STD']/(DF['Hourly Count']**(1/2))))
DF['Lower CI(95%)']= DF['Hourly AVG']-(DF['Tvalue']* (DF['Hourly STD']/(DF['Hourly Count']**(1/2))))

In [95]:
Con1=[
    (DF['Minutes per Minute'] > DF['Upper CI(95%)']),
    (DF['Minutes per Minute'] < DF['Lower CI(95%)']),
    (DF['Minutes per Minute'] <= DF['Upper CI(95%)']) & (DF['Minutes per Minute'] >= DF['Lower CI(95%)'])     
]

Values1= ['Above','Below','Neutral']

DF['Above/Below/Neutral']=np.select(Con1,Values1, default=np.nan)

In [96]:
Con1=[
    (DF['Above/Below/Neutral']== 'Above'), (DF['Above/Below/Neutral']== 'Below'), (DF['Above/Below/Neutral']== 'Neutral'),
]

Values1=[(DF['Minutes per Minute']-DF['Upper CI(95%)']),
         (DF['Minutes per Minute']-DF['Lower CI(95%)']),
         0]
DF['Minutes Diff CI']= np.select(Con1,Values1, default=np.nan)

In [97]:
DF['%Minutes Diff CI']=(DF['Minutes Diff CI']/DF['Hourly AVG'])

In [98]:
DF['Minutes Diff']= DF['Minutes per Minute']-DF['Hourly AVG']
DF['%Minutes Diff']= DF['Minutes Diff']/DF['Hourly AVG']
DF['Total Minutes in Hour']= DF['Minutes per Minute']*DF['Length(min) in the Hour']
DF['Total Sessions in Hour']= DF['Sessions per Minute']*DF['Length(min) in the Hour']

del DF['Count']

In [99]:
DF['TS_test']= DF['TS'].dt.day

In [100]:
CondM=[
    (DF['Month']== 1) | (DF['Month']==2) | (DF['Month']==3),
    (DF['Month']== 4) | (DF['Month']==5) | (DF['Month']==6),
    (DF['Month']== 7) | (DF['Month']==8) | (DF['Month']==9),
    (DF['Month']== 10) | (DF['Month']==11) | (DF['Month']==12)
]

ValuesQ=['Q1','Q2','Q3','Q4']

DF['Quarter']= np.select(CondM,ValuesQ)

In [101]:
Condm=[
    (DF['Month']== 1) , (DF['Month']==2) , (DF['Month']==3),
    (DF['Month']== 4) , (DF['Month']==5) , (DF['Month']==6),
    (DF['Month']== 7) , (DF['Month']==8) , (DF['Month']==9),
    (DF['Month']== 10) , (DF['Month']==11) , (DF['Month']==12)
]

Valuesq=['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']

DF['Month(name)']= np.select(Condm,Valuesq)

In [102]:
DF['M_Y']= DF['Month'].astype(str)+'-'+ DF['Year'].astype(str)

# Created Day(Adj.)

In [103]:
# ***********************************

# ISSUES

# - When Overlapping Daily minutes data by date, days of the week don't like up with days of the month so i created a new label


# ***********************************

In [104]:
if Channel == 'ElRey':
    DF = DF.loc[~((DF['Month']==11) & (DF['Day']<17) & (DF['Year']==2021))]
    DF = DF.loc[~((DF['Month']==10) & (DF['Year']==2021))]

In [105]:
if Channel == 'FilmDetective':
    DF = DF.loc[~((DF['Month']==8) & (DF['Year']==2022))]
    DF = DF.loc[~((DF['Month']==9) & (DF['Year']==2022))]

In [106]:
M_Y_list = DF['M_Y'].unique()

In [107]:
M_Y_list

array(['7-2021', '8-2021', '9-2021', '10-2021', '11-2021', '12-2021',
       '1-2022', '2-2022', '3-2022', '4-2022', '5-2022', '6-2022',
       '7-2022', '8-2022', '9-2022', '10-2022', '11-2022', '12-2022',
       '1-2023', '2-2023', '3-2023', '4-2023'], dtype=object)

In [108]:
# M_Ydf= pd.DataFrame()
    
# for i in M_Y_list:
#     MY_DF= DF.loc[DF['M_Y']== i,('M_Y','Day','Weekday(num)')].drop_duplicates()
#     MY_DF['Day (adj.)']= MY_DF.loc[:,'Day']
#     if (MY_DF.loc[MY_DF['Day'] == 1,'Weekday(num)']==1).bool():
#         MY_DF['Day (adj.)'] = MY_DF['Day (adj.)'] + 1
#     elif (MY_DF.loc[MY_DF['Day'] == 1,'Weekday(num)']==2).bool():
#         MY_DF['Day (adj.)'] = MY_DF['Day (adj.)'] + 2
#     elif (MY_DF.loc[MY_DF['Day'] == 1,'Weekday(num)']==3).bool():
#         MY_DF['Day (adj.)'] = MY_DF['Day (adj.)'] + 3
#     elif (MY_DF.loc[MY_DF['Day'] == 1,'Weekday(num)']==4).bool():
#         MY_DF['Day (adj.)'] = MY_DF['Day (adj.)'] + 4
#     elif (MY_DF.loc[MY_DF['Day'] == 1,'Weekday(num)']==5).bool():
#         MY_DF['Day (adj.)'] = MY_DF['Day (adj.)'] + 5
#     elif (MY_DF.loc[MY_DF['Day'] == 1,'Weekday(num)']==6).bool():
#         MY_DF['Day (adj.)'] = MY_DF['Day (adj.)'] + 6
#     else:
#         (MY_DF.loc[MY_DF['Day'] == 1,'Weekday(num)']==0).bool()
#         MY_DF['Day (adj.)'] = MY_DF['Day (adj.)'] + 0
        
        
#     M_Ydf = M_Ydf.append(MY_DF)

In [109]:
M_Ydf= pd.DataFrame()
    
for i in M_Y_list:
    MY_DF= DF.loc[DF['M_Y']== i,('M_Y','Day','Weekday(num)')].drop_duplicates()
    MY_DF['Day (adj.)']= MY_DF.loc[:,'Day']
    if (MY_DF.loc[MY_DF['Day'] == 28,'Weekday(num)']==1).bool():
        MY_DF['Day (adj.)'] = MY_DF['Day (adj.)'] + 2
    elif (MY_DF.loc[MY_DF['Day'] == 28,'Weekday(num)']==2).bool():
        MY_DF['Day (adj.)'] = MY_DF['Day (adj.)'] + 3
    elif (MY_DF.loc[MY_DF['Day'] == 28,'Weekday(num)']==3).bool():
        MY_DF['Day (adj.)'] = MY_DF['Day (adj.)'] + 4
    elif (MY_DF.loc[MY_DF['Day'] == 28,'Weekday(num)']==4).bool():
        MY_DF['Day (adj.)'] = MY_DF['Day (adj.)'] + 5
    elif (MY_DF.loc[MY_DF['Day'] == 28,'Weekday(num)']==5).bool():
        MY_DF['Day (adj.)'] = MY_DF['Day (adj.)'] + 6
    elif (MY_DF.loc[MY_DF['Day'] == 28,'Weekday(num)']==6).bool():
        MY_DF['Day (adj.)'] = MY_DF['Day (adj.)'] + 0
    else:
        (MY_DF.loc[MY_DF['Day'] == 28,'Weekday(num)']==0).bool()
        MY_DF['Day (adj.)'] = MY_DF['Day (adj.)'] + 1
        
        
    M_Ydf = M_Ydf.append(MY_DF)

In [110]:
DF = pd.merge(DF,M_Ydf.loc[:,('M_Y','Day','Day (adj.)')], on = ('M_Y','Day'), how='left')

In [111]:
Cond_wknd = [ (DF['Weekdays/Weekends'] == 'Weekday') , (DF['Weekdays/Weekends'] == 'Weekend') ]

Values_wknd = ['Wk','Wknd']

DF['Wk/Wknd'] = np.select(Cond_wknd,Values_wknd)

In [112]:
if Timezone == 'West':
    DF['TS_East'] = DF['TS'] + timedelta(hours=3)
else:
    DF['TS_East'] = ''

In [113]:
DF['Title']=DF.groupby('Content ID')['Title'].transform('last')

# Add Number To Dayparts

In [114]:
cond = [
    DF['Daypart']=='Overnight', DF['Daypart']=='Weekday Morning', DF['Daypart']=='Weekday Day', DF['Daypart']=='Weekday Fringe', DF['Daypart']=='Prime', DF['Daypart']=='Late Night',
    DF['Daypart']=='Overnight (wknd)', DF['Daypart']=='Weekend Morning', DF['Daypart']=='Weekend Day', DF['Daypart']=='Prime (wknd)', DF['Daypart']=='Late Night (wknd)'
]

values = ['1.Overnight','2.Weekday Morning','3.Weekday Day','4.Weekday Fringe','5.Prime','6.Late Night',
          '1.Overnight (wknd)','2.Weekend Morning','3.Weekend Day','4.Prime (wknd)','5.Late Night (wknd)']

DF['Daypart#'] = np.select(cond,values)

In [115]:
DF['Title'] = DF.groupby('Content ID')['Title'].transform('last')

In [116]:
DF=DF.loc[:,('Content ID','TS','TS End','TS_East','M_Y','Year','Quarter','Month','Month(name)','Day','Day (adj.)','Weekday(name)','Hour(24-Pac)','Pacific_Time','East_Time','Weekdays/Weekends','Wk/Wknd',
             'Daypart','Daypart#','Series/Film','Title','Length(min) in the Hour','Total Minutes in Hour','Total Sessions in Hour','Minutes per Minute','Sessions per Minute',
            'Session Duration','Minutes Diff','%Minutes Diff','Hourly AVG','Hourly STD','Upper CI(95%)','Lower CI(95%)','Minutes Diff CI','%Minutes Diff CI','Above/Below/Neutral')]

In [117]:
# LAST DF_______________LAST DF______________________________

DF

,Content ID,TS,TS End,TS_East,M_Y,Year,Quarter,Month,Month(name),Day,Day (adj.),Weekday(name),Hour(24-Pac),Pacific_Time,East_Time,Weekdays/Weekends,Wk/Wknd,Daypart,Daypart#,Series/Film,Title,Length(min) in the Hour,Total Minutes in Hour,Total Sessions in Hour,Minutes per Minute,Sessions per Minute,Session Duration,Minutes Diff,%Minutes Diff,Hourly AVG,Hourly STD,Upper CI(95%),Lower CI(95%),Minutes Diff CI,%Minutes Diff CI,Above/Below/Neutral
0,CD10010210,2021-07-01 00:00:00.000,2021-07-01 01:05:41.080,2021-07-01 03:00:00.000,7-2021,2021,Q3,7,Jul,1,4,Thu,0,0am-pt,3am-et,Weekday,Wk,Overnight,1.Overnight,Film,Tom Green: Live! - Clean,60.00,54322.000000,3958.000000,905.366667,65.966667,13.724608,-89.343182,-0.089818,994.709848,235.542747,1098.861737,890.557960,0.000000,0.000000,Neutral
1,CD10010210,2021-07-01 00:00:00.000,2021-07-01 01:05:41.080,2021-07-01 03:00:00.000,7-2021,2021,Q3,7,Jul,1,4,Thu,1,1am-pt,4am-et,Weekday,Wk,Overnight,1.Overnight,Film,Tom Green: Live! - Clean,5.00,4420.000000,286.583333,884.000000,57.316667,15.423088,34.487879,0.040597,849.512121,190.435589,933.718604,765.305639,0.000000,0.000000,Neutral
2,CD10010171,2021-07-01 01:05:41.080,2021-07-01 02:27:05.070,2021-07-01 04:05:41.080,7-2021,2021,Q3,7,Jul,1,4,Thu,1,1am-pt,4am-et,Weekday,Wk,Overnight,1.Overnight,Film,Dave Foley: Relatively Well - Clean,55.00,48620.000000,3152.416667,884.000000,57.316667,15.423088,34.487879,0.040597,849.512121,190.435589,933.718604,765.305639,0.000000,0.000000,Neutral
3,CD10010171,2021-07-01 01:05:41.080,2021-07-01 02:27:05.070,2021-07-01 04:05:41.080,7-2021,2021,Q3,7,Jul,1,4,Thu,2,2am-pt,5am-et,Weekday,Wk,Overnight,1.Overnight,Film,Dave Foley: Relatively Well - Clean,27.00,23174.550000,1469.250000,858.316667,54.416667,15.773047,84.605303,0.109350,773.711364,156.460209,842.894671,704.528057,15.421996,0.019932,Above
4,CEG10027749,2021-07-01 02:27:05.070,2021-07-01 03:45:00.130,2021-07-01 05:27:05.070,7-2021,2021,Q3,7,Jul,1,4,Thu,2,2am-pt,5am-et,Weekday,Wk,Overnight,1.Overnight,Film,Harland Williams: What A Treat - Clean,33.00,28324.450000,1795.750000,858.316667,54.416667,15.773047,84.605303,0.109350,773.711364,156.460209,842.894671,704.528057,15.421996,0.019932,Above
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30473,CD10010178,2023-04-30 20:48:32.060,2023-04-30 22:15:57.040,2023-04-30 23:48:32.060,4-2023,2023,Q2,4,Apr,30,35,Sun,21,9pm-pt,0am-et,Weekend,Wknd,Late Night (wknd),5.Late Night (wknd),Film,Iliza Shlesinger: War Paint - Clean,60.00,57582.930000,4604.000000,959.715500,76.733333,12.507152,-628.567900,-0.395753,1588.283400,796.307142,2149.325947,1027.240853,-67.525353,-0.042515,Below
30474,CD10010178,2023-04-30 20:48:32.060,2023-04-30 22:15:57.040,2023-04-30 23:48:32.060,4-2023,2023,Q2,4,Apr,30,35,Sun,22,10pm-pt,1am-et,Weekend,Wknd,Late Night (wknd),5.Late Night (wknd),Film,Iliza Shlesinger: War Paint - Clean,15.00,4720.062500,880.750000,314.670833,58.716667,5.359140,-957.794250,-0.752708,1272.465083,616.690307,1706.957608,837.972559,-523.301726,-0.411250,Below
30475,CEG10027755,2023-04-30 22:15:57.040,2023-04-30 23:28:38.270,2023-05-01 01:15:57.040,4-2023,2023,Q2,4,Apr,30,35,Sun,22,10pm-pt,1am-et,Weekend,Wknd,Late Night (wknd),5.Late Night (wknd),Film,Doug Benson: Doug Dynasty - Clean,45.00,14160.187500,2642.250000,314.670833,58.716667,5.359140,-957.794250,-0.752708,1272.465083,616.690307,1706.957608,837.972559,-523.301726,-0.411250,Below
30476,CEG10027755,2023-04-30 22:15:57.040,2023-04-30 23:28:38.270,2023-05-01 01:15:57.040,4-2023,2023,Q2,4,Apr,30,35,Sun,23,11pm-pt,2am-et,Weekend,Wknd,Overnight (wknd),1.Overnight (wknd),Film,Doug Benson: Doug Dynasty - Clean,28.00,14295.031333,1387.400000,510.536833,49.550000,10.303468,-723.211250,-0.586190,1233.748083,625.353754,1674.344487,793.151680,-282.614847,-0.229070,Below


# Adding 3 Hour Avg Interval

In [118]:
# **********************************

# -Adding new kpi since an hour avg has less data than a 3 hour avg

# **********************************

In [119]:
Amagi.head(10)

,Date,Platform/Channel,Session Count,Total Viewership Minutes,Unique Viewers,Avg Session Count,Avg Session Duration Per Session,Avg Session Duration Per Viewer,Platform,Channel,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Hour,Day,Year,Month
9,2021-07-01,Cinedigm_ComedyDynamics_TCL,4,69.0,4.0,NaN,NaN,NaN,TCL,ComedyDynamics,NaN,NaN,NaN,NaN,0,1,2021,7
20,2021-07-01,comedydynamics,198,2244.0,186.0,NaN,NaN,NaN,Xumo(?),ComedyDynamics,NaN,NaN,NaN,NaN,0,1,2021,7
21,2021-07-01,ComedyDynamics Clean Samsung US,2387,31445.0,1611.0,NaN,NaN,NaN,Samsung,ComedyDynamics,NaN,NaN,NaN,NaN,0,1,2021,7
22,2021-07-01,comedydynamics_junction,1,0.0,1.0,NaN,NaN,NaN,Junction,ComedyDynamics,NaN,NaN,NaN,NaN,0,1,2021,7
23,2021-07-01,comedydynamics_littlstar,2,60.0,2.0,NaN,NaN,NaN,Littlstar,ComedyDynamics,NaN,NaN,NaN,NaN,0,1,2021,7
24,2021-07-01,comedydynamics_plex,10,100.0,10.0,NaN,NaN,NaN,Plex,ComedyDynamics,NaN,NaN,NaN,NaN,0,1,2021,7
25,2021-07-01,comedydynamicsroku,2,60.0,2.0,NaN,NaN,NaN,Roku,ComedyDynamics,NaN,NaN,NaN,NaN,0,1,2021,7
26,2021-07-01,Comedy Dynamics Roku US,1345,20029.0,1095.0,NaN,NaN,NaN,Roku,ComedyDynamics,NaN,NaN,NaN,NaN,0,1,2021,7
52,2021-07-01,Redbox ComedyDynamics,6,135.0,6.0,NaN,NaN,NaN,Redbox,ComedyDynamics,NaN,NaN,NaN,NaN,0,1,2021,7
61,2021-07-01,xumo-host-comedydynamics-vizio,3,180.0,3.0,NaN,NaN,NaN,Vizio,ComedyDynamics,NaN,NaN,NaN,NaN,0,1,2021,7


In [120]:
Avg3=Amagi.loc[:,:]
Avg3['Wn']=Avg3['Date'].dt.weekday

In [121]:
condweek=[
    (Avg3['Wn']== 0) | (Avg3['Wn']==1) | (Avg3['Wn']== 2) | (Avg3['Wn']==3) | (Avg3['Wn']== 4), 
    (Avg3['Wn']== 5) | (Avg3['Wn']==6) 
]
results=('Weekday','Weekend')

Avg3['week/weekend']=np.select(condweek,results, default=np.nan)

In [122]:
Avg3=Amagi.groupby(by=['Year','Month','week/weekend','Day','Hour'])['Total Viewership Minutes'].agg(sum).reset_index()

In [123]:
Avg3['Min/Min']=Avg3['Total Viewership Minutes']/60
Avg3=Avg3.loc[:,('Year','Month','week/weekend','Day','Hour','Min/Min')]

In [124]:
Avg3.head(10)

,Year,Month,week/weekend,Day,Hour,Min/Min
0,2021,7,Weekday,1,0,905.366667
1,2021,7,Weekday,1,1,884.000000
2,2021,7,Weekday,1,2,858.316667
3,2021,7,Weekday,1,3,801.733333
4,2021,7,Weekday,1,4,706.283333
5,2021,7,Weekday,1,5,684.316667
6,2021,7,Weekday,1,6,644.516667
7,2021,7,Weekday,1,7,656.466667
8,2021,7,Weekday,1,8,704.933333
9,2021,7,Weekday,1,9,760.300000


In [125]:
def Year_Month(y,m):
    #Weekdays
    def Mean_Wk(H):
        HourwM = Avg3.loc[((Avg3['Hour']== H-1) | (Avg3['Hour']== H) | (Avg3['Hour']== H+1)) & (Avg3['week/weekend']== 'Weekday') & (Avg3['Year']==y) & (Avg3['Month']==m), ('Min/Min')].mean()
        return HourwM

    def STD_Wk(H):
        HourwM = Avg3.loc[((Avg3['Hour']== H-1) | (Avg3['Hour']== H) | (Avg3['Hour']== H+1)) & (Avg3['week/weekend']== 'Weekday') & (Avg3['Year']==y) & (Avg3['Month']==m), ('Min/Min')].std()
        return HourwM

    def Cnt_Wk(H):
        HourwM = Avg3.loc[((Avg3['Hour']== H-1) | (Avg3['Hour']== H) | (Avg3['Hour']== H+1)) & (Avg3['week/weekend']== 'Weekday') & (Avg3['Year']==y) & (Avg3['Month']==m), ('Min/Min')].count()
        return HourwM

    #Weekends
    def Mean_Wknd(H):
        HourwM = Avg3.loc[((Avg3['Hour']== H-1) | (Avg3['Hour']== H) | (Avg3['Hour']== H+1)) & (Avg3['week/weekend']== 'Weekend') & (Avg3['Year']==y) & (Avg3['Month']==m), ('Min/Min')].mean()
        return HourwM

    def STD_Wknd(H):
        HourwM = Avg3.loc[((Avg3['Hour']== H-1) | (Avg3['Hour']== H) | (Avg3['Hour']== H+1)) & (Avg3['week/weekend']== 'Weekend') & (Avg3['Year']==y) & (Avg3['Month']==m), ('Min/Min')].std()
        return HourwM

    def Cnt_Wknd(H):
        HourwM = Avg3.loc[((Avg3['Hour']== H-1) | (Avg3['Hour']== H) | (Avg3['Hour']== H+1)) & (Avg3['week/weekend']== 'Weekend') & (Avg3['Year']==y) & (Avg3['Month']==m), ('Min/Min')].count()
        return HourwM

        #Exceptions___23 and 0 for Month 1 Year 2022_________________________________________________________

       #Weekdays
    Hour23wM = Avg3.loc[((Avg3['Hour']== 22) | (Avg3['Hour']== 23) | (Avg3['Hour']== 0)) & (Avg3['week/weekend']== 'Weekday') & (Avg3['Year']==y) & (Avg3['Month']==m), ('Min/Min')].mean()
    Hour23wS = Avg3.loc[((Avg3['Hour']== 22) | (Avg3['Hour']== 23) | (Avg3['Hour']== 0)) & (Avg3['week/weekend']== 'Weekday') & (Avg3['Year']==y) & (Avg3['Month']==m), ('Min/Min')].std()
    Hour23wC = Avg3.loc[((Avg3['Hour']== 22) | (Avg3['Hour']== 23) | (Avg3['Hour']== 0)) & (Avg3['week/weekend']== 'Weekday') & (Avg3['Year']==y) & (Avg3['Month']==m), ('Min/Min')].count()
       #Weekends
    Hour23wkM = Avg3.loc[((Avg3['Hour']== 22) | (Avg3['Hour']== 23) | (Avg3['Hour']== 0)) & (Avg3['week/weekend']== 'Weekend') & (Avg3['Year']==y) & (Avg3['Month']==m), ('Min/Min')].mean()
    Hour23wkS = Avg3.loc[((Avg3['Hour']== 22) | (Avg3['Hour']== 23) | (Avg3['Hour']== 0)) & (Avg3['week/weekend']== 'Weekend') & (Avg3['Year']==y) & (Avg3['Month']==m), ('Min/Min')].std()
    Hour23wkC = Avg3.loc[((Avg3['Hour']== 22) | (Avg3['Hour']== 23) | (Avg3['Hour']== 0)) & (Avg3['week/weekend']== 'Weekend') & (Avg3['Year']==y) & (Avg3['Month']==m), ('Min/Min')].count()

       #Weekdays
    Hour0wM = Avg3.loc[((Avg3['Hour']== 23) | (Avg3['Hour']== 0) | (Avg3['Hour']== 1)) & (Avg3['week/weekend']== 'Weekday') & (Avg3['Year']==y) & (Avg3['Month']==m), ('Min/Min')].mean()
    Hour0wS = Avg3.loc[((Avg3['Hour']== 23) | (Avg3['Hour']== 0) | (Avg3['Hour']== 1)) & (Avg3['week/weekend']== 'Weekday') & (Avg3['Year']==y) & (Avg3['Month']==m), ('Min/Min')].std()
    Hour0wC = Avg3.loc[((Avg3['Hour']== 23) | (Avg3['Hour']== 0) | (Avg3['Hour']== 1)) & (Avg3['week/weekend']== 'Weekday') & (Avg3['Year']==y) & (Avg3['Month']==m), ('Min/Min')].count()
       #Weekends
    Hour0wkM = Avg3.loc[((Avg3['Hour']== 23) | (Avg3['Hour']== 0) | (Avg3['Hour']== 1)) & (Avg3['week/weekend']== 'Weekend') & (Avg3['Year']==y) & (Avg3['Month']==m), ('Min/Min')].mean()
    Hour0wkS = Avg3.loc[((Avg3['Hour']== 23) | (Avg3['Hour']== 0) | (Avg3['Hour']== 1)) & (Avg3['week/weekend']== 'Weekend') & (Avg3['Year']==y) & (Avg3['Month']==m), ('Min/Min')].std()
    Hour0wkC = Avg3.loc[((Avg3['Hour']== 23) | (Avg3['Hour']== 0) | (Avg3['Hour']== 1)) & (Avg3['week/weekend']== 'Weekend') & (Avg3['Year']==y) & (Avg3['Month']==m), ('Min/Min')].count()
    
    #Creating TABLE________________________________________________________
    
    H3Data={'Year':[y,y,y,y,y,y,y,y,y,y,y,y,y,y,y,y,y,y,y,y,y,y,y,y,y,y,y,y,y,y,y,y,y,y,y,y,y,y,y,y,y,y,y,y,y,y,y,y],
        'Month':[m,m,m,m,m,m,m,m,m,m,m,m,m,m,m,m,m,m,m,m,m,m,m,m,m,m,m,m,m,m,m,m,m,m,m,m,m,m,m,m,m,m,m,m,m,m,m,m],
    
        'Weekdays/Weekends':['Weekday','Weekday','Weekday','Weekday','Weekday','Weekday','Weekday','Weekday','Weekday','Weekday','Weekday','Weekday','Weekday','Weekday','Weekday','Weekday',
                             'Weekday','Weekday','Weekday','Weekday','Weekday','Weekday','Weekday','Weekday',
                  'Weekend','Weekend','Weekend','Weekend','Weekend','Weekend','Weekend','Weekend','Weekend','Weekend','Weekend','Weekend','Weekend','Weekend','Weekend','Weekend','Weekend',
                             'Weekend','Weekend','Weekend','Weekend','Weekend','Weekend','Weekend'],
        
        'Hour(24-Pac)':[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,     0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23],
        
        'Mean (H3)':[Hour0wM,Mean_Wk(1),Mean_Wk(2),Mean_Wk(3),Mean_Wk(4),Mean_Wk(5),Mean_Wk(6),Mean_Wk(7),Mean_Wk(8),Mean_Wk(9),Mean_Wk(10),Mean_Wk(11),
                Mean_Wk(12),Mean_Wk(13),Mean_Wk(14),Mean_Wk(15),Mean_Wk(16),Mean_Wk(17),Mean_Wk(18),Mean_Wk(19),Mean_Wk(20),Mean_Wk(21),Mean_Wk(22),Hour23wM,
               
                Hour0wkM,Mean_Wknd(1),Mean_Wknd(2),Mean_Wknd(3),Mean_Wknd(4),Mean_Wknd(5),Mean_Wknd(6),Mean_Wknd(7),Mean_Wknd(8),Mean_Wknd(9),Mean_Wknd(10),Mean_Wknd(11),
                Mean_Wknd(12),Mean_Wknd(13),Mean_Wknd(14),Mean_Wknd(15),Mean_Wknd(16),Mean_Wknd(17),Mean_Wknd(18),Mean_Wknd(19),Mean_Wknd(20),Mean_Wknd(21),Mean_Wknd(22),Hour23wkM],
        
        
        'STD (H3)':[Hour0wS,STD_Wk(1),STD_Wk(2),STD_Wk(3),STD_Wk(4),STD_Wk(5),STD_Wk(6),STD_Wk(7),STD_Wk(8),STD_Wk(9),STD_Wk(10),STD_Wk(11),
                STD_Wk(12),STD_Wk(13),STD_Wk(14),STD_Wk(15),STD_Wk(16),STD_Wk(17),STD_Wk(18),STD_Wk(19),STD_Wk(20),STD_Wk(21),STD_Wk(22),Hour23wS,
               
                Hour0wkS,STD_Wknd(1),STD_Wknd(2),STD_Wknd(3),STD_Wknd(4),STD_Wknd(5),STD_Wknd(6),STD_Wknd(7),STD_Wknd(8),STD_Wknd(9),STD_Wknd(10),STD_Wknd(11),
                STD_Wknd(12),STD_Wknd(13),STD_Wknd(14),STD_Wknd(15),STD_Wknd(16),STD_Wknd(17),STD_Wknd(18),STD_Wknd(19),STD_Wknd(20),STD_Wknd(21),STD_Wknd(22),Hour23wkS],
        
        
        'Count (H3)':[Hour0wC,Cnt_Wk(1),Cnt_Wk(2),Cnt_Wk(3),Cnt_Wk(4),Cnt_Wk(5),Cnt_Wk(6),Cnt_Wk(7),Cnt_Wk(8),Cnt_Wk(9),Cnt_Wk(10),Cnt_Wk(11),
                Cnt_Wk(12),Cnt_Wk(13),Cnt_Wk(14),Cnt_Wk(15),Cnt_Wk(16),Cnt_Wk(17),Cnt_Wk(18),Cnt_Wk(19),Cnt_Wk(20),Cnt_Wk(21),Cnt_Wk(22),Hour23wC,
               
                Hour0wkC,Cnt_Wknd(1),Cnt_Wknd(2),Cnt_Wknd(3),Cnt_Wknd(4),Cnt_Wknd(5),Cnt_Wknd(6),Cnt_Wknd(7),Cnt_Wknd(8),Cnt_Wknd(9),Cnt_Wknd(10),Cnt_Wknd(11),
                Cnt_Wknd(12),Cnt_Wknd(13),Cnt_Wknd(14),Cnt_Wknd(15),Cnt_Wknd(16),Cnt_Wknd(17),Cnt_Wknd(18),Cnt_Wknd(19),Cnt_Wknd(20),Cnt_Wknd(21),Cnt_Wknd(22),Hour23wkC]
    }

    return pd.DataFrame(H3Data)


In [126]:
UY=Avg3['Year'].unique()
UM=Avg3['Month'].unique()

In [127]:
DataH3=pd.DataFrame(columns=['Year','Month','Weekdays/Weekends','Hour(24-Pac)','Mean (H3)','STD (H3)','Count (H3)'])
for i,j in [(i,j) for i in UY for j in UM]:
    H3=pd.DataFrame(Year_Month(i,j))
    DataH3=DataH3.append(H3)

In [128]:
DataH3 = DataH3.loc[DataH3['Count (H3)']!=0, :]

In [129]:
DataH3=pd.merge(DataH3,T_values, left_on='Count (H3)', right_on='Count', how='left')
DataH3['Z-value']= [1.96 if ele =='Weekday' else DataH3['Tvalue'] for ele in DataH3['Weekdays/Weekends']]

In [130]:
Cond33= [
    DataH3['Weekdays/Weekends']=='Weekday',
    DataH3['Weekdays/Weekends']=='Weekend'
]
Choices= [ DataH3['Z-value'], DataH3['Tvalue']]

DataH3['Value']= np.select(Cond33, Choices, default=np.nan)

In [131]:
DataH3['Upper CI(95%) (3H)']= DataH3['Mean (H3)']+(DataH3['Value']* (DataH3['STD (H3)']/(DataH3['Count (H3)']**(1/2))))
DataH3['Lower CI(95%) (3H)']= DataH3['Mean (H3)']-(DataH3['Value']* (DataH3['STD (H3)']/(DataH3['Count (H3)']**(1/2))))

In [132]:
DataH3= DataH3.loc[:,('Year','Month','Weekdays/Weekends','Hour(24-Pac)','Mean (H3)','STD (H3)','Count (H3)','Value','Upper CI(95%) (3H)','Lower CI(95%) (3H)')]

In [133]:
DF= pd.merge(DF, DataH3.loc[:,('Year','Month','Weekdays/Weekends','Hour(24-Pac)','Mean (H3)','Upper CI(95%) (3H)','Lower CI(95%) (3H)')], on=['Year','Month','Weekdays/Weekends','Hour(24-Pac)'], how='left')

In [134]:
Con13=[
    (DF['Minutes per Minute'] > DF['Upper CI(95%) (3H)']),
    (DF['Minutes per Minute'] < DF['Lower CI(95%) (3H)']),
    (DF['Minutes per Minute'] <= DF['Upper CI(95%) (3H)']) & (DF['Minutes per Minute'] >= DF['Lower CI(95%) (3H)'])     
]

Values13= ['Above','Below','Neutral']

DF['Above/Below/Neutral (3H)']=np.select(Con13,Values13, default=np.nan)

In [135]:
Con14=[
    (DF['Above/Below/Neutral (3H)']== 'Above'), (DF['Above/Below/Neutral (3H)']== 'Below'), (DF['Above/Below/Neutral (3H)']== 'Neutral'),
]

Values14=[(DF['Minutes per Minute']-DF['Upper CI(95%) (3H)']),
         (DF['Minutes per Minute']-DF['Lower CI(95%) (3H)']),
         0]
DF['Minutes Diff CI (3H)']= np.select(Con14,Values14, default=np.nan)

In [136]:
DF['%Minutes Diff CI (3H)']=(DF['Minutes Diff CI (3H)']/DF['Mean (H3)'])

In [137]:
DF['Upper CI(95%) (3H)']= DF['Upper CI(95%) (3H)'].astype(float)
DF['Lower CI(95%) (3H)']= DF['Lower CI(95%) (3H)'].astype(float)
DF['Minutes Diff CI (3H)']= DF['Minutes Diff CI (3H)'].astype(float)
DF['%Minutes Diff CI (3H)']= DF['%Minutes Diff CI (3H)'].astype(float)

# Daypart Average Using DF Table

In [138]:
DFdpavg=DF.loc[:,('Year','Month','Daypart','Minutes per Minute')]

In [139]:
DFdpavg= DFdpavg.groupby(by=['Year','Month','Daypart']).mean().reset_index()

In [140]:
DFdpavg.rename(columns={'Minutes per Minute':'Mean (Month DP)'},inplace=True)

In [141]:
DF= pd.merge(DF,DFdpavg, on=['Year','Month','Daypart'], how='left')

In [142]:
DF['%Minutes Diff (Month DP)']= (DF['Minutes per Minute']-DF['Mean (Month DP)'])/DF['Mean (Month DP)']

In [143]:
DF

,Content ID,TS,TS End,TS_East,M_Y,Year,Quarter,Month,Month(name),Day,Day (adj.),Weekday(name),Hour(24-Pac),Pacific_Time,East_Time,Weekdays/Weekends,Wk/Wknd,Daypart,Daypart#,Series/Film,Title,Length(min) in the Hour,Total Minutes in Hour,Total Sessions in Hour,Minutes per Minute,Sessions per Minute,Session Duration,Minutes Diff,%Minutes Diff,Hourly AVG,Hourly STD,Upper CI(95%),Lower CI(95%),Minutes Diff CI,%Minutes Diff CI,Above/Below/Neutral,Mean (H3),Upper CI(95%) (3H),Lower CI(95%) (3H),Above/Below/Neutral (3H),Minutes Diff CI (3H),%Minutes Diff CI (3H),Mean (Month DP),%Minutes Diff (Month DP)
0,CD10010210,2021-07-01 00:00:00.000,2021-07-01 01:05:41.080,2021-07-01 03:00:00.000,7-2021,2021,Q3,7,Jul,1,4,Thu,0,0am-pt,3am-et,Weekday,Wk,Overnight,1.Overnight,Film,Tom Green: Live! - Clean,60.00,54322.000000,3958.000000,905.366667,65.966667,13.724608,-89.343182,-0.089818,994.709848,235.542747,1098.861737,890.557960,0.000000,0.000000,Neutral,996.530808,1060.918947,932.142670,Below,-26.776003,-0.026869,920.147096,-0.016063
1,CD10010210,2021-07-01 00:00:00.000,2021-07-01 01:05:41.080,2021-07-01 03:00:00.000,7-2021,2021,Q3,7,Jul,1,4,Thu,1,1am-pt,4am-et,Weekday,Wk,Overnight,1.Overnight,Film,Tom Green: Live! - Clean,5.00,4420.000000,286.583333,884.000000,57.316667,15.423088,34.487879,0.040597,849.512121,190.435589,933.718604,765.305639,0.000000,0.000000,Neutral,872.644444,924.437635,820.851254,Neutral,0.000000,0.000000,920.147096,-0.039284
2,CD10010171,2021-07-01 01:05:41.080,2021-07-01 02:27:05.070,2021-07-01 04:05:41.080,7-2021,2021,Q3,7,Jul,1,4,Thu,1,1am-pt,4am-et,Weekday,Wk,Overnight,1.Overnight,Film,Dave Foley: Relatively Well - Clean,55.00,48620.000000,3152.416667,884.000000,57.316667,15.423088,34.487879,0.040597,849.512121,190.435589,933.718604,765.305639,0.000000,0.000000,Neutral,872.644444,924.437635,820.851254,Neutral,0.000000,0.000000,920.147096,-0.039284
3,CD10010171,2021-07-01 01:05:41.080,2021-07-01 02:27:05.070,2021-07-01 04:05:41.080,7-2021,2021,Q3,7,Jul,1,4,Thu,2,2am-pt,5am-et,Weekday,Wk,Overnight,1.Overnight,Film,Dave Foley: Relatively Well - Clean,27.00,23174.550000,1469.250000,858.316667,54.416667,15.773047,84.605303,0.109350,773.711364,156.460209,842.894671,704.528057,15.421996,0.019932,Above,782.280051,822.042347,742.517754,Above,36.274320,0.046370,920.147096,-0.067196
4,CEG10027749,2021-07-01 02:27:05.070,2021-07-01 03:45:00.130,2021-07-01 05:27:05.070,7-2021,2021,Q3,7,Jul,1,4,Thu,2,2am-pt,5am-et,Weekday,Wk,Overnight,1.Overnight,Film,Harland Williams: What A Treat - Clean,33.00,28324.450000,1795.750000,858.316667,54.416667,15.773047,84.605303,0.109350,773.711364,156.460209,842.894671,704.528057,15.421996,0.019932,Above,782.280051,822.042347,742.517754,Above,36.274320,0.046370,920.147096,-0.067196
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30473,CD10010178,2023-04-30 20:48:32.060,2023-04-30 22:15:57.040,2023-04-30 23:48:32.060,4-2023,2023,Q2,4,Apr,30,35,Sun,21,9pm-pt,0am-et,Weekend,Wknd,Late Night (wknd),5.Late Night (wknd),Film,Iliza Shlesinger: War Paint - Clean,60.00,57582.930000,4604.000000,959.715500,76.733333,12.507152,-628.567900,-0.395753,1588.283400,796.307142,2149.325947,1027.240853,-67.525353,-0.042515,Below,1547.187639,1841.108865,1253.266412,Below,-293.550912,-0.189732,1583.442613,-0.393906
30474,CD10010178,2023-04-30 20:48:32.060,2023-04-30 22:15:57.040,2023-04-30 23:48:32.060,4-2023,2023,Q2,4,Apr,30,35,Sun,22,10pm-pt,1am-et,Weekend,Wknd,Late Night (wknd),5.Late Night (wknd),Film,Iliza Shlesinger: War Paint - Clean,15.00,4720.062500,880.750000,314.670833,58.716667,5.359140,-957.794250,-0.752708,1272.465083,616.690307,1706.957608,837.972559,-523.301726,-0.411250,Below,1364.832189,1618.311152,1111.353226,Below,-796.682393,-0.583722,1583.442613,-0.801274
30475,CEG10027755,2023-04-30 22:15:57.040,2023-04-30 23:28:38.270,2023-05-01 01:15:57.040,4-2023,2023,Q2,4,Apr,30,35,S

# Adding TOTALs

In [144]:
TotS= DF.loc[:,('Title','Length(min) in the Hour','Total Minutes in Hour','Total Sessions in Hour')]
TotS=TotS.groupby('Title').agg(sum).reset_index()
TotS['Session Duration']= TotS['Total Minutes in Hour']/TotS['Total Sessions in Hour']

TotS.rename(columns={'Length(min) in the Hour':'Total Min Aired','Total Minutes in Hour':'Total Min Viewed','Total Sessions in Hour':'Total Sessions'}, inplace=True)

In [145]:
TotSD= DF.loc[:,('Title','Daypart','Length(min) in the Hour','Total Minutes in Hour','Total Sessions in Hour')]
TotSD=TotSD.groupby(by=['Title','Daypart']).agg(sum).reset_index()
TotSD['Session Duration']= TotSD['Total Minutes in Hour']/TotSD['Total Sessions in Hour']

TotSD.rename(columns={'Length(min) in the Hour':'Total Min Aired','Total Minutes in Hour':'Total Min Viewed','Total Sessions in Hour':'Total Sessions'}, inplace=True)

In [146]:
# ***************************

# Most of the rest of this data builds out the excel report from the DF table

# ***************************

# Content Summary

In [147]:
CS=DF.loc[:,('Title','Series/Film','Above/Below/Neutral (3H)','Content ID')]

In [148]:
CS=CS.pivot_table(index= ['Series/Film','Title'], values='Content ID', columns='Above/Below/Neutral (3H)', aggfunc=pd.Series.count)
CS=CS.reset_index()

In [149]:
CS['Above']=CS['Above'].fillna(0)
CS['Below']=CS['Below'].fillna(0)
CS['Neutral']=CS['Neutral'].fillna(0)


CS['Count']= (CS['Above']+ CS['Below']+ CS['Neutral'])
CS['%Below CI']= CS['Below']/CS['Count']
CS['%Inside CI']= CS['Neutral']/CS['Count']
CS['%Above CI']= CS['Above']/CS['Count']
CS['Score']= ((CS['Above']**2)-(CS['Below']**2))/CS['Count']

In [150]:
CS1=DF.loc[:,('Title','Series/Film','%Minutes Diff CI','%Minutes Diff CI (3H)','%Minutes Diff (Month DP)','Minutes per Minute')]
CS1= CS1.groupby('Title').mean().reset_index()
CSsumm= pd.merge(CS,CS1, on='Title', how='left')

In [151]:
#Combined Score

CondSC=[
    (CSsumm['Score']> 0) & (CSsumm['%Minutes Diff CI (3H)']> 0),
     (CSsumm['Score']> 0) & (CSsumm['%Minutes Diff CI (3H)']< 0),
     (CSsumm['Score']== 0) | (CSsumm['%Minutes Diff CI (3H)']== 0),
     (CSsumm['Score']< 0) & (CSsumm['%Minutes Diff CI (3H)']< 0)   
]

ValuesSC=[CSsumm['Score']*CSsumm['%Minutes Diff CI (3H)'],
         0,
         0,
         (CSsumm['Score']*CSsumm['%Minutes Diff CI (3H)'])*-1
        ]

CSsumm['Combined Score']= np.select(CondSC,ValuesSC)

In [152]:
CSsumm= pd.merge(CSsumm,TotS, on='Title', how='left')

In [153]:
CSsumm= CSsumm.loc[:,('Series/Film','Title','Count','Above','Neutral','Below','Minutes per Minute','%Minutes Diff CI (3H)','%Minutes Diff (Month DP)','Combined Score',
                      'Total Min Aired','Total Min Viewed','Total Sessions','Session Duration')].sort_values('Combined Score', ascending=False)

In [154]:
CSsumm.head(10)

,Series/Film,Title,Count,Above,Neutral,Below,Minutes per Minute,%Minutes Diff CI (3H),%Minutes Diff (Month DP),Combined Score,Total Min Aired,Total Min Viewed,Total Sessions,Session Duration
304,Film,Jeff Dunham: Controlled Chaos - Clean,299.0,273.0,20.0,6.0,1578.728560,0.552614,0.735616,137.678613,12021.0,1.950768e+07,2.340669e+06,8.334234
306,Film,Jeff Dunham: Spark of Insanity - Clean,246.0,219.0,13.0,14.0,1631.650814,0.517283,0.681905,100.439122,9514.0,1.593396e+07,1.893043e+06,8.417116
305,Film,Jeff Dunham: Minding The Monsters - Clean,272.0,234.0,21.0,17.0,1366.145091,0.446896,0.608443,89.489236,9716.0,1.366279e+07,1.867219e+06,7.317190
303,Film,Jeff Dunham: Arguing with Myself - Clean,191.0,170.0,12.0,9.0,1510.757789,0.522578,0.699031,78.849126,6875.0,1.078737e+07,1.343441e+06,8.029659
302,Film,Jeff Dunham: All Over The Map - Clean,218.0,187.0,19.0,12.0,1492.934242,0.480398,0.630980,76.742492,8250.0,1.246009e+07,1.585596e+06,7.858301
307,Film,Jeff Dunham: Unhinged In Hollywood,190.0,164.0,15.0,11.0,1455.296241,0.453021,0.630262,63.840142,6523.0,9.657666e+06,1.198251e+06,8.059802
308,Film,Jeff Dunham: Very Special Christmas Special - ...,150.0,124.0,19.0,7.0,1338.666597,0.481820,0.633022,49.232374,5942.0,8.204462e+06,9.934587e+05,8.258483
324,Film,Jim Gaffigan: Quality Time,247.0,178.0,47.0,22.0,1353.931935,0.249358,0.354462,31.497830,8667.0,1.207816e+07,1.787308e+06,6.757741
323,Film,Jim Gaffigan: Noble Ape,173.0,140.0,27.0,6.0,1423.987688,0.240314,0.370817,27.176342,6264.0,8.981145e+06,1.206498e+06,7.443979
559,Film,Sinbad: Where U Been? - Clean,188.0,109.0,54.0,25.0,1047.343900,0.128881,0.205550,7.716435,7608.0,8.235154e+06,1.171569e+06,7.029167


# Content Summary with Dayparts

In [155]:
CSD=DF.loc[:,('Title','Daypart','Series/Film','Above/Below/Neutral (3H)','Content ID')]

In [156]:
CSD=CSD.pivot_table(index= ['Series/Film','Title','Daypart'], values='Content ID', columns='Above/Below/Neutral (3H)', aggfunc=pd.Series.count)
CSD=CSD.reset_index()

In [157]:
CSD['Above']=CSD['Above'].fillna(0)
CSD['Below']=CSD['Below'].fillna(0)
CSD['Neutral']=CSD['Neutral'].fillna(0)


CSD['Count']= (CSD['Above']+ CSD['Below']+ CSD['Neutral'])
CSD['%Below CI']= CSD['Below']/CSD['Count']
CSD['%Inside CI']= CSD['Neutral']/CSD['Count']
CSD['%Above CI']= CSD['Above']/CSD['Count']
CSD['Score']= ((CSD['Above']**2)-(CSD['Below']**2))/CSD['Count']

In [158]:
CSD1=DF.loc[:,('Title','Series/Film','Daypart','Minutes per Minute','%Minutes Diff CI (3H)','%Minutes Diff (Month DP)')]
CSD1= CSD1.groupby(['Title','Daypart']).mean().reset_index()
CSDsumm= pd.merge(CSD,CSD1, on=['Title','Daypart'], how='left')

In [159]:
CondSCD=[
    (CSDsumm['Score']> 0) & (CSDsumm['%Minutes Diff CI (3H)']> 0),
     (CSDsumm['Score']> 0) & (CSDsumm['%Minutes Diff CI (3H)']< 0),
     (CSDsumm['Score']== 0) | (CSDsumm['%Minutes Diff CI (3H)']== 0),
     (CSDsumm['Score']< 0) & (CSDsumm['%Minutes Diff CI (3H)']< 0)   
]

ValuesSCD=[CSDsumm['Score']*CSDsumm['%Minutes Diff CI (3H)'],
         0,
         0,
         (CSDsumm['Score']*CSDsumm['%Minutes Diff CI (3H)'])*-1
        ]

CSDsumm['Combined Score']= np.select(CondSCD,ValuesSCD)

In [160]:
CSDsumm= pd.merge(CSDsumm,TotSD, on=['Title','Daypart'], how='left')

In [161]:
CSDsumm=CSDsumm.loc[:,('Series/Film','Title','Daypart','Count','Above','Neutral','Below','Minutes per Minute','%Minutes Diff CI (3H)','%Minutes Diff (Month DP)','Combined Score',
                      'Total Min Aired','Total Min Viewed','Total Sessions','Session Duration')]

# Monthly Summary

In [162]:
DF

,Content ID,TS,TS End,TS_East,M_Y,Year,Quarter,Month,Month(name),Day,Day (adj.),Weekday(name),Hour(24-Pac),Pacific_Time,East_Time,Weekdays/Weekends,Wk/Wknd,Daypart,Daypart#,Series/Film,Title,Length(min) in the Hour,Total Minutes in Hour,Total Sessions in Hour,Minutes per Minute,Sessions per Minute,Session Duration,Minutes Diff,%Minutes Diff,Hourly AVG,Hourly STD,Upper CI(95%),Lower CI(95%),Minutes Diff CI,%Minutes Diff CI,Above/Below/Neutral,Mean (H3),Upper CI(95%) (3H),Lower CI(95%) (3H),Above/Below/Neutral (3H),Minutes Diff CI (3H),%Minutes Diff CI (3H),Mean (Month DP),%Minutes Diff (Month DP)
0,CD10010210,2021-07-01 00:00:00.000,2021-07-01 01:05:41.080,2021-07-01 03:00:00.000,7-2021,2021,Q3,7,Jul,1,4,Thu,0,0am-pt,3am-et,Weekday,Wk,Overnight,1.Overnight,Film,Tom Green: Live! - Clean,60.00,54322.000000,3958.000000,905.366667,65.966667,13.724608,-89.343182,-0.089818,994.709848,235.542747,1098.861737,890.557960,0.000000,0.000000,Neutral,996.530808,1060.918947,932.142670,Below,-26.776003,-0.026869,920.147096,-0.016063
1,CD10010210,2021-07-01 00:00:00.000,2021-07-01 01:05:41.080,2021-07-01 03:00:00.000,7-2021,2021,Q3,7,Jul,1,4,Thu,1,1am-pt,4am-et,Weekday,Wk,Overnight,1.Overnight,Film,Tom Green: Live! - Clean,5.00,4420.000000,286.583333,884.000000,57.316667,15.423088,34.487879,0.040597,849.512121,190.435589,933.718604,765.305639,0.000000,0.000000,Neutral,872.644444,924.437635,820.851254,Neutral,0.000000,0.000000,920.147096,-0.039284
2,CD10010171,2021-07-01 01:05:41.080,2021-07-01 02:27:05.070,2021-07-01 04:05:41.080,7-2021,2021,Q3,7,Jul,1,4,Thu,1,1am-pt,4am-et,Weekday,Wk,Overnight,1.Overnight,Film,Dave Foley: Relatively Well - Clean,55.00,48620.000000,3152.416667,884.000000,57.316667,15.423088,34.487879,0.040597,849.512121,190.435589,933.718604,765.305639,0.000000,0.000000,Neutral,872.644444,924.437635,820.851254,Neutral,0.000000,0.000000,920.147096,-0.039284
3,CD10010171,2021-07-01 01:05:41.080,2021-07-01 02:27:05.070,2021-07-01 04:05:41.080,7-2021,2021,Q3,7,Jul,1,4,Thu,2,2am-pt,5am-et,Weekday,Wk,Overnight,1.Overnight,Film,Dave Foley: Relatively Well - Clean,27.00,23174.550000,1469.250000,858.316667,54.416667,15.773047,84.605303,0.109350,773.711364,156.460209,842.894671,704.528057,15.421996,0.019932,Above,782.280051,822.042347,742.517754,Above,36.274320,0.046370,920.147096,-0.067196
4,CEG10027749,2021-07-01 02:27:05.070,2021-07-01 03:45:00.130,2021-07-01 05:27:05.070,7-2021,2021,Q3,7,Jul,1,4,Thu,2,2am-pt,5am-et,Weekday,Wk,Overnight,1.Overnight,Film,Harland Williams: What A Treat - Clean,33.00,28324.450000,1795.750000,858.316667,54.416667,15.773047,84.605303,0.109350,773.711364,156.460209,842.894671,704.528057,15.421996,0.019932,Above,782.280051,822.042347,742.517754,Above,36.274320,0.046370,920.147096,-0.067196
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30473,CD10010178,2023-04-30 20:48:32.060,2023-04-30 22:15:57.040,2023-04-30 23:48:32.060,4-2023,2023,Q2,4,Apr,30,35,Sun,21,9pm-pt,0am-et,Weekend,Wknd,Late Night (wknd),5.Late Night (wknd),Film,Iliza Shlesinger: War Paint - Clean,60.00,57582.930000,4604.000000,959.715500,76.733333,12.507152,-628.567900,-0.395753,1588.283400,796.307142,2149.325947,1027.240853,-67.525353,-0.042515,Below,1547.187639,1841.108865,1253.266412,Below,-293.550912,-0.189732,1583.442613,-0.393906
30474,CD10010178,2023-04-30 20:48:32.060,2023-04-30 22:15:57.040,2023-04-30 23:48:32.060,4-2023,2023,Q2,4,Apr,30,35,Sun,22,10pm-pt,1am-et,Weekend,Wknd,Late Night (wknd),5.Late Night (wknd),Film,Iliza Shlesinger: War Paint - Clean,15.00,4720.062500,880.750000,314.670833,58.716667,5.359140,-957.794250,-0.752708,1272.465083,616.690307,1706.957608,837.972559,-523.301726,-0.411250,Below,1364.832189,1618.311152,1111.353226,Below,-796.682393,-0.583722,1583.442613,-0.801274
30475,CEG10027755,2023-04-30 22:15:57.040,2023-04-30 23:28:38.270,2023-05-01 01:15:57.040,4-2023,2023,Q2,4,Apr,30,35,S

In [163]:
#Add %Minutes Diff CI (3H)

MS= DF.loc[:,('TS','Title','Series/Film','%Minutes Diff CI (3H)')]
MS['Date']=MS['TS'].dt.strftime('%m-%y')
MS= MS.groupby(by=['Date','Title','Series/Film'])[['%Minutes Diff CI (3H)']].mean().reset_index()
MS= MS.pivot(index=['Series/Film','Title'], columns='Date', values=['%Minutes Diff CI (3H)']).reset_index()
MS=MS.droplevel(level=0,axis=1)
MS.columns.values[0]='Series/Film'
MS.columns.values[1]='Title'
MS['Metric']='%Minutes Diff CI (3H)'


In [164]:
# #Add Counts

# MSc= DF.loc[:,('TS','Title','Series/Film','Content ID')]
# MSc['Date']=MSc['TS'].dt.strftime('%m-%y')
# MSc=MSc.pivot_table(index= ['Series/Film','Title'], values='Content ID', columns='Date', aggfunc=pd.Series.count).reset_index()
# MSc['Metric']='Count'


In [165]:
#Add Minutes Aired

MSc= DF.loc[:,('TS','Title','Series/Film','Length(min) in the Hour')]
MSc['Date']=MSc['TS'].dt.strftime('%m-%y')
MSc=MSc.groupby(by=['Date','Title','Series/Film'])[['Length(min) in the Hour']].agg(sum).reset_index()
MSc=MSc.pivot_table(index= ['Series/Film','Title'], values='Length(min) in the Hour', columns='Date').reset_index()
MSc['Metric']='Minutes Aired'

In [166]:
#Add Minutes per Minute

MM= DF.loc[:,('TS','Title','Series/Film','Minutes per Minute')]
MM['Date']=MM['TS'].dt.strftime('%m-%y')
MM= MM.groupby(by=['Date','Title','Series/Film'])[['Minutes per Minute']].mean().reset_index()
MM=MM.pivot_table(index= ['Series/Film','Title'], values='Minutes per Minute', columns='Date').reset_index()
MM['Metric']='Min/Min'


In [167]:
#Add Daypart avg benchmark

MSd= DF.loc[:,('TS','Title','Series/Film','%Minutes Diff (Month DP)')]
MSd['Date']=MSd['TS'].dt.strftime('%m-%y')
MSd= MSd.groupby(by=['Date','Title','Series/Film'])[['%Minutes Diff (Month DP)']].mean().reset_index()
MSd=MSd.pivot_table(index= ['Series/Film','Title'], values='%Minutes Diff (Month DP)', columns='Date').reset_index()
MSd['Metric']='%Minutes Diff (Month DP)'


In [168]:
#Concat Tables

MS_MSc=pd.concat([MS,MSc,MM])
MS_MSc['Average']=MS_MSc.mean(axis=1)


In [169]:
MS_MSc= MS_MSc.reset_index()

In [170]:
if (Channel == 'CONtv' or  Channel == 'ComedyDynamics' or Channel == 'Docurama' or Channel == 'Dove'):   
    MS_MSc=MS_MSc.loc[:,('Series/Film','Title','07-21','08-21','09-21','10-21','11-21','12-21','01-22','02-22','03-22','04-22','05-22','06-22','07-22','08-22','09-22','10-22','11-22','12-22',
                         '01-23','02-23','03-23','04-23','Metric','Average')]

elif (Channel == 'SoReal'):
    MS_MSc=MS_MSc.loc[:,('Series/Film','Title','10-21','11-21','12-21','01-22','02-22','03-22','04-22','05-22','06-22','07-22','08-22','09-22','10-22','11-22','12-22',
                         '01-23','02-23','03-23','04-23','Metric','Average')]
elif (Channel == 'ElRey'):
    MS_MSc=MS_MSc.loc[:,('Series/Film','Title','11-21','12-21','01-22','02-22','03-22','04-22','05-22','06-22','07-22','08-22','09-22','10-22','11-22','12-22',
                         '01-23','02-23','03-23','04-23','Metric','Average')]
elif (Channel == 'BloodyDisgusting' or Channel == 'Towie'):
    MS_MSc=MS_MSc.loc[:,('Series/Film','Title','01-22','02-22','03-22','04-22','05-22','06-22','07-22','08-22','09-22','10-22','11-22','12-22',
                         '01-23','02-23','03-23','04-23','Metric','Average')]
elif (Channel == 'FilmDetective'):
    MS_MSc=MS_MSc.loc[:,('Series/Film','Title','10-22','11-22','12-22','01-23','02-23','03-23','04-23','Metric','Average')]
    
elif (Channel == 'RetroCrush'):
    MS_MSc=MS_MSc.loc[:,('Series/Film','Title',
                         '01-23','02-23','03-23','04-23','Metric','Average')]
    
else:
    # Towie
    MS_MSc=MS_MSc.loc[:,('Series/Film','Title','08-22','09-22','10-22','11-22','12-22',
                         '01-23','02-23','03-23','04-23','Metric','Average')]
    

In [171]:
MS_MSc['Count']=MS_MSc.loc[:,MS_MSc.columns[2]:MS_MSc.columns[-3]].count(axis=1)
X_mean=[]

for i in range(0,MS_MSc['Count'].count()):
    X = statistics.mean(list(range(1,MS_MSc['Count'].iloc[i]+1)))
    X_mean = X_mean + [X]

MS_MSc['X_mean']= X_mean

In [172]:
Slope = []
B1e = []
B2e = []
S =[]

for i in range(0,MS_MSc['Count'].count()):
    x = list(range(1, MS_MSc['Count'].iloc[i]+1))
    y = [x for x in list(MS_MSc.loc[i,MS_MSc.columns[2]:MS_MSc.columns[-5]]) if math.isnan(x) == False]
    for u,v in zip(x,y):
        B1 = ((u - MS_MSc['X_mean'].iloc[i]) * (v - MS_MSc['Average'].iloc[i]))
        B2 = ((u - MS_MSc['X_mean'].iloc[i])**2)
        
        B1e = B1e + [B1]
        B2e = B2e + [B2]
        
    S = S + [(sum(B1e)/sum(B2e))]
    B1e.clear()
    B2e.clear()
    
    
MS_MSc['Slope'] = S


<ipython-input-172-eb4776767459>:16: RuntimeWarning: invalid value encountered in double_scalars
  S = S + [(sum(B1e)/sum(B2e))]


In [173]:
if (Channel == 'CONtv' or  Channel == 'ComedyDynamics' or Channel == 'Docurama' or Channel == 'Dove'):   
    MS_MSc=MS_MSc.loc[:,('Series/Film','Title','07-21','08-21','09-21','10-21','11-21','12-21','01-22','02-22','03-22','04-22','05-22','06-22','07-22','08-22','09-22','10-22','11-22','12-22',
                         '01-23','02-23','03-23','04-23','Metric','Average','Slope')]

elif (Channel == 'SoReal'):
    MS_MSc=MS_MSc.loc[:,('Series/Film','Title','10-21','11-21','12-21','01-22','02-22','03-22','04-22','05-22','06-22','07-22','08-22','09-22','10-22','11-22','12-22',
                         '01-23','02-23','03-23','04-23','Metric','Average','Slope')]
elif (Channel == 'ElRey'):
    MS_MSc=MS_MSc.loc[:,('Series/Film','Title','11-21','12-21','01-22','02-22','03-22','04-22','05-22','06-22','07-22','08-22','09-22','10-22','11-22','12-22',
                         '01-23','02-23','03-23','04-23','Metric','Average','Slope')]
    
elif (Channel == 'BloodyDisgusting' or Channel == 'Towie'):
    MS_MSc=MS_MSc.loc[:,('Series/Film','Title','01-22','02-22','03-22','04-22','05-22','06-22','07-22','08-22','09-22','10-22','11-22','12-22',
                         '01-23','02-23','03-23','04-23','Metric','Average','Slope')]
elif (Channel == 'FilmDetective'):
    MS_MSc=MS_MSc.loc[:,('Series/Film','Title','10-22','11-22','12-22',
                             '01-23','02-23','03-23','04-23','Metric','Average','Slope')]
    
elif (Channel == 'RetroCrush'):
    MS_MSc=MS_MSc.loc[:,('Series/Film','Title',
                         '01-23','02-23','03-23','04-23','Metric','Average','Slope')] 

else:
    MS_MSc=MS_MSc.loc[:,('Series/Film','Title','08-22','09-22','10-22','11-22','12-22',
                         '01-23','02-23','03-23','04-23','Metric','Average','Slope')]
    
    

# Monthly Summary by Daypart

In [174]:
#Add %Minutes Diff CI

MS1= DF.loc[:,('TS','Daypart','Title','Series/Film','%Minutes Diff CI (3H)')]
MS1['Date']=MS1['TS'].dt.strftime('%m-%y')
MS1= MS1.groupby(by=['Date','Title','Daypart','Series/Film'])[['%Minutes Diff CI (3H)']].mean().reset_index()
MS1= MS1.pivot(index=['Series/Film','Title','Daypart'], columns='Date', values=['%Minutes Diff CI (3H)']).reset_index()
MS1=MS1.droplevel(level=0,axis=1)
MS1.columns.values[0]='Series/Film'
MS1.columns.values[1]='Title'
MS1.columns.values[2]='Daypart'
MS1['Metric']='%Minutes Diff CI (3H)'


In [175]:
# #Add Counts

# MSc1= DF.loc[:,('TS','Daypart','Title','Series/Film','Content ID')]
# MSc1['Date']=MSc1['TS'].dt.strftime('%m-%y')
# MSc1=MSc1.pivot_table(index= ['Series/Film','Title','Daypart'], values='Content ID', columns='Date', aggfunc=pd.Series.count).reset_index()
# MSc1['Metric']='Count'


In [176]:
#Add Minutes Aired

MSc1= DF.loc[:,('TS','Daypart','Title','Series/Film','Length(min) in the Hour')]
MSc1['Date']=MSc1['TS'].dt.strftime('%m-%y')
MSc1=MSc1.groupby(by=['Date','Title','Daypart','Series/Film'])[['Length(min) in the Hour']].agg(sum).reset_index()
MSc1=MSc1.pivot_table(index= ['Series/Film','Title','Daypart'], values='Length(min) in the Hour', columns='Date').reset_index()
MSc1['Metric']='Minutes Aired'

In [177]:
#Add Minutes per Minute

MM1= DF.loc[:,('TS','Daypart','Title','Series/Film','Minutes per Minute')]
MM1['Date']=MM1['TS'].dt.strftime('%m-%y')
MM1= MM1.groupby(by=['Date','Title','Daypart','Series/Film'])[['Minutes per Minute']].mean().reset_index()
MM1= MM1.pivot(index=['Series/Film','Title','Daypart'], columns='Date', values='Minutes per Minute').reset_index()
MM1['Metric']='Min/Min'


In [178]:
#Add %Minutes Diff (Month DP)

MS1dp= DF.loc[:,('TS','Daypart','Title','Series/Film','%Minutes Diff (Month DP)')]
MS1dp['Date']=MS1dp['TS'].dt.strftime('%m-%y')
MS1dp= MS1dp.groupby(by=['Date','Title','Daypart','Series/Film'])[['%Minutes Diff (Month DP)']].mean().reset_index()
MS1dp= MS1dp.pivot(index=['Series/Film','Title','Daypart'], columns='Date', values='%Minutes Diff (Month DP)').reset_index()
MS1dp['Metric']='%Minutes Diff (Month DP)'

In [179]:
#Concat Tables

MS1_MSc1=pd.concat([MS1,MSc1,MM1])
MS1_MSc1['Average']=MS1_MSc1.mean(axis=1)
MS1_MSc1.reset_index(drop=True, inplace=True)

In [180]:
MS1_MSc1= MS1_MSc1.reset_index()

In [181]:
if (Channel == 'CONtv' or  Channel == 'ComedyDynamics' or Channel == 'Docurama' or Channel == 'Dove'):   
    MS1_MSc1=MS1_MSc1.loc[:,('Series/Film','Title','Daypart','07-21','08-21','09-21','10-21','11-21','12-21','01-22','02-22','03-22','04-22','05-22','06-22','07-22','08-22','09-22','10-22','11-22','12-22',
                             '01-23','02-23','03-23','04-23','Metric','Average')]

elif (Channel == 'SoReal'):
    MS1_MSc1=MS1_MSc1.loc[:,('Series/Film','Title','Daypart','10-21','11-21','12-21','01-22','02-22','03-22','04-22','05-22','06-22','07-22','08-22','09-22','10-22','11-22','12-22',
                             '01-23','02-23','03-23','04-23','Metric','Average')]
elif (Channel == 'ElRey'):
    MS1_MSc1=MS1_MSc1.loc[:,('Series/Film','Title','Daypart','11-21','12-21','01-22','02-22','03-22','04-22','05-22','06-22','07-22','08-22','09-22','10-22','11-22','12-22',
                             '01-23','02-23','03-23','04-23','Metric','Average')]
    
elif (Channel == 'BloodyDisgusting' or Channel == 'Towie'):
    MS1_MSc1=MS1_MSc1.loc[:,('Series/Film','Title','Daypart','01-22','02-22','03-22','04-22','05-22','06-22','07-22','08-22','09-22','10-22','11-22','12-22',
                             '01-23','02-23','03-23','04-23','Metric','Average')]
elif (Channel == 'FilmDetective'):
    MS1_MSc1=MS1_MSc1.loc[:,('Series/Film','Title','Daypart','10-22','11-22','12-22',
                             '01-23','02-23','03-23','04-23','Metric','Average')]
    
elif (Channel == 'RetroCrush'):
    MS1_MSc1=MS1_MSc1.loc[:,('Series/Film','Title','Daypart',
                             '01-23','02-23','03-23','04-23','Metric','Average')]    
    
else:
    MS1_MSc1=MS1_MSc1.loc[:,('Series/Film','Title','Daypart','08-22','09-22','10-22','11-22','12-22',
                             '01-23','02-23','03-23','04-23','Metric','Average')]

In [182]:
MS1_MSc1['Count']=MS1_MSc1.loc[:,MS1_MSc1.columns[3]:MS1_MSc1.columns[-3]].count(axis=1)
X_mean=[]

for i in range(0,MS1_MSc1['Count'].count()):
    X = statistics.mean(list(range(1,MS1_MSc1['Count'].iloc[i]+1)))
    X_mean = X_mean + [X]

MS1_MSc1['X_mean']= X_mean

In [183]:
Slope = []
B1e = []
B2e = []
S =[]

for i in range(0,MS1_MSc1['Count'].count()):
    x = list(range(1, MS1_MSc1['Count'].iloc[i]+1))
    y = [x for x in list(MS1_MSc1.loc[i,MS1_MSc1.columns[3]:MS1_MSc1.columns[-5]]) if math.isnan(x) == False]
    for u,v in zip(x,y):
        B1 = ((u - MS1_MSc1['X_mean'].iloc[i]) * (v - MS1_MSc1['Average'].iloc[i]))
        B2 = ((u - MS1_MSc1['X_mean'].iloc[i])**2)
        
        B1e = B1e + [B1]
        B2e = B2e + [B2]
        
    S = S + [(sum(B1e)/sum(B2e))]
    B1e.clear()
    B2e.clear()
    
    
MS1_MSc1['Slope'] = S


<ipython-input-183-344369c2615a>:16: RuntimeWarning: invalid value encountered in double_scalars
  S = S + [(sum(B1e)/sum(B2e))]


In [184]:
if (Channel == 'CONtv' or  Channel == 'ComedyDynamics' or Channel == 'Docurama' or Channel == 'Dove'):   
    MS1_MSc1=MS1_MSc1.loc[:,('Series/Film','Title','Daypart','07-21','08-21','09-21','10-21','11-21','12-21','01-22','02-22','03-22','04-22','05-22','06-22','07-22','08-22','09-22','10-22','11-22','12-22',
                             '01-23','02-23','03-23','04-23','Metric','Average','Slope')]

elif (Channel == 'SoReal'):
    MS1_MSc1=MS1_MSc1.loc[:,('Series/Film','Title','Daypart','10-21','11-21','12-21','01-22','02-22','03-22','04-22','05-22','06-22','07-22','08-22','09-22','10-22','11-22','12-22',
                             '01-23','02-23','03-23','04-23','Metric','Average','Slope')]
elif (Channel == 'ElRey'):
    MS1_MSc1=MS1_MSc1.loc[:,('Series/Film','Title','Daypart','11-21','12-21','01-22','02-22','03-22','04-22','05-22','06-22','07-22','08-22','09-22','10-22','11-22','12-22',
                             '01-23','02-23','03-23','04-23','Metric','Average','Slope')]
    
elif (Channel == 'BloodyDisgusting' or Channel == 'Towie'):
    MS1_MSc1=MS1_MSc1.loc[:,('Series/Film','Title','Daypart','01-22','02-22','03-22','04-22','05-22','06-22','07-22','08-22','09-22','10-22','11-22','12-22',
                             '01-23','02-23','03-23','04-23','Metric','Average','Slope')]
elif (Channel == 'FilmDetective'):
    MS1_MSc1=MS1_MSc1.loc[:,('Series/Film','Title','Daypart','10-22','11-22','12-22',
                             '01-23','02-23','03-23','04-23','Metric','Average','Slope')]
    
elif (Channel == 'RetroCrush'):
    MS1_MSc1=MS1_MSc1.loc[:,('Series/Film','Title','Daypart',
                             '01-23','02-23','03-23','04-23','Metric','Average','Slope')]    
    
else:
    MS1_MSc1=MS1_MSc1.loc[:,('Series/Film','Title','Daypart','08-22','09-22','10-22','11-22','12-22',
                             '01-23','02-23','03-23','04-23','Metric','Average','Slope')]


# Platform Monthly Total Minutes

In [185]:
# ********************************************

# I Dont really use this information anymore

# ********************************************

In [186]:
Platform['Year']=Platform['Date'].dt.year

In [187]:
if Channel == 'ElRey':
    Platform = Platform.loc[~((Platform['Year']==2021) & (Platform['Month']==10))]

In [188]:
Platform

,Date,Platform/Channel,Session Count,Total Viewership Minutes,Unique Viewers,Avg Session Count,Avg Session Duration Per Session,Avg Session Duration Per Viewer,Platform,Channel,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Hour,Day,Year,Month,Wn,week/weekend
9,2021-07-01 00:00:00,Cinedigm_ComedyDynamics_TCL,4,69.0,4.0,NaN,NaN,NaN,TCL,ComedyDynamics,NaN,NaN,NaN,NaN,0,1,2021,7,3,Weekday
20,2021-07-01 00:00:00,comedydynamics,198,2244.0,186.0,NaN,NaN,NaN,Xumo(?),ComedyDynamics,NaN,NaN,NaN,NaN,0,1,2021,7,3,Weekday
21,2021-07-01 00:00:00,ComedyDynamics Clean Samsung US,2387,31445.0,1611.0,NaN,NaN,NaN,Samsung,ComedyDynamics,NaN,NaN,NaN,NaN,0,1,2021,7,3,Weekday
22,2021-07-01 00:00:00,comedydynamics_junction,1,0.0,1.0,NaN,NaN,NaN,Junction,ComedyDynamics,NaN,NaN,NaN,NaN,0,1,2021,7,3,Weekday
23,2021-07-01 00:00:00,comedydynamics_littlstar,2,60.0,2.0,NaN,NaN,NaN,Littlstar,ComedyDynamics,NaN,NaN,NaN,NaN,0,1,2021,7,3,Weekday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
337,2023-04-30 19:00:00,NaN,3165,36723.0,NaN,NaN,NaN,NaN,Roku,ComedyDynamics,NaN,NaN,NaN,NaN,19,30,2023,4,6,Weekend
338,2023-04-30 20:00:00,NaN,3449,43404.0,NaN,NaN,NaN,NaN,Roku,ComedyDynamics,NaN,NaN,NaN,NaN,20,30,2023,4,6,Weekend
339,2023-04-30 21:00:00,NaN,3180,47687.4,NaN,NaN,NaN,NaN,Roku,ComedyDynamics,NaN,NaN,NaN,NaN,21,30,2023,4,6,Weekend
340,2023-04-30 22:00:00,NaN,2341,10322.4,NaN,NaN,NaN,NaN,Roku,ComedyDynamics,NaN,NaN,NaN,NaN,22,30,2023,4,6,Weekend


In [189]:
CondMonth=[
    Platform['Month'] == 1,Platform['Month'] == 2,Platform['Month'] == 3,
    Platform['Month'] == 4,Platform['Month'] == 5,Platform['Month'] == 6,
    Platform['Month'] == 7,Platform['Month'] == 8,Platform['Month'] == 9,
    Platform['Month'] == 10,Platform['Month'] == 11,Platform['Month'] == 12,
]

ValueM=['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']

Platform['Months']= np.select(CondMonth,ValueM)

In [190]:
Platform= Platform.pivot_table(index='Platform',values='Total Viewership Minutes',columns=['Year','Months'], aggfunc=pd.Series.sum).reset_index()

In [191]:
Platform

Year     Platform        2021                                       \
Months                    Aug          Dec         Jul         Nov   
0       Cineverse         NaN          NaN         NaN         NaN   
1        Junction      3703.0      3604.40      6470.0      2709.0   
2              LG         NaN          NaN         NaN         NaN   
3       Littlstar      2727.0      4955.89      2048.0      2589.0   
4            Plex    202239.0          NaN    183721.0         NaN   
5         Rakuten     37291.0     40381.02     38083.0     35063.0   
6          Redbox    190543.0    195095.15    166737.0    272159.0   
7            Roku  12975688.0  12853141.19  14124777.0  10358973.0   
8         Samsung  24587028.0  18869836.42  26076769.0  16816135.0   
9        SelectTv       584.0          NaN       353.0         NaN   
10            TCL     27800.0     16461.41     23164.0     38840.0   
11           Vewd        53.0       981.76       123.0       709.0   
12          Vizio   1596673.0   1027423.54    889993.0    957964.0   
13         Xiaomi         NaN          NaN         NaN         NaN   
14        Xumo(?)   2277979.0   3013434.91   1951331.0   2289158.0   

Year                                   2022                            \
Months         Oct         Sep          Apr          Aug          Dec   
0              NaN         NaN          NaN          NaN      2459.77   
1            876.0      1679.0      6764.44     12821.51      1451.59   
2              NaN         NaN   2565071.21   2134497.68   4143228.98   
3           1287.0      2812.0      1127.38       975.48          NaN   
4              NaN         NaN          NaN    169396.32          NaN   
5          35967.0     36168.0          NaN          NaN          NaN   
6         165241.0    160746.0    270628.45    210052.66    241698.53   
7        9356430.0   8207238.0  14766176.97          NaN          NaN   
8       17951769.0  15931380.0  19861883.07  23306343.16  23313857.31   
9              NaN      1025.0          NaN          NaN          NaN   
10         38139.0     26695.0     21047.08         0.59      6899.25   
11          1572.0         2.0      1638.63      3867.81       314.92   
12       1243996.0    719615.0   1342533.64   1144153.69   2282810.12   
13             NaN         NaN      4134.54      8024.89     29105.48   
14       2545318.0   2170995.0    417397.72    232128.53    620032.41   

Year                                                                     \
Months          Feb          Jan          Jul          Jun          Mar   
0               NaN          NaN          NaN          NaN          NaN   
1           1967.32      1633.62      6162.83      3901.70      6410.23   
2        4114860.83          NaN   2416242.54   2837574.36   1470717.42   
3           1397.98      2171.36       603.71      1190.12      1754.08   
4               NaN          NaN    163337.44          NaN          NaN   
5          38578.77     43163.01          NaN          NaN     13301.77   
6         514174.96    491417.68    185778.82    276362.52    265941.20   
7       11947071.47  14536228.44          NaN          NaN  11158198.45   
8       17006277.44  18571018.32  26537673.25  26535547.07  16621062.95   
9               NaN          NaN          NaN          NaN          NaN   
10         15530.09     16005.96      9126.94     29068.95     19757.69   
11           997.10       161.11      1377.74      3104.37      1819.00   
12        893944.52   1143363.95   1348892.76   1423045.79    932640.83   
13           305.08       266.57      6843.92      2010.80      2201.19   
14       2271326.04   3089020.17    325562.05    426632.95    631647.75   

Year                                                               2023  \
Months          May          Nov          Oct          Sep          Apr   
0               NaN          NaN          NaN          NaN      5250.44   
1           4206.08      4337.08       666.75      2459.85      34

In [192]:
if (Channel == 'CONtv' or  Channel == 'ComedyDynamics' or Channel == 'Docurama' or Channel == 'Dove'):   
    Platform.columns=('Platform','Aug-21','Dec-21','Jul-21','Nov-21','Oct-21','Sep-21','Apr-22','Aug-22','Dec-22','Feb-22','Jan-22','Jul-22','Jun-22','Mar-22','May-22','Nov-22','Oct-22','Sep-22',
                      'Apr-23','Feb-23','Jan-23','Mar-23')
    Platform=Platform.loc[:,('Platform','Jul-21','Aug-21','Sep-21','Oct-21','Nov-21','Dec-21','Jan-22','Feb-22','Mar-22','Apr-22','May-22','Jun-22','Jul-22','Aug-22','Sep-22','Oct-22','Nov-22','Dec-22',
                             'Jan-23','Feb-23','Mar-23','Apr-23')]

elif (Channel == 'SoReal'):
    Platform.columns=('Platform','Dec-21','Nov-21','Oct-21','Apr-22','Aug-22','Dec-22','Feb-22','Jan-22','Jul-22','Jun-22','Mar-22','May-22','Nov-22','Oct-22','Sep-22',
                             'Apr-23','Feb-23','Jan-23','Mar-23')
    Platform=Platform.loc[:,('Platform','Oct-21','Nov-21','Dec-21','Jan-22','Feb-22','Mar-22','Apr-22','May-22','Jun-22','Jul-22','Aug-22','Sep-22','Oct-22','Nov-22','Dec-22',
                             'Jan-23','Feb-23','Mar-23','Apr-23')]
elif (Channel == 'ElRey'):
    Platform.columns=('Platform','Dec-21','Nov-21','Apr-22','Aug-22','Dec-22','Feb-22','Jan-22','Jul-22','Jun-22','Mar-22','May-22','Nov-22','Oct-22','Sep-22',
                             'Apr-23','Feb-23','Jan-23','Mar-23')
    Platform=Platform.loc[:,('Platform','Nov-21','Dec-21','Jan-22','Feb-22','Mar-22','Apr-22','May-22','Jun-22','Jul-22','Aug-22','Sep-22','Oct-22','Nov-22','Dec-22',
                             'Jan-23','Feb-23','Mar-23','Apr-23')]
    
elif (Channel == 'BloodyDisgusting' or Channel == 'Towie'):
    Platform.columns=('Platform','Apr-22','Aug-22','Dec-22','Feb-22','Jan-22','Jul-22','Jun-22','Mar-22','May-22','Nov-22','Oct-22','Sep-22',
                             'Apr-23','Feb-23','Jan-23','Mar-23')
    Platform=Platform.loc[:,('Platform','Jan-22','Feb-22','Mar-22','Apr-22','May-22','Jun-22','Jul-22','Aug-22','Sep-22','Oct-22','Nov-22','Dec-22',
                             'Jan-23','Feb-23','Mar-23','Apr-23')]
    
elif (Channel == 'FilmDetective'):
    Platform.columns=('Platform','Aug-22','Dec-22','Nov-22','Oct-22','Sep-22',
                             'Apr-23','Feb-23','Jan-23','Mar-23')
    Platform=Platform.loc[:,('Platform','Aug-22','Sep-22','Oct-22','Nov-22','Dec-22',
                             'Jan-23','Feb-23','Mar-23','Apr-23')]
    
elif (Channel == 'RetroCrush'):
    Platform.columns=('Platform',
                             'Apr-23','Feb-23','Jan-23','Mar-23')
    Platform=Platform.loc[:,('Platform',
                             'Jan-23','Feb-23','Mar-23','Apr-23')]    
    
else:
    Platform.columns=('Platform','Aug-22','Dec-22','Nov-22','Oct-22','Sep-22',
                             'Apr-23','Feb-23','Jan-23','Mar-23')
    Platform=Platform.loc[:,('Platform','Aug-22','Sep-22','Oct-22','Nov-22','Dec-22',
                             'Jan-23','Feb-23','Mar-23','Apr-23')]
    

# Total Minutes Per Month for each Content

In [193]:
# *****************************

# This Creates the %Minutes Table

# ***************************

In [194]:
TotChnMonMin=DF.groupby(['Year','Month'])['Total Minutes in Hour'].agg(sum).reset_index()

In [195]:
TotMinC= DF.groupby(by=['Year','Month','Series/Film','Title'])['Total Minutes in Hour'].agg(sum).reset_index()

In [196]:
TotMinC=pd.merge(TotMinC, TotChnMonMin, on=['Year','Month'], how='left')

In [197]:
TotMinC.rename(columns={'Total Minutes in Hour_x':'Tot Content Month Min','Total Minutes in Hour_y':'Tot Chn Month Min'}, inplace=True)

In [198]:
# Percent Content minutes from total monthly Minutes

TotMinC['% of Mon Min']= TotMinC['Tot Content Month Min']/TotMinC['Tot Chn Month Min']

In [199]:
TotMinCP= TotMinC.pivot(index=['Series/Film','Title'], columns=['Year','Month'], values='% of Mon Min').reset_index()

In [200]:
TotMinCP.head()

Year  Series/Film                                Title 2021            \
Month                                                     7         8   
0            Film                     #IMomSoHard Live  NaN  0.005667   
1            Film         (Romance) In The Digital Age  NaN  0.005943   
2            Film                         1/2 New Year  NaN  0.006084   
3            Film  2 In The Bush: A Love Story - Clean  NaN       NaN   
4            Film                              25 Sets  NaN  0.005541   

Year                                         2022                          \
Month         9        10  11        12         1         2         3   4   
0      0.003481  0.010598 NaN       NaN       NaN  0.003650  0.006213 NaN   
1      0.003037  0.004902 NaN       NaN       NaN  0.001436       NaN NaN   
2      0.007460       NaN NaN  0.003511  0.002329       NaN       NaN NaN   
3      0.004224  0.009018 NaN       NaN       NaN  0.006315  0.004195 NaN   
4      0.007903  0.006275 NaN       NaN  0.006572       NaN       NaN NaN   

Year                                        2023                    
Month         5   6   7   8   9  10  11  12    1         2   3   4  
0      0.001001 NaN NaN NaN NaN NaN NaN NaN  NaN       NaN NaN NaN  
1           NaN NaN NaN NaN NaN NaN NaN NaN  NaN  0.001144 NaN NaN  
2           NaN NaN NaN NaN NaN NaN NaN NaN  NaN       NaN NaN NaN  
3           NaN NaN NaN NaN NaN NaN NaN NaN  NaN       NaN NaN NaN  
4           NaN NaN NaN NaN NaN NaN NaN NaN  NaN       NaN NaN NaN

In [201]:
# _____________________________________________RENAME COLUMNS CAREFUl____________________________________


if (Channel == 'CONtv' or  Channel == 'ComedyDynamics' or Channel == 'Docurama' or Channel == 'Dove'):   
    TotMinCP.columns=['Series/Film','Title','7-2021','8-2021','9-2021','10-2021','11-2021','12-2021','1-2022','2-2022','3-2022','4-2022','5-2022','6-2022','7-2022','8-2022','9-2022','10-2022','11-2022','12-2022',
                     '1-2023','2-2023','3-2023','4-2023']
    TotMinCP['Average']= TotMinCP.loc[:,'7-2021':'4-2023'].mean(axis=1)
    
elif (Channel == 'SoReal'):
    TotMinCP.columns=['Series/Film','Title','10-2021','11-2021','12-2021','1-2022','2-2022','3-2022','4-2022','5-2022','6-2022','7-2022','8-2022','9-2022','10-2022','11-2022','12-2022',
                     '1-2023','2-2023','3-2023','4-2023']
    TotMinCP['Average']= TotMinCP.loc[:,'10-2021':'4-2023'].mean(axis=1)
elif (Channel == 'ElRey'):
    TotMinCP.columns=['Series/Film','Title','11-2021','12-2021','1-2022','2-2022','3-2022','4-2022','5-2022','6-2022','7-2022','8-2022','9-2022','10-2022','11-2022','12-2022',
                     '1-2023','2-2023','3-2023','4-2023']
    TotMinCP['Average']= TotMinCP.loc[:,'11-2021':'4-2023'].mean(axis=1)
    
elif (Channel == 'BloodyDisgusting' or Channel == 'Towie'):
    TotMinCP.columns=['Series/Film','Title','1-2022','2-2022','3-2022','4-2022','5-2022','6-2022','7-2022','8-2022','9-2022','10-2022','11-2022','12-2022',
                     '1-2023','2-2023','3-2023','4-2023']
    TotMinCP['Average']= TotMinCP.loc[:,'1-2022':'4-2023'].mean(axis=1)
    
elif (Channel == 'FilmDetective'):
    TotMinCP.columns=['Series/Film','Title','10-2022','11-2022','12-2022',
                     '1-2023','2-2023','3-2023','4-2023']
    TotMinCP['Average']= TotMinCP.loc[:,'10-2022':'4-2023'].mean(axis=1)

elif (Channel == 'RetroCrush'):
    TotMinCP.columns=['Series/Film','Title',
                     '1-2023','2-2023','3-2023','4-2023']
    TotMinCP['Average']= TotMinCP.loc[:,'1-2023':'4-2023'].mean(axis=1)    
    
else:
    TotMinCP.columns=['Series/Film','Title','8-2022','9-2022','10-2022','11-2022','12-2022',
                     '1-2023','2-2023','3-2023','4-2023']
    TotMinCP['Average']= TotMinCP.loc[:,'8-2022':'4-2023'].mean(axis=1)
    


In [202]:
if Timezone == 'East':
    DF.rename(columns={'Hour(24-Pac)':'Hour(24-East)'},inplace=True)

# Last Look

In [203]:
DF

,Content ID,TS,TS End,TS_East,M_Y,Year,Quarter,Month,Month(name),Day,Day (adj.),Weekday(name),Hour(24-Pac),Pacific_Time,East_Time,Weekdays/Weekends,Wk/Wknd,Daypart,Daypart#,Series/Film,Title,Length(min) in the Hour,Total Minutes in Hour,Total Sessions in Hour,Minutes per Minute,Sessions per Minute,Session Duration,Minutes Diff,%Minutes Diff,Hourly AVG,Hourly STD,Upper CI(95%),Lower CI(95%),Minutes Diff CI,%Minutes Diff CI,Above/Below/Neutral,Mean (H3),Upper CI(95%) (3H),Lower CI(95%) (3H),Above/Below/Neutral (3H),Minutes Diff CI (3H),%Minutes Diff CI (3H),Mean (Month DP),%Minutes Diff (Month DP)
0,CD10010210,2021-07-01 00:00:00.000,2021-07-01 01:05:41.080,2021-07-01 03:00:00.000,7-2021,2021,Q3,7,Jul,1,4,Thu,0,0am-pt,3am-et,Weekday,Wk,Overnight,1.Overnight,Film,Tom Green: Live! - Clean,60.00,54322.000000,3958.000000,905.366667,65.966667,13.724608,-89.343182,-0.089818,994.709848,235.542747,1098.861737,890.557960,0.000000,0.000000,Neutral,996.530808,1060.918947,932.142670,Below,-26.776003,-0.026869,920.147096,-0.016063
1,CD10010210,2021-07-01 00:00:00.000,2021-07-01 01:05:41.080,2021-07-01 03:00:00.000,7-2021,2021,Q3,7,Jul,1,4,Thu,1,1am-pt,4am-et,Weekday,Wk,Overnight,1.Overnight,Film,Tom Green: Live! - Clean,5.00,4420.000000,286.583333,884.000000,57.316667,15.423088,34.487879,0.040597,849.512121,190.435589,933.718604,765.305639,0.000000,0.000000,Neutral,872.644444,924.437635,820.851254,Neutral,0.000000,0.000000,920.147096,-0.039284
2,CD10010171,2021-07-01 01:05:41.080,2021-07-01 02:27:05.070,2021-07-01 04:05:41.080,7-2021,2021,Q3,7,Jul,1,4,Thu,1,1am-pt,4am-et,Weekday,Wk,Overnight,1.Overnight,Film,Dave Foley: Relatively Well - Clean,55.00,48620.000000,3152.416667,884.000000,57.316667,15.423088,34.487879,0.040597,849.512121,190.435589,933.718604,765.305639,0.000000,0.000000,Neutral,872.644444,924.437635,820.851254,Neutral,0.000000,0.000000,920.147096,-0.039284
3,CD10010171,2021-07-01 01:05:41.080,2021-07-01 02:27:05.070,2021-07-01 04:05:41.080,7-2021,2021,Q3,7,Jul,1,4,Thu,2,2am-pt,5am-et,Weekday,Wk,Overnight,1.Overnight,Film,Dave Foley: Relatively Well - Clean,27.00,23174.550000,1469.250000,858.316667,54.416667,15.773047,84.605303,0.109350,773.711364,156.460209,842.894671,704.528057,15.421996,0.019932,Above,782.280051,822.042347,742.517754,Above,36.274320,0.046370,920.147096,-0.067196
4,CEG10027749,2021-07-01 02:27:05.070,2021-07-01 03:45:00.130,2021-07-01 05:27:05.070,7-2021,2021,Q3,7,Jul,1,4,Thu,2,2am-pt,5am-et,Weekday,Wk,Overnight,1.Overnight,Film,Harland Williams: What A Treat - Clean,33.00,28324.450000,1795.750000,858.316667,54.416667,15.773047,84.605303,0.109350,773.711364,156.460209,842.894671,704.528057,15.421996,0.019932,Above,782.280051,822.042347,742.517754,Above,36.274320,0.046370,920.147096,-0.067196
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30473,CD10010178,2023-04-30 20:48:32.060,2023-04-30 22:15:57.040,2023-04-30 23:48:32.060,4-2023,2023,Q2,4,Apr,30,35,Sun,21,9pm-pt,0am-et,Weekend,Wknd,Late Night (wknd),5.Late Night (wknd),Film,Iliza Shlesinger: War Paint - Clean,60.00,57582.930000,4604.000000,959.715500,76.733333,12.507152,-628.567900,-0.395753,1588.283400,796.307142,2149.325947,1027.240853,-67.525353,-0.042515,Below,1547.187639,1841.108865,1253.266412,Below,-293.550912,-0.189732,1583.442613,-0.393906
30474,CD10010178,2023-04-30 20:48:32.060,2023-04-30 22:15:57.040,2023-04-30 23:48:32.060,4-2023,2023,Q2,4,Apr,30,35,Sun,22,10pm-pt,1am-et,Weekend,Wknd,Late Night (wknd),5.Late Night (wknd),Film,Iliza Shlesinger: War Paint - Clean,15.00,4720.062500,880.750000,314.670833,58.716667,5.359140,-957.794250,-0.752708,1272.465083,616.690307,1706.957608,837.972559,-523.301726,-0.411250,Below,1364.832189,1618.311152,1111.353226,Below,-796.682393,-0.583722,1583.442613,-0.801274
30475,CEG10027755,2023-04-30 22:15:57.040,2023-04-30 23:28:38.270,2023-05-01 01:15:57.040,4-2023,2023,Q2,4,Apr,30,35,S

In [204]:
CSsumm.head()

,Series/Film,Title,Count,Above,Neutral,Below,Minutes per Minute,%Minutes Diff CI (3H),%Minutes Diff (Month DP),Combined Score,Total Min Aired,Total Min Viewed,Total Sessions,Session Duration
304,Film,Jeff Dunham: Controlled Chaos - Clean,299.0,273.0,20.0,6.0,1578.728560,0.552614,0.735616,137.678613,12021.0,1.950768e+07,2.340669e+06,8.334234
306,Film,Jeff Dunham: Spark of Insanity - Clean,246.0,219.0,13.0,14.0,1631.650814,0.517283,0.681905,100.439122,9514.0,1.593396e+07,1.893043e+06,8.417116
305,Film,Jeff Dunham: Minding The Monsters - Clean,272.0,234.0,21.0,17.0,1366.145091,0.446896,0.608443,89.489236,9716.0,1.366279e+07,1.867219e+06,7.317190
303,Film,Jeff Dunham: Arguing with Myself - Clean,191.0,170.0,12.0,9.0,1510.757789,0.522578,0.699031,78.849126,6875.0,1.078737e+07,1.343441e+06,8.029659
302,Film,Jeff Dunham: All Over The Map - Clean,218.0,187.0,19.0,12.0,1492.934242,0.480398,0.630980,76.742492,8250.0,1.246009e+07,1.585596e+06,7.858301


In [205]:
CSDsumm

,Series/Film,Title,Daypart,Count,Above,Neutral,Below,Minutes per Minute,%Minutes Diff CI (3H),%Minutes Diff (Month DP),Combined Score,Total Min Aired,Total Min Viewed,Total Sessions,Session Duration
0,Film,#IMomSoHard Live,Late Night,9.0,1.0,0.0,8.0,786.873704,-0.120471,-0.183555,-0.843294,251.0,196367.797333,39434.016667,4.979655
1,Film,#IMomSoHard Live,Late Night (wknd),1.0,0.0,1.0,0.0,995.883333,0.000000,-0.031912,0.000000,12.0,11950.600000,1723.600000,6.933511
2,Film,#IMomSoHard Live,Overnight,3.0,2.0,0.0,1.0,714.457056,0.051902,0.139057,0.051902,123.0,94022.013500,11823.200000,7.952332
3,Film,#IMomSoHard Live,Overnight (wknd),2.0,0.0,2.0,0.0,735.775000,0.000000,0.135371,0.000000,63.0,51747.450000,6302.450000,8.210688
4,Film,#IMomSoHard Live,Prime,10.0,0.0,0.0,10.0,943.467267,-0.125899,-0.228563,-1.258987,346.0,323939.031333,77944.466667,4.156023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5310,Series,"The Guest List_The Sklar Brothers, Mike Lawren...",Prime,1.0,0.0,0.0,1.0,517.804333,-0.304241,-0.404338,-0.304241,24.0,12427.304000,3250.000000,3.823786
5311,Series,"The Guest List_The Sklar Brothers, Mike Lawren...",Weekday Fringe,1.0,0.0,0.0,1.0,508.628500,-0.314502,-0.366594,-0.314502,56.0,28483.196000,6624.800000,4.299480
5312,Series,"The Guest List_The Sklar Brothers, Mike Lawren...",Weekend Day,2.0,0.0,0.0,2.0,626.937417,-0.236259,-0.269759,-0.472518,76.0,48131.769167,10555.583333,4.559840
5313,Series,"The Guest List_Todd Glass, Jesse Joyce, KC Aro...",Prime (wknd),2.0,0.0,0.0,2.0,494.607667,-0.206864,-0.374060,-0.413728,67.0,34213.246167,8623.950000,3.967236


In [206]:
MS_MSc.head(15)

Date,Series/Film,Title,07-21,08-21,09-21,10-21,11-21,12-21,01-22,02-22,03-22,04-22,05-22,06-22,07-22,08-22,09-22,10-22,11-22,12-22,01-23,02-23,03-23,04-23,Metric,Average,Slope
0,Film,#IMomSoHard Live,NaN,-0.054406,-0.035171,-0.045840,NaN,NaN,NaN,-0.137763,-0.042864,NaN,-0.387395,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,%Minutes Diff CI (3H),-0.117240,-0.050856
1,Film,(Romance) In The Digital Age,NaN,-0.129224,-0.080506,-0.144975,NaN,NaN,NaN,-0.153482,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.350484,NaN,NaN,%Minutes Diff CI (3H),-0.171734,-0.051550
2,Film,1/2 New Year,NaN,-0.164416,-0.007104,NaN,NaN,-0.104279,-0.142986,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,%Minutes Diff CI (3H),-0.104696,-0.003289
3,Film,2 In The Bush: A Love Story - Clean,NaN,NaN,-0.143945,-0.099864,NaN,NaN,NaN,-0.190057,-0.004209,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,%Minutes Diff CI (3H),-0.109519,0.032901
4,Film,25 Sets,NaN,-0.101187,-0.039978,-0.025054,NaN,NaN,-0.074513,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,%Minutes Diff CI (3H),-0.060183,0.009495
5,Film,37-Teen,-0.115386,-0.198875,-0.107179,-0.029881,NaN,NaN,NaN,-0.118743,0.033725,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,%Minutes Diff CI (3H),-0.089390,0.030379
6,Film,8989 Redstone,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.383840,-0.133082,NaN,NaN,NaN,0.024756,NaN,%Minutes Diff CI (3H),-0.164056,0.204298
7,Film,Aaron Karo: The Rest Is History,-0.030021,NaN,NaN,NaN,NaN,-0.040773,-0.183518,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,%Minutes Diff CI (3H),-0.084771,-0.076748
8,Film,Aaron Woodall: Sad and Needy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.049770,0.038897,0.517982,-0.036195,NaN,NaN,NaN,-0.069755,NaN,0.018899,-0.182541,NaN,NaN,-0.135796,NaN,%Minutes Diff CI (3H),0.012715,-0.038574
9,Film,Aaron Woodall: Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.059853,NaN,-0.075759,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,%Minutes Diff CI (3H),-0.067806,-0.015906


In [207]:
MS1_MSc1

Date,Series/Film,Title,Daypart,07-21,08-21,09-21,10-21,11-21,12-21,01-22,02-22,03-22,04-22,05-22,06-22,07-22,08-22,09-22,10-22,11-22,12-22,01-23,02-23,03-23,04-23,Metric,Average,Slope
0,Film,#IMomSoHard Live,Late Night,NaN,NaN,-0.060798,0.073896,NaN,NaN,NaN,-0.151397,-0.282821,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,%Minutes Diff CI (3H),-0.105280,-0.089136
1,Film,#IMomSoHard Live,Late Night (wknd),NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,%Minutes Diff CI (3H),0.000000,NaN
2,Film,#IMomSoHard Live,Overnight,NaN,NaN,NaN,0.071241,NaN,NaN,NaN,NaN,0.013223,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,%Minutes Diff CI (3H),0.042232,-0.058018
3,Film,#IMomSoHard Live,Overnight (wknd),NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,%Minutes Diff CI (3H),0.000000,NaN
4,Film,#IMomSoHard Live,Prime,NaN,-0.066085,-0.079884,-0.189965,NaN,NaN,NaN,-0.110494,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,%Minutes Diff CI (3H),-0.111607,-0.024331
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15940,Series,"The Guest List_The Sklar Brothers, Mike Lawren...",Prime,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,517.804333,NaN,NaN,NaN,NaN,Min/Min,517.804333,NaN
15941,Series,"The Guest List_The Sklar Brothers, Mike Lawren...",Weekday Fringe,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,508.628500,NaN,NaN,NaN,NaN,Min/Min,508.628500,NaN
15942,Series,"The Guest List_The Sklar Brothers, Mike Lawren...",Weekend Day,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,626.937417,NaN,NaN,NaN,Min/Min,626.937417,NaN
15943,Series,"The Guest List_Todd Glass, Jesse Joyce, KC Aro...",Prime (wknd),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,494.607667,NaN,NaN,NaN,NaN,Min/Min,494.607667,NaN


In [208]:
Platform

,Platform,Jul-21,Aug-21,Sep-21,Oct-21,Nov-21,Dec-21,Jan-22,Feb-22,Mar-22,Apr-22,May-22,Jun-22,Jul-22,Aug-22,Sep-22,Oct-22,Nov-22,Dec-22,Jan-23,Feb-23,Mar-23,Apr-23
0,Cineverse,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2459.77,2739.35,6852.46,17827.12,5250.44
1,Junction,6470.0,3703.0,1679.0,876.0,2709.0,3604.40,1633.62,1967.32,6410.23,6764.44,4206.08,3901.70,6162.83,12821.51,2459.85,666.75,4337.08,1451.59,1273.82,2191.75,1463.11,3428.67
2,LG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4114860.83,1470717.42,2565071.21,2723094.21,2837574.36,2416242.54,2134497.68,2405495.14,2704962.42,2310828.76,4143228.98,5211272.81,4050145.46,4399094.12,4542062.85
3,Littlstar,2048.0,2727.0,2812.0,1287.0,2589.0,4955.89,2171.36,1397.98,1754.08,1127.38,2015.79,1190.12,603.71,975.48,2024.62,1729.40,NaN,NaN,NaN,NaN,NaN,NaN
4,Plex,183721.0,202239.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,163337.44,169396.32,190117.05,NaN,184052.06,NaN,NaN,NaN,NaN,NaN
5,Rakuten,38083.0,37291.0,36168.0,35967.0,35063.0,40381.02,43163.01,38578.77,13301.77,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Redbox,166737.0,190543.0,160746.0,165241.0,272159.0,195095.15,491417.68,514174.96,265941.20,270628.45,242982.05,276362.52,185778.82,210052.66,221094.71,218140.37,248732.80,241698.53,238529.38,188889.05,223545.04,216692.77
7,Roku,14124777.0,12975688.0,8207238.0,9356430.0,10358973.0,12853141.19,14536228.44,11947071.47,11158198.45,14766176.97,19219945.26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20293909.80,19934032.20
8,Samsung,26076769.0,24587028.0,15931380.0,17951769.0,16816135.0,18869836.42,18571018.32,17006277.44,16621062.95,19861883.07,25836584.05,26535547.07,26537673.25,23306343.16,25307136.32,24696284.78,22950804.07,23313857.31,24816188.12,22470012.70,25387944.27,21663744.11
9,SelectTv,353.0,584.0,1025.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [209]:
TotMinCP.head(10)

,Series/Film,Title,7-2021,8-2021,9-2021,10-2021,11-2021,12-2021,1-2022,2-2022,3-2022,4-2022,5-2022,6-2022,7-2022,8-2022,9-2022,10-2022,11-2022,12-2022,1-2023,2-2023,3-2023,4-2023,Average
0,Film,#IMomSoHard Live,NaN,0.005667,0.003481,0.010598,NaN,NaN,NaN,0.003650,0.006213,NaN,0.001001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.005102
1,Film,(Romance) In The Digital Age,NaN,0.005943,0.003037,0.004902,NaN,NaN,NaN,0.001436,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.001144,NaN,NaN,0.003292
2,Film,1/2 New Year,NaN,0.006084,0.007460,NaN,NaN,0.003511,0.002329,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.004846
3,Film,2 In The Bush: A Love Story - Clean,NaN,NaN,0.004224,0.009018,NaN,NaN,NaN,0.006315,0.004195,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.005938
4,Film,25 Sets,NaN,0.005541,0.007903,0.006275,NaN,NaN,0.006572,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.006573
5,Film,37-Teen,0.015117,0.007112,0.005106,0.001994,NaN,NaN,NaN,0.006250,0.006703,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.007047
6,Film,8989 Redstone,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.001265,0.001551,NaN,NaN,NaN,0.002144,NaN,0.001653
7,Film,Aaron Karo: The Rest Is History,0.007879,NaN,NaN,NaN,NaN,0.003823,0.005371,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.005691
8,Film,Aaron Woodall: Sad and Needy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.004301,0.002309,0.001250,0.000800,NaN,NaN,NaN,0.001029,NaN,0.001554,0.000768,NaN,NaN,0.000851,NaN,0.001608
9,Film,Aaron Woodall: Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.004426,NaN,0.002151,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.003289


In [210]:
output= pd.ExcelWriter(Channel+ ' Content Full Summary out ' + Date_span + '.xlsx', engine='xlsxwriter')

DF.to_excel(output, sheet_name= 'DF',index=False)
CSsumm.to_excel(output, sheet_name= 'Cont Summ',index=False)
CSDsumm.to_excel(output, sheet_name='Cont Summ DayP', index=False)
MS_MSc.to_excel(output, sheet_name='Monthly Trend', index=False)
MS1_MSc1.to_excel(output, sheet_name='Monthly DayP Trend',index=False)
Platform.to_excel(output, sheet_name='Graphs',index=False)
TotMinCP.to_excel(output, sheet_name='% Month Min',index=False)
#Bucks.to_excel(output, sheet_name='Bucket',index=False)



output.save()